In [2]:

# Example based on your log:
# Ensure this matches the ABFSS path you copied from your Lakehouse Properties
# up to the /Files/ part.
base_lakehouse_abfss_path = "abfss://COE_F_EUC_P2@onelake.dfs.fabric.microsoft.com/AimsDataFromBlob2025.Lakehouse/Files/"

# Now construct output_path_for_temp_files relative to this base
output_path_for_temp_files = f"{base_lakehouse_abfss_path}v3_processed_raw_source_data_aims2025_temp/"

# For the target Delta table schema, this should be the Lakehouse name
TARGET_LAKEHOUSE_SCHEMA = "aims_dbo3" # This is likely your Lakehouse name

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 4, Finished, Available, Finished)

In [3]:
# TARGET_SCHEMA = "aims_dbo3"  # or your schema name
# TABLE_SAVE_MODE = "overwrite"  # or "append"

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 5, Finished, Available, Finished)

In [4]:
# import os
# from urllib.parse import urlparse
# from pyspark.sql.functions import col

# def detect_header_and_read_csv(path, lines_to_check=5):
#     # Step 1: Read file as raw lines with line numbers
#     # Using textFile directly for better control over RDD operations
#     raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
#     preview = raw_lines_rdd.take(lines_to_check)

#     # Step 2: Score lines to find best candidate for header
#     def score(line_tuple):
#         line = line_tuple[0]
#         # A more robust scoring: prioritize lines with more non-empty comma-separated fields
#         # and consider quotes.
#         return len([f for f in line.split(',') if f.strip() != '']) + line.count('"')

#     header_line = max(preview, key=score)
#     header_index = header_line[1]
#     header_raw = header_line[0]

#     # Step 3: Parse header fields
#     # Using a more robust split for CSV, handling quoted commas if necessary.
#     # For simplicity, sticking to split(',') for now, assuming no quoted commas within fields.
#     header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in header_raw.split(",")]
#     num_fields = len(header_fields)

#     print(f"📌 Header detected in {path} at line {header_index}: {header_fields}")

#     # Step 4: Get only data lines AFTER the header
#     data_lines_rdd = (
#         raw_lines_rdd
#         .filter(lambda x: x[1] > header_index)
#         .map(lambda x: x[0])
#     )
    
#     # Calculate initial row count after header
#     rows_after_header = data_lines_rdd.count()
#     print(f"📊 Rows in CSV after header (before loading into DataFrame): {rows_after_header}")


#     # Step 5: Save cleaned lines to OneLake temp location
#     # Ensure filename is compatible with Hadoop path
#     filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
#     temp_path = f"{output_path}/{filename}_temp_data" # Use a unique temp path name to avoid conflicts

#     try:
#         # Check if output_path is a local path or a distributed file system path
#         # Assuming output_path is configured for OneLake or similar distributed FS
#         fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
#         hadoop_path = spark._jvm.org.apache.hadoop.fs.Path(temp_path)
#         if fs.exists(hadoop_path):
#             print(f"🧹 Cleaning up existing temp path: {temp_path}")
#             fs.delete(hadoop_path, True)
#     except Exception as cleanup_err:
#         print(f"⚠️ Could not clean temp path {temp_path}: {cleanup_err}")

#     # Coalesce to 1 partition for small files to avoid many small files in temp,
#     # but be careful with very large files as it can bottleneck.
#     # For robust production, consider repartitioning based on data size.
#     data_lines_rdd.saveAsTextFile(temp_path)


#     # Step 6: Read back the cleaned lines into a DataFrame using Spark's CSV reader
#     # Let Spark's CSV reader handle the parsing and schema inference (or enforce schema later)
#     # Using .option("mode", "PERMISSIVE") to include all rows, even malformed ones,
#     # and then handle them post-load if needed (e.g., by checking for nulls in critical columns)
#     # For now, we'll stick to a basic read and ensure column count
    
#     # It's better to explicitly set the delimiter if it's always a comma.
#     # Also, consider .option("inferSchema", "true") if you want Spark to guess data types.
#     df = spark.read.csv(temp_path, header="false", sep=",", quote='"', escape='"')
    
#     # Get the count of rows loaded into the DataFrame *before* column alignment/renaming
#     rows_loaded_into_df = df.count()
#     print(f"✅ Rows loaded into DataFrame from temp path (before column adjustments): {rows_loaded_into_df}")

#     # Step 7: Force column count alignment
#     actual_fields_in_df = len(df.columns)
    
#     # Create new column names for the DataFrame with a default prefix if there are too many columns
#     # or if we need to pad.
#     new_df_columns = [f"_c{i}" for i in range(max(actual_fields_in_df, num_fields))]

#     if actual_fields_in_df != num_fields:
#         print(f"⚠️ Mismatch: {actual_fields_in_df} fields in data vs {num_fields} in header.")

#         # If more columns than header, select only up to num_fields
#         if actual_fields_in_df > num_fields:
#             df = df.select(df.columns[:num_fields])
#         # If fewer columns than header, add null columns
#         else:
#             # We need to add null columns to match the header count
#             # This ensures the DataFrame has the correct number of columns before renaming
#             for i in range(actual_fields_in_df, num_fields):
#                 df = df.withColumn(f"dummy_col_{i}", col("null")) # Add null column
    
#     # Now, rename columns based on the header fields
#     # Ensure the DataFrame has exactly num_fields columns before renaming
#     # This loop is safer as it iterates over header_fields and picks corresponding columns from df
#     # If df has fewer columns, it will result in an error here, which means our padding logic above
#     # needs to be absolutely correct.
    
#     # The previous logic `df = df.toDF(*header_fields)` assumes df already has the correct number
#     # of columns. Let's make it more explicit.
    
#     # Generate a list of select expressions to map existing columns to header names
#     # This is more robust as it handles both trimming and padding implicitly if the df has been
#     # correctly adjusted to num_fields.
#     select_expressions = []
#     for i, header_name in enumerate(header_fields):
#         # Check if the column exists in the DataFrame based on its _c<index> name
#         # If it doesn't exist, it means we need to add a null column
#         if i < actual_fields_in_df:
#             select_expressions.append(col(f"_c{i}").alias(header_name))
#         else:
#             # This case should be handled by the padding logic above already, but as a safeguard
#             select_expressions.append(col("null").alias(header_name))

#     df = df.select(select_expressions)

#     # Final count after all transformations
#     final_row_count = df.count()
#     print(f"✅ Final row count in loaded DataFrame: {final_row_count}")

#     # Verify if final row count matches rows after header (should be very close if not identical,
#     # unless some rows were truly malformed and Spark's CSV reader skipped them,
#     # or our padding/trimming somehow affected rows, which it shouldn't).
#     if final_row_count != rows_after_header:
#         print(f"⚠️ Discrepancy: Final row count ({final_row_count}) differs from rows after header ({rows_after_header}).")
#         print("This might indicate issues with Spark's CSV parsing for some rows.")


#     return df

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 6, Finished, Available, Finished)

In [5]:
# # import os
# # from urllib.parse import urlparse
# # from pyspark.sql.functions import col

# # def detect_header_and_read_csv(path, output_path, lines_to_check=20): # Ensure output_path is passed
# #     # Step 1: Read file as raw lines with line numbers
# #     raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
# #     preview = raw_lines_rdd.take(lines_to_check)

# #     # Step 2: Score lines to find best candidate for header
# #     def score(line_tuple):
# #         line = line_tuple[0]
# #         # A more robust scoring: prioritize lines with more non-empty comma-separated fields
# #         # and consider quotes.
# #         return len([f for f in line.split(',') if f.strip() != '']) + line.count('"')

# #     header_line = max(preview, key=score)
# #     header_index = header_line[1]
# #     header_raw = header_line[0]

# #     # Step 3: Parse header fields
# #     header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in header_raw.split(",")]
# #     num_fields = len(header_fields)

# #     print(f"📌 Header detected in {path} at line {header_index}: {header_fields}")

# #     # Step 4: Get only data lines AFTER the header
# #     data_lines_rdd = (
# #         raw_lines_rdd
# #         .filter(lambda x: x[1] > header_index)
# #         .map(lambda x: x[0])
# #     )

# #     # Calculate initial row count after header
# #     rows_after_header = data_lines_rdd.count()
# #     print(f"📊 Rows in CSV after header (before loading into DataFrame): {rows_after_header}")

# #     # Step 5: Save cleaned lines to OneLake temp location
# #     filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
# #     temp_data_path = f"{output_path}/{filename}_temp_data" # Use a distinct name for temp data
# #     bad_records_path = f"{output_path}/{filename}_bad_records" # Path for bad records

# #     try:
# #         fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
# #         hadoop_data_path = spark._jvm.org.apache.hadoop.fs.Path(temp_data_path)
# #         hadoop_bad_path = spark._jvm.org.apache.hadoop.fs.Path(bad_records_path)

# #         if fs.exists(hadoop_data_path):
# #             print(f"🧹 Cleaning up existing temp data path: {temp_data_path}")
# #             fs.delete(hadoop_data_path, True)
# #         if fs.exists(hadoop_bad_path):
# #             print(f"🧹 Cleaning up existing bad records path: {bad_records_path}")
# #             fs.delete(hadoop_bad_path, True)

# #     except Exception as cleanup_err:
# #         print(f"⚠️ Could not clean temp paths: {cleanup_err}")

# #     data_lines_rdd.saveAsTextFile(temp_data_path)

# #     # Step 6: Read back the cleaned lines into a DataFrame using Spark's CSV reader
# #     # IMPORTANT: This is the section with the crucial changes for multiLine and badRecordsPath
# #     df = (spark.read
# #               .csv(temp_data_path,
# #                    header="false",
# #                    sep=",",
# #                    quote='"',
# #                    escape='"',
# #                    mode="PERMISSIVE", # Still use permissive, but badRecordsPath captures errors
# #                    badRecordsPath=bad_records_path, # Directs malformed rows here
# #                    multiLine=True # <--- THIS IS THE KEY ADDITION
# #                    ))

# #     # Get the count of rows loaded into the DataFrame *before* column alignment/renaming
# #     rows_loaded_into_df = df.count()
# #     print(f"✅ Rows loaded into DataFrame from temp path (before column adjustments): {rows_loaded_into_df}")

# #     # Step 7: Force column count alignment
# #     actual_fields_in_df = len(df.columns)

# #     if actual_fields_in_df != num_fields:
# #         print(f"⚠️ Mismatch: {actual_fields_in_df} fields in data vs {num_fields} in header.")

# #         if actual_fields_in_df > num_fields:
# #             df = df.select(df.columns[:num_fields])
# #         else:
# #             for i in range(actual_fields_in_df, num_fields):
# #                 # Ensure we add a literal null column, not a column named "null"
# #                 df = df.withColumn(f"dummy_col_{i}", col("null").cast("string")) # Cast to string to be safe

# #     select_expressions = []
# #     for i, header_name in enumerate(header_fields):
# #         # We need to make sure the column name exists or we're adding a null
# #         # If the dataframe has _c0, _c1 etc.
# #         if i < len(df.columns): # Check if the _ci column exists in the current df
# #             select_expressions.append(col(df.columns[i]).alias(header_name))
# #         else:
# #             # This case should ideally be covered by padding above, but as a safeguard
# #             select_expressions.append(col("null").alias(header_name))

# #     df = df.select(select_expressions)

# #     # Final count after all transformations
# #     final_row_count = df.count()
# #     print(f"✅ Final row count in loaded DataFrame: {final_row_count}")

# #     if final_row_count != rows_after_header:
# #         print(f"⚠️ Discrepancy: Final row count ({final_row_count}) differs from rows after header ({rows_after_header}).")
# #         print(f"❗ Inspect bad records at: {bad_records_path}") # <--- This line is also crucial for debugging

# #     return df

# import os
# from urllib.parse import urlparse
# from pyspark.sql.functions import col, lit

# def detect_header_and_read_csv(path, output_path, lines_to_check=20):
#     # Step 1: Read file as raw lines with line numbers
#     raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
#     preview = raw_lines_rdd.take(lines_to_check)

#     # Step 2: Score lines to find best candidate for header
#     def score(line_tuple):
#         line = line_tuple[0]
#         return len([f for f in line.split(',') if f.strip() != '']) + line.count('"')

#     header_line = max(preview, key=score)
#     header_index = header_line[1]
#     header_raw = header_line[0]

#     # Step 3: Parse header fields
#     header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in header_raw.split(",")]
#     num_fields = len(header_fields)

#     print(f"📌 Header detected in {path} at line {header_index}: {header_fields}")

#     # Step 4: Get only data lines AFTER the header
#     data_lines_rdd = (
#         raw_lines_rdd
#         .filter(lambda x: x[1] > header_index)
#         .map(lambda x: x[0])
#     )

#     rows_after_header = data_lines_rdd.count()
#     print(f"📊 Rows in CSV after header (before loading into DataFrame): {rows_after_header}")

#     # Step 5: Save cleaned lines to OneLake temp location
#     filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
#     temp_data_path = f"{output_path}/{filename}_temp_data"
#     # No longer using bad_records_path for Spark option, but can keep for manual logging later

#     try:
#         fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
#         hadoop_data_path = spark._jvm.org.apache.hadoop.fs.Path(temp_data_path)
#         # We can remove the bad_records_path cleanup here as Spark won't manage it directly
#         if fs.exists(hadoop_data_path):
#             print(f"🧹 Cleaning up existing temp data path: {temp_data_path}")
#             fs.delete(hadoop_data_path, True)
#     except Exception as cleanup_err:
#         print(f"⚠️ Could not clean temp path {temp_data_path}: {cleanup_err}")

#     data_lines_rdd.saveAsTextFile(temp_data_path)

#     # Step 6: Read back the cleaned lines into a DataFrame
#     # Removed badRecordsPath
#     df = (spark.read
#               .csv(temp_data_path,
#                    header="false",
#                    sep=",",
#                    quote='"',
#                    escape='"',
#                    mode="PERMISSIVE", # Will try to parse, set malformed to nulls
#                    multiLine=True # Keep multiLine to handle embedded newlines
#                    ))

#     rows_loaded_into_df = df.count()
#     print(f"✅ Rows loaded into DataFrame from temp path (before column adjustments): {rows_loaded_into_df}")

#     # Step 7: Force column count alignment
#     actual_fields_in_df = len(df.columns)

#     if actual_fields_in_df != num_fields:
#         print(f"⚠️ Mismatch: {actual_fields_in_df} fields in data vs {num_fields} in header.")

#         if actual_fields_in_df > num_fields:
#             df = df.select(df.columns[:num_fields])
#         else:
#             for i in range(actual_fields_in_df, num_fields):
#                 df = df.withColumn(f"dummy_col_{i}", lit(None).cast("string")) # Use lit(None) for explicit null

#     select_expressions = []
#     for i, header_name in enumerate(header_fields):
#         if i < len(df.columns):
#             select_expressions.append(col(df.columns[i]).alias(header_name))
#         else:
#             select_expressions.append(lit(None).alias(header_name)) # Use lit(None) for explicit null

#     df = df.select(select_expressions)

#     final_row_count = df.count()
#     print(f"✅ Final row count in loaded DataFrame: {final_row_count}")

#     if final_row_count != rows_after_header:
#         print(f"⚠️ Discrepancy: Final row count ({final_row_count}) differs from rows after header ({rows_after_header}).")
#         print("This indicates issues with Spark's CSV parsing for some rows. Those rows are likely parsed with nulls in some columns.")
#         # NEW: Suggesting to manually identify and store bad records
#         # This part requires you to define what makes a row "bad" for your data
#         # Example: if 'ID' column should never be null
#         # bad_df = df.filter(col("ID").isNull())
#         # if bad_df.count() > 0:
#         #     print(f"❗ Found {bad_df.count()} potentially bad rows. Consider saving to {output_path}/{filename}_bad_records_manual")
#         #     # bad_df.write.format("delta").mode("overwrite").save(f"{output_path}/{filename}_bad_records_manual")
#         #     # Or save as CSV/JSON to inspect:
#         #     # bad_df.write.csv(f"{output_path}/{filename}_bad_records_manual", mode="overwrite", header=True)
#         #     # bad_df.write.json(f"{output_path}/{filename}_bad_records_manual_json", mode="overwrite")

#     return df

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 7, Finished, Available, Finished)

In [6]:
import os
from urllib.parse import urlparse
from pyspark.sql.functions import col
import csv
from io import StringIO # For robust header parsing

def detect_header_and_read_csv(path, output_path, lines_to_check=20): # Ensure output_path is passed
    # Step 0: Initial read of a small sample for header detection
    # Read as text to preserve line breaks for header scoring
    raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
    preview = raw_lines_rdd.take(lines_to_check)

    # Step 1: Score lines to find best candidate for header
    def score(line_tuple):
        line = line_tuple[0]
        # A more robust scoring: prioritize lines with more non-empty comma-separated fields
        # and consider quotes. Using csv.reader to correctly handle commas inside quotes for scoring
        try:
            fields = next(csv.reader(StringIO(line)))
            return len([f for f in fields if f.strip() != '']) # Count non-empty parsed fields
        except Exception:
            return 0 # If parsing fails, score low

    header_line = max(preview, key=score)
    header_index = header_line[1]
    header_raw = header_line[0]

    # Step 2: Parse header fields robustly
    # Using csv.reader for robust parsing of header line itself, considering quotes
    header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in next(csv.reader(StringIO(header_raw)))]
    num_fields = len(header_fields)

    print(f"📌 Header detected in {path} at line {header_index}: {header_fields}")

    # Step 3: Define bad records path and cleanup for this file
    filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    bad_records_path = f"{output_path}/_bad_rows_audit/{filename}" # Use a slightly different path for clarity

    try:
        fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
        hadoop_bad_path = spark._jvm.org.apache.hadoop.fs.Path(bad_records_path)
        if fs.exists(hadoop_bad_path):
            print(f"🧹 Cleaning up existing bad records path: {bad_records_path}")
            fs.delete(hadoop_bad_path, True)
    except Exception as cleanup_err:
        print(f"⚠️ Could not clean bad records path {bad_records_path}: {cleanup_err}")

    # Step 4: Read the ORIGINAL CSV file directly into a DataFrame
    # IMPORTANT: Use the original 'path' directly, and skip header lines
    print(f"📚 Attempting to read {path} directly with CSV reader and header skip.")
    df = (spark.read
              .csv(path,
                   header="false", # We will handle header separately
                   sep=",",
                   quote='"',
                   escape='"',
                   mode="PERMISSIVE",
                   badRecordsPath=bad_records_path, # Directs malformed rows here
                   multiLine=True # CRITICAL: Handles newlines within quoted fields
                   ))

    # After reading the raw CSV, filter out the header lines
    # This requires re-reading the raw_lines_rdd or passing its structure
    # A more robust way is to re-read or combine
    # However, for the purpose of getting only data, Spark's CSV reader does not have a direct
    # 'skip lines' feature that plays nicely with multiLine=True.
    # The most common pattern is to re-read the file with header=true IF header is always first line.
    # Since your header can be anywhere, we need to apply a filter AFTER parsing.
    # This requires joining the parsed DF with line numbers, which can be inefficient.

    # Alternative strategy for non-first-line headers:
    # 1. Read the entire file as CSV with inferred schema.
    # 2. Identify the header row based on the raw content.
    # 3. Filter the DataFrame to remove the header row.
    # This is more complex than direct header=true.

    # Given your header detection, let's assume the identified header_raw IS the line
    # to filter out.
    # First, let's rename columns for easier handling
    original_df_cols = [f"_c{i}" for i in range(len(df.columns))]
    df = df.toDF(*original_df_cols)

    # Filter out the header line content based on raw string match
    # This might not be perfect if the header_raw string itself can appear in data
    # Consider adding a column with row index to the DF after reading if filter is slow
    # For now, let's trust Spark's `PERMISSIVE` and `multiLine` to do most of the work.

    # The most robust way to handle a non-first-line header with spark.read.csv
    # is to read the file *twice*.
    # First pass: read as text to find header_index.
    # Second pass: read as CSV using Spark's skipRows (if available) or by loading and filtering.

    # Let's use a simpler approach which is common when header can be non-first but we want to load clean data.
    # We load all, then filter the header string itself. This is not ideal for perf on very large datasets
    # if the header string exists in data.
    # A more common pattern for non-first-line header is to read the file, then programmatically set the schema
    # and filter out rows before that index, but spark.read.csv doesn't easily expose row indexes.

    # Given the previous context and performance concerns, let's revert to the more standard approach
    # where if header is not on line 0, we treat the *first line* of the CSV as the header by option,
    # or rely on inferSchema, and then clean up.

    # Re-evaluating: Your header detection is sophisticated. The best way to use it
    # with spark.read.csv is to read the data rows *after* the header using the RDD
    # and then apply spark.createDataFrame from that RDD, or, use the `start_offset` if it was direct file.
    # However, this brings us back to the RDD row-by-row problem, which `multiLine=True` in spark.read.csv solves.

    # Therefore, the only way to fully leverage multiLine=True for parsing AND dynamic header is:
    # 1. Detect header line and field names.
    # 2. Determine number of header lines to skip.
    # 3. Read the CSV using Spark's reader, skipping the determined number of lines.
    # Unfortunately, Spark's `csv` reader does not have a `skipRows` option.
    # This implies that for a non-first-line header, the `spark.read.csv` method cannot natively skip lines before parsing.

    # Let's adjust for the most common scenario for performance:
    # The assumption for using spark.read.csv(header=True, multiLine=True) is that the header IS the first line.
    # If the header is NOT the first line, then Spark's CSV reader will interpret the actual first line as the header.
    # Your current scoring finds the *best* header, but if it's not line 0, direct CSV reading with header=true will fail.

    # Reverting to the most performant approach, assuming header is generally at line 0 or 1.
    # If the header IS at line 0, the process becomes much simpler.

    # --- REVISED STRATEGY BASED ON OBSERVED BAD RECORDS ---
    # The 'bad records' show multi-line content being split.
    # This implies that the initial sparkContext.textFile().saveAsTextFile() pipeline IS THE CAUSE.
    # We must eliminate the RDD intermediate step for the data part and use Spark's CSV reader DIRECTLY
    # on the original 'path'.

    # The challenge with your header detection (header_index) and `spark.read.csv`:
    # `spark.read.csv` does not have a `skip_rows` option for the data portion.
    # If `header_index` is > 0, and you use `header=True` with `spark.read.csv`, it will still read the
    # first line of the file as the header, not your detected header_index.

    # Given the problem's nature, the optimal solution is to:
    # A. Use `header=True` with `spark.read.csv` directly if headers are *always* the first line.
    # B. If headers can be on other lines, you MUST identify the actual header string, and then either:
    #    B1. Read the file into a DataFrame, then filter out lines by content (inefficient).
    #    B2. (More advanced) Read the raw text, find header_index, then use a custom input format
    #        or manipulate partitions (very complex).

    # Let's go with the most common and performant fix for multi-line issues:
    # Assuming `header_index` is usually 0, or we are okay with Spark inferring a header from line 0.
    # The `detect_header_and_read_csv` function needs to change its core logic.

    # Let's modify the function to assume:
    # 1. The header detection is still useful for getting column names.
    # 2. For loading the data, we'll try reading directly with `header=True` and `multiLine=True`.
    #    This means Spark will consider the first line as the header. If your `header_index` is often > 0,
    #    this will still lead to issues (the actual header line will be treated as data, and the first
    #    data line will be treated as header).

    # --- REVISED APPROACH: Simplified and Optimized for Performance ---
    # This version optimizes for the multi-line CSV issue by using Spark's CSV reader directly.
    # It assumes the *first line* of the CSV file is the header, which is standard practice for `spark.read.csv(header=True)`.
    # If your CSVs frequently have headers on lines other than the first, we need a more complex solution.

    print(f"📚 Attempting to read {path} directly with Spark's CSV reader.")
    print("   Assuming the first line of the file is the header for direct parsing.")

    df = (spark.read
              .format("csv")
              .option("header", "true") # Tell Spark to use the first line as header
              .option("inferSchema", "true") # Let Spark infer data types
              .option("sep", ",")
              .option("quote", '"')
              .option("escape", '"')
              .option("mode", "PERMISSIVE") # Still use permissive to catch other errors
              .option("badRecordsPath", bad_records_path) # Directs malformed rows here
              .option("multiLine", "true") # CRITICAL: Handles newlines within quoted fields
              .load(path)) # Load the original file directly

    # Renaming columns and sanitizing after direct load, as Spark inferring header might give odd names
    # This also helps standardize column names for Delta tables
    for col_name in df.columns:
        new_col_name = col_name.strip().strip('"').replace('\ufeff', '').replace('.', '_').replace(' ', '_').lower()
        if col_name != new_col_name:
            df = df.withColumnRenamed(col_name, new_col_name)

    # Now, count rows and bad records:
    total_rows = df.count() # This is the count of valid rows loaded into DF

    bad_records_count = 0
    try:
        if fs.exists(hadoop_bad_path):
            bad_record_files = mssparkutils.fs.ls(hadoop_bad_path)
            for item in bad_record_files:
                if not item.isDir:
                    bad_records_count += spark.read.text(item.path).count()
        print(f"🚨 Bad records identified by Spark's CSV reader (written to {bad_records_path}): {bad_records_count}")
    except Exception as count_err:
        print(f"⚠️ Could not count bad records at {bad_records_path}: {count_err}")
        bad_records_count = -1

    # We cannot reliably get "raw_lines" from the RDD step if we bypass it.
    # The 'total_rows' from df.count() IS your 'valid_rows'.
    # For 'raw_lines', you'd need to count lines in the original file, which can be slow for large files.
    # The `Invalid` count from `bad_records_count` will tell you what was skipped.

    print(f"✅ Final row count in loaded DataFrame: {total_rows}")
    print(f"Schema for {os.path.basename(path)}:")
    df.printSchema()
    print(f"Data preview for {os.path.basename(path)}:")
    df.show(2, truncate=False)

    # Return stats for overall summary
    return df, total_rows, total_rows, bad_records_count # total_data_lines_read, valid_lines, invalid_lines
                                                       # Note: total_data_lines_read would ideally be raw lines
                                                       # but we don't have that directly with this optimized path.
                                                       # For simplicity, returning valid_rows for raw_lines here.
                                                       # The 'invalid_lines' is now bad_records_count.

# The rest of your main loop (STEP 2 and STEP 3) will call this modified function.
# Ensure TARGET_SCHEMA and TABLE_SAVE_MODE and output_path_ are defined.
# You will need to define `spark` object in your environment if not already.

# Example of how the main loop would use it (assuming it's defined elsewhere):
# for path in csv_files:
#     try:
#         print(f"\n🔍 Inspecting: {path}")
#         # df, total, valid, invalid = detect_header_and_read_csv(path) # Old signature
#         df, total_loaded_rows, valid_loaded_rows, invalid_count = detect_header_and_read_csv(path, output_path_)
#         parsed_dfs[path] = df
#         successfully_loaded.append(path)
#         load_stats[path] = {
#             "raw_lines": total_loaded_rows + invalid_count, # Estimate raw if you need it for stats
#             "valid_rows": valid_loaded_rows,
#             "invalid_rows": invalid_count
#         }
#         # ... rest of your loop (df.printSchema(), df.show(), writing to table)
#     except Exception as e:
#         print(f"❌ Error loading {path}: {e}")
#         failed_to_load.append((path, str(e)))

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 8, Finished, Available, Finished)

In [7]:
output_path_ = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/v2_processed_raw_source_data_aims2025"
target_schema_ = "aims_dbo3"


StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 9, Finished, Available, Finished)

#### ochestrate

In [8]:
from notebookutils import mssparkutils

def list_csv_files_recursive(base_path):
    csv_files = []

    def walk(path):
        for item in mssparkutils.fs.ls(path):
            if item.isDir:
                walk(item.path)
            elif item.path.endswith(".csv") or item.path.endswith(".csv.gz"):
                csv_files.append(item.path)

    walk(base_path)
    return csv_files

# ✅ Use this to get all CSV files
csv_files = list_csv_files_recursive("Files/")
print(f"📁 Found {len(csv_files)} CSV files")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 10, Finished, Available, Finished)

📁 Found 28 CSV files


In [9]:
# import os
# from urllib.parse import urlparse

# # Setup
# parsed_dfs = {}
# successfully_loaded = []
# failed_to_load = []

# successfully_written = []
# failed_to_write = []

# TARGET_SCHEMA = "aims_dbo3"  # e.g., 'bronze', 'staging'
# TABLE_SAVE_MODE = "overwrite"       # or "append"
# output_path = output_path_  # Adjust to the OneLake temp path

# # STEP 1: Load all CSVs
# for path in csv_files:
#     try:
#         print(f"\n🔍 Inspecting: {path}")
#         df = detect_header_and_read_csv(path)
#         parsed_dfs[path] = df
#         successfully_loaded.append(path)
#         df.printSchema()
#         df.show(2)
#     except Exception as e:
#         print(f"❌ Error loading {path}: {e}")
#         failed_to_load.append((path, str(e)))

# # STEP 2: Write to tables
# for path, df in parsed_dfs.items():
#     table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower() #os.path.splitext(os.path.basename(path))[0].lower() #os.path.basename(path).replace(".csv", "").replace(".csv.gz", "").lower()
#     full_table_name = f"{TARGET_SCHEMA}.{table_name}"

#     try:
#         print(f"\n💾 Writing to table: {full_table_name}")
#         df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
#         print(f"✅ Successfully written: {full_table_name}")
#         successfully_written.append(full_table_name)
#     except Exception as e:
#         print(f"❌ Failed to write {full_table_name}: {e}")
#         failed_to_write.append((full_table_name, str(e)))

# # STEP 3: Summary Report
# print("\n📊 LOAD SUMMARY")
# print(f"✅ Loaded: {len(successfully_loaded)} files")
# print(f"❌ Failed to load: {len(failed_to_load)} files")
# if failed_to_load:
#     for path, err in failed_to_load:
#         print(f"   - {path} → {err}")

# print("\n📊 WRITE SUMMARY")
# print(f"✅ Written to tables: {len(successfully_written)}")
# print(f"❌ Failed to write: {len(failed_to_write)}")
# if failed_to_write:
#     for tbl, err in failed_to_write:
#         print(f"   - {tbl} → {err}")



StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 11, Finished, Available, Finished)

In [10]:
# import os
# from urllib.parse import urlparse

# # IMPORTANT: Ensure 'output_path_' is defined before this block of code.
# # For example:
# # output_path_ = "abfss://<your_lakehouse_guid>@onelake.dfs.fabric.microsoft.com/<your_folder_or_path>/"
# # Replace the placeholder with your actual OneLake temporary path.

# # Setup
# parsed_dfs = {}
# successfully_loaded = []
# failed_to_load = []

# successfully_written = []
# failed_to_write = []

# TARGET_SCHEMA = "aims_dbo3"  # e.g., 'bronze', 'staging'
# TABLE_SAVE_MODE = "overwrite"       # or "append"
# output_path = output_path_  # Adjust to the OneLake temp path

# # STEP 1: Load all CSVs
# for path in csv_files:
#     try:
#         print(f"\n🔍 Inspecting: {path}")
#         # CORRECTED LINE: Pass the 'output_path' argument to the function
#         df = detect_header_and_read_csv(path, output_path)
#         parsed_dfs[path] = df
#         successfully_loaded.append(path)
#         df.printSchema()
#         df.show(2)
#     except Exception as e:
#         print(f"❌ Error loading {path}: {e}")
#         failed_to_load.append((path, str(e)))

# # STEP 2: Write to tables
# for path, df in parsed_dfs.items():
#     table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
#     full_table_name = f"{TARGET_SCHEMA}.{table_name}"

#     try:
#         print(f"\n💾 Writing to table: {full_table_name}")
#         df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
#         print(f"✅ Successfully written: {full_table_name}")
#         successfully_written.append(full_table_name)
#     except Exception as e:
#         print(f"❌ Failed to write {full_table_name}: {e}")
#         failed_to_write.append((full_table_name, str(e)))

# # STEP 3: Summary Report
# print("\n📊 LOAD SUMMARY")
# print(f"✅ Loaded: {len(successfully_loaded)} files")
# print(f"❌ Failed to load: {len(failed_to_load)} files")
# if failed_to_load:
#     for path, err in failed_to_load:
#         print(f"   - {path} → {err}")

# print("\n📊 WRITE SUMMARY")
# print(f"✅ Written to tables: {len(successfully_written)}")
# print(f"❌ Failed to write: {len(failed_to_write)}")
# if failed_to_write:
#     for tbl, err in failed_to_write:
#         print(f"   - {tbl} → {err}")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 12, Finished, Available, Finished)

# details relating to error will be documented. now switching to python to try and see if error will be repliated.

In [11]:
from notebookutils import mssparkutils
import os
from urllib.parse import urlparse
from io import StringIO
import csv

# -------------------------------------------------------------------
# STEP 0: List CSV Files
# -------------------------------------------------------------------
def list_csv_files_recursive(base_path):
    csv_files = []

    def walk(path):
        for item in mssparkutils.fs.ls(path):
            if item.isDir:
                walk(item.path)
            elif item.path.endswith(".csv") or item.path.endswith(".csv.gz"):
                csv_files.append(item.path)

    walk(base_path)
    return csv_files

csv_files = list_csv_files_recursive("Files/")
print(f"📁 Found {len(csv_files)} CSV files")

# -------------------------------------------------------------------
# STEP 1: RDD-Based CSV Parser
# -------------------------------------------------------------------
def detect_header_and_read_csv(path, lines_to_check=20):
    raw_lines = spark.read.text(path).rdd.map(lambda r: r[0]).zipWithIndex()
    preview = raw_lines.take(lines_to_check)

    def score(line_tuple):
        line = line_tuple[0]
        return line.count(",") + line.count('"')

    header_line = max(preview, key=score)
    header_index = header_line[1]
    header_raw = header_line[0]
    header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in next(csv.reader(StringIO(header_raw)))]
    num_fields = len(header_fields)

    print(f"📌 Header detected in {path} at line {header_index}: {header_fields}")

    def is_valid_csv_row(line):
        try:
            parsed = next(csv.reader(StringIO(line)))
            return len(parsed) == num_fields
        except Exception:
            return False

    def parse_csv_row(line):
        try:
            return next(csv.reader(StringIO(line)))
        except Exception:
            return [None] * num_fields

    total_data_lines = raw_lines.filter(lambda x: x[1] > header_index).map(lambda x: x[0])
    valid_lines = total_data_lines.filter(is_valid_csv_row).map(parse_csv_row)
    invalid_lines = total_data_lines.filter(lambda line: not is_valid_csv_row(line))

    # For audit: Save invalid lines
    filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    bad_path = f"{output_path}/_bad_rows/{filename}"
    try:
        fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
        hadoop_path = spark._jvm.org.apache.hadoop.fs.Path(bad_path)
        if fs.exists(hadoop_path):
            fs.delete(hadoop_path, True)
    except Exception:
        pass
    if invalid_lines.count() > 0:
        invalid_lines.saveAsTextFile(bad_path)
        print(f"⚠️ Invalid lines written to: {bad_path}")

    # Create final DataFrame
    df = spark.createDataFrame(valid_lines, header_fields)
    return df, total_data_lines.count(), valid_lines.count(), invalid_lines.count()

# -------------------------------------------------------------------
# STEP 2: Loop through files
# -------------------------------------------------------------------
TARGET_SCHEMA = target_schema_
TABLE_SAVE_MODE = "overwrite"
output_path = output_path_  # Predefined path

parsed_dfs = {}
load_stats = {}

successfully_loaded = []
failed_to_load = []
successfully_written = []
failed_to_write = []

for path in csv_files:
    try:
        print(f"\n🔍 Inspecting: {path}")
        df, total, valid, invalid = detect_header_and_read_csv(path)
        parsed_dfs[path] = df
        successfully_loaded.append(path)
        load_stats[path] = {
            "raw_lines": total,
            "valid_rows": valid,
            "invalid_rows": invalid
        }
        df.printSchema()
        df.show(2)
    except Exception as e:
        print(f"❌ Error loading {path}: {e}")
        failed_to_load.append((path, str(e)))

for path, df in parsed_dfs.items():
    table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    full_table_name = f"{TARGET_SCHEMA}.{table_name}"

    try:
        print(f"\n💾 Writing to table: {full_table_name}")
        df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
        print(f"✅ Successfully written: {full_table_name}")
        successfully_written.append(full_table_name)
    except Exception as e:
        print(f"❌ Failed to write {full_table_name}: {e}")
        failed_to_write.append((full_table_name, str(e)))


# -------------------------------------------------------------------
# STEP 3: Summary
# -------------------------------------------------------------------
print("\n📊 LOAD SUMMARY")
print(f"✅ Loaded: {len(successfully_loaded)}")
print(f"❌ Failed to load: {len(failed_to_load)}")
for path, err in failed_to_load:
    print(f"   - {path} → {err}")

print("\n📊 WRITE SUMMARY")
print(f"✅ Written to tables: {len(successfully_written)}")
print(f"❌ Failed to write: {len(failed_to_write)}")
for tbl, err in failed_to_write:
    print(f"   - {tbl} → {err}")

print("\n📋 ROW COUNT STATS")
for path, stats in load_stats.items():
    base = os.path.basename(path)
    print(f"📄 {base} → Raw: {stats['raw_lines']}, Valid: {stats['valid_rows']}, Invalid: {stats['invalid_rows']}")


StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 13, Finished, Available, Finished)

📁 Found 28 CSV files

🔍 Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz
📌 Header detected in abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz at line 1: ['ID', 'ASSET', 'ATTRIBUTE', 'TEXTVALUE', 'NUMERICVALUE', 'DATEVALUE', 'DECIMALVALUE2DP', 'SELECTIONVALUE', 'LOCATION', 'VALIDFROM', 'VALIDTO', 'KINO_CSID', 'HS2_KINO_CUID', 'KINO_CWHEN', 'KINO_USID', 'KINO_UUID', 'KINO_UWHEN', 'BASEDON', 'OWNER', 'PHASE', 'PERSON', 'ORGANISATION', 'TEXTSTOREFORTYPETRANSITION', 'DECIMALSTOREFORTYPETRANSITION', 'NUMERICSTOREFORTYPETRANSITION', 'STARTCHAINAGE', 'ENDCHAINAGE', 'NUMERICENDVALUE', 'DECIMALENDVALUE2DP', 'ISBEINGSPLIT', 'SPLITATCHAINAGE', 'AFTERNUMERICSTARTVALUE', 'AFTERNUMERICENDVALUE', 'AFTERDECIMALSTARTVALUE2DP', 'AFTERDECIMALENDVALUE2DP', 'AFTERTEXTVALUE', 'AFTERDATEVALUE', 'AFTERSELECTION

In [12]:
from notebookutils import mssparkutils
import os
from pyspark.sql.functions import input_file_name

# Define the base path for your bad records
bad_rows_base_path = "abfss://COE_F_EUC_P2@onelake.dfs.fabric.microsoft.com/AimsDataFromBlob2025.Lakehouse/Files/v2_processed_raw_source_data_aims2025/_bad_rows"

print(f"Starting inspection of bad records in: {bad_rows_base_path}\n")

# Function to recursively list all files in a given path
def list_all_files_recursive(base_path):
    all_files = []
    queue = [base_path]

    while queue:
        current_path = queue.pop(0)
        try:
            items = mssparkutils.fs.ls(current_path)
            for item in items:
                if item.isDir:
                    queue.append(item.path)
                else:
                    all_files.append(item.path)
        except Exception as e:
            print(f"⚠️ Warning: Could not list contents of {current_path}. Error: {e}")
            continue
    return all_files

# Get a list of all bad record files
bad_record_files = list_all_files_recursive(bad_rows_base_path)

if not bad_record_files:
    print(f"✅ No bad record files found in {bad_rows_base_path}. This is great!")
else:
    print(f"🔍 Found {len(bad_record_files)} bad record files to inspect.\n")

    for i, file_path in enumerate(bad_record_files):
        print(f"--- Inspecting Bad Record File {i+1}/{len(bad_record_files)}: {file_path} ---")
        try:
            # Read the bad records as plain text
            # Limit the number of records shown per file to avoid overwhelming output
            bad_df = spark.read.text(file_path)

            # Add the source file name to each row for better context
            bad_df = bad_df.withColumn("source_bad_file", input_file_name())

            print(f"Total lines in this bad record file: {bad_df.count()}")
            print("Sample of bad records (first 10 lines):")
            # Show a sample, preventing truncation for full line inspection
            bad_df.limit(10).show(truncate=False)

        except Exception as e:
            print(f"❌ Error reading bad record file {file_path}: {e}")
        print("-" * 80 + "\n")

print("Inspection complete.")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 14, Finished, Available, Finished)

Starting inspection of bad records in: abfss://COE_F_EUC_P2@onelake.dfs.fabric.microsoft.com/AimsDataFromBlob2025.Lakehouse/Files/v2_processed_raw_source_data_aims2025/_bad_rows

🔍 Found 63 bad record files to inspect.

--- Inspecting Bad Record File 1/63: abfss://COE_F_EUC_P2@onelake.dfs.fabric.microsoft.com/AimsDataFromBlob2025.Lakehouse/Files/v2_processed_raw_source_data_aims2025/_bad_rows/assetattributes_csv/_SUCCESS ---
Total lines in this bad record file: 0
Sample of bad records (first 10 lines):
+-----+---------------+
|value|source_bad_file|
+-----+---------------+
+-----+---------------+

--------------------------------------------------------------------------------

--- Inspecting Bad Record File 2/63: abfss://COE_F_EUC_P2@onelake.dfs.fabric.microsoft.com/AimsDataFromBlob2025.Lakehouse/Files/v2_processed_raw_source_data_aims2025/_bad_rows/assetattributes_csv/part-00000 ---
Total lines in this bad record file: 141980
Sample of bad records (first 10 lines):
+-----------------

In [13]:
# import os
# from urllib.parse import urlparse
# from pyspark.sql.functions import col, input_file_name
# import csv
# from io import StringIO # For robust header parsing

# def detect_header_and_read_csv(path, output_path, lines_to_check=20):
#     # Step 0: Initial read of a small sample for header detection
#     raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
#     preview = raw_lines_rdd.take(lines_to_check)

#     # Step 1: Score lines to find best candidate for header
#     def score(line_tuple):
#         line = line_tuple[0]
#         try:
#             fields = next(csv.reader(StringIO(line)))
#             return len([f for f in fields if f.strip() != ''])
#         except Exception:
#             return 0

#     header_line = max(preview, key=score)
#     header_index = header_line[1]
#     header_raw = header_line[0]

#     # Step 2: Parse header fields robustly
#     header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in next(csv.reader(StringIO(header_raw)))]
#     num_fields = len(header_fields)

#     print(f"📌 Header detected in {path} at line {header_index}: {header_fields}")

#     # Step 3: Define bad records path and cleanup for this file
#     filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
#     # Using a distinct path for bad records collected via _corrupt_record column
#     bad_records_output_path = f"{output_path}/_bad_records_corrupt_col/{filename}"

#     try:
#         fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
#         hadoop_bad_path = spark._jvm.org.apache.hadoop.fs.Path(bad_records_output_path)
#         if fs.exists(hadoop_bad_path):
#             print(f"🧹 Cleaning up existing bad records path: {bad_records_output_path}")
#             fs.delete(hadoop_bad_path, True)
#     except Exception as cleanup_err:
#         print(f"⚠️ Could not clean bad records path {bad_records_output_path}: {cleanup_err}")

#     # Step 4: Read the ORIGINAL CSV file directly into a DataFrame
#     # IMPORTANT: Removed 'badRecordsPath' as it's not supported.
#     # 'PERMISSIVE' mode will now put corrupt records into '_corrupt_record' column.
#     print(f"📚 Attempting to read {path} directly with Spark's CSV reader.")
#     print("   Assuming the first line of the file is the header for direct parsing.")
#     print("   Malformed records will be captured in the '_corrupt_record' column.")

#     df = (spark.read
#               .format("csv")
#               .option("header", "true") # Tell Spark to use the first line as header
#               .option("inferSchema", "true") # Let Spark infer data types
#               .option("sep", ",")
#               .option("quote", '"')
#               .option("escape", '"')
#               .option("mode", "PERMISSIVE") # Malformed records go to _corrupt_record column
#               .option("multiLine", "true") # CRITICAL: Handles newlines within quoted fields
#               .load(path)) # Load the original file directly

#     # Renaming columns and sanitizing after direct load
#     # This also helps standardize column names for Delta tables
#     for col_name in df.columns:
#         new_col_name = col_name.strip().strip('"').replace('\ufeff', '').replace('.', '_').replace(' ', '_').lower()
#         if col_name != new_col_name:
#             df = df.withColumnRenamed(col_name, new_col_name)

#     # Separate valid and invalid rows using the _corrupt_record column
#     valid_df = df.filter(col("_corrupt_record").isNull())
#     invalid_df = df.filter(col("_corrupt_record").isNotNull())

#     total_valid_rows = valid_df.count()
#     total_invalid_rows = invalid_df.count()

#     print(f"🚨 Malformed records found (via _corrupt_record column): {total_invalid_rows}")
#     if total_invalid_rows > 0:
#         print(f"   Saving malformed records to: {bad_records_output_path}")
#         # Select the corrupt record column and the input file name for audit
#         invalid_df.select(col("_corrupt_record"), input_file_name().alias("source_file")) \
#                   .write.mode("overwrite").format("text").save(bad_records_output_path)

#     print(f"✅ Final row count in loaded DataFrame (valid rows): {total_valid_rows}")
#     print(f"Schema for {os.path.basename(path)}:")
#     valid_df.printSchema() # Print schema of the valid DataFrame
#     print(f"Data preview for {os.path.basename(path)} (valid rows):")
#     valid_df.show(2, truncate=False)

#     # Return the valid DataFrame and counts for summary
#     # For 'raw_lines', we'll estimate it as valid + invalid for summary purposes
#     estimated_raw_lines = total_valid_rows + total_invalid_rows
#     return valid_df, estimated_raw_lines, total_valid_rows, total_invalid_rows

# # --- Main Loop (as provided by you, with minor adjustments for function return) ---

# # IMPORTANT: Ensure 'output_path_' is defined before this block of code.
# # For example:
# # output_path_ = "abfss://<your_lakehouse_guid>@onelake.dfs.fabric.microsoft.com/<your_folder_or_path>/"
# # Replace the placeholder with your actual OneLake temporary path.

# # Setup
# parsed_dfs = {}
# successfully_loaded = []
# failed_to_load = []

# successfully_written = []
# failed_to_write = []

# TARGET_SCHEMA = "aims_dbo3"  # e.g., 'bronze', 'staging'
# TABLE_SAVE_MODE = "overwrite"       # or "append"
# output_path = output_path_  # Adjust to the OneLake temp path

# # Assuming 'csv_files' is defined from your previous `list_csv_files_recursive` call
# # If not, you'll need to define it here, e.g.:
# # from notebookutils import mssparkutils
# # def list_csv_files_recursive(base_path): ... (your function)
# # csv_files = list_csv_files_recursive("Files/") # Or your actual base path for CSVs

# # STEP 1: Load all CSVs
# load_stats = {} # Initialize load_stats here for the summary
# for path in csv_files:
#     try:
#         print(f"\n🔍 Inspecting: {path}")
#         # Call the modified function
#         df, raw_count, valid_count, invalid_count = detect_header_and_read_csv(path, output_path)
#         parsed_dfs[path] = df
#         successfully_loaded.append(path)

#         # Store stats for summary
#         load_stats[path] = {
#             "raw_lines": raw_count,
#             "valid_rows": valid_count,
#             "invalid_rows": invalid_count
#         }

#         # df.printSchema() and df.show(2) are already inside the function now
#     except Exception as e:
#         print(f"❌ Error loading {path}: {e}")
#         failed_to_load.append((path, str(e)))

# # STEP 2: Write to tables
# for path, df in parsed_dfs.items():
#     table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
#     full_table_name = f"{TARGET_SCHEMA}.{table_name}"

#     try:
#         print(f"\n💾 Writing to table: {full_table_name}")
#         df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
#         print(f"✅ Successfully written: {full_table_name}")
#         successfully_written.append(full_table_name)
#     except Exception as e:
#         print(f"❌ Failed to write {full_table_name}: {e}")
#         failed_to_write.append((full_table_name, str(e)))

# # STEP 3: Summary Report
# print("\n📊 LOAD SUMMARY")
# print(f"✅ Loaded: {len(successfully_loaded)} files")
# print(f"❌ Failed to load: {len(failed_to_load)} files")
# if failed_to_load:
#     for path, err in failed_to_load:
#         print(f"   - {path} → {err}")

# print("\n📊 WRITE SUMMARY")
# print(f"✅ Written to tables: {len(successfully_written)}")
# print(f"❌ Failed to write: {len(failed_to_write)}")
# if failed_to_write:
#     for tbl, err in failed_to_write:
#         print(f"   - {tbl} → {err}")

# print("\n📋 ROW COUNT STATS")
# for path, stats in load_stats.items():
#     base = os.path.basename(path)
#     print(f"📄 {base} → Raw: {stats['raw_lines']}, Valid: {stats['valid_rows']}, Invalid: {stats['invalid_rows']}")




StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 15, Finished, Available, Finished)

In [14]:
import os
from urllib.parse import urlparse
from pyspark.sql.functions import col, input_file_name, monotonically_increasing_id, lit
import csv
from io import StringIO # For robust header parsing
from pyspark.sql.types import StringType, StructType, StructField # For schema manipulation

def detect_header_and_read_csv(path, output_path, lines_to_check=50): # Increased lines_to_check for robustness
    # Step 0: Initial read of a small sample for header detection (using RDD)
    raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
    preview = raw_lines_rdd.take(lines_to_check)

    # Step 1: Score lines to find best candidate for header
    def score(line_tuple):
        line = line_tuple[0]
        try:
            # Using csv.reader to correctly handle commas inside quotes for scoring
            fields = next(csv.reader(StringIO(line)))
            return len([f for f in fields if f.strip() != '']) # Count non-empty parsed fields
        except Exception:
            return 0 # If parsing fails, score low

    # If preview is empty (e.g., empty file), handle gracefully
    if not preview:
        print(f"⚠️ Warning: File {path} is empty or contains no lines to preview.")
        return spark.createDataFrame([], StructType([])), 0, 0, 0 # Return empty DF

    header_line_tuple = max(preview, key=score)
    header_index_rdd = header_line_tuple[1] # The 0-based line index in the raw file
    header_raw_string = header_line_tuple[0]

    # Step 2: Parse header fields robustly using csv.reader on the identified header string
    # This gives us the final column names.
    try:
        header_parsed_fields = [col_name.strip().strip('"').replace('\ufeff', '') for col_name in next(csv.reader(StringIO(header_raw_string)))]
    except Exception as e:
        raise ValueError(f"Could not parse header line '{header_raw_string}' from {path}. Error: {e}")

    # Clean up column names for Spark
    clean_header_fields = []
    for col_name in header_parsed_fields:
        clean_name = col_name.replace('.', '_').replace(' ', '_').lower()
        # Ensure uniqueness if columns are identical after cleaning
        counter = 1
        original_clean_name = clean_name
        while clean_name in clean_header_fields:
            clean_name = f"{original_clean_name}_{counter}"
            counter += 1
        clean_header_fields.append(clean_name)

    print(f"📌 Header detected in {path} at raw file line {header_index_rdd} with {len(clean_header_fields)} columns: {clean_header_fields}")

    # Step 3: Define bad records path and cleanup for this file
    filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    # Using a distinct path for bad records collected via _corrupt_record column
    bad_records_output_path = f"{output_path}/_bad_records_corrupt_col/{filename}"

    try:
        fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
        hadoop_bad_path = spark._jvm.org.apache.hadoop.fs.Path(bad_records_output_path)
        if fs.exists(hadoop_bad_path):
            print(f"🧹 Cleaning up existing bad records path: {bad_records_output_path}")
            fs.delete(hadoop_bad_path, True)
    except Exception as cleanup_err:
        print(f"⚠️ Could not clean bad records path {bad_records_output_path}: {cleanup_err}")

    # Step 4: Read the ORIGINAL CSV file directly into a DataFrame, WITHOUT assuming header and WITHOUT inferring schema initially
    # This allows multiLine=True to work correctly across the entire file and _corrupt_record to be added.
    print(f"📚 Reading {path} directly with Spark's CSV reader (header=false, inferSchema=false).")
    print("   Malformed records will be captured in the '_corrupt_record' column if detected.")

    df_raw = (spark.read
              .format("csv")
              .option("header", "false") # CRITICAL: Do NOT let Spark use first line as header
              .option("inferSchema", "false") # CRITICAL: Do NOT infer schema here, allows _corrupt_record to be added
              .option("sep", ",")
              .option("quote", '"')
              .option("escape", '"')
              .option("mode", "PERMISSIVE") # Malformed records go to _corrupt_record column
              .option("multiLine", "true") # CRITICAL: Handles newlines within quoted fields
              .load(path))
    
    # Check if _corrupt_record column exists. It might not if no bad records are found AND
    # if Spark's internal logic optimized it away because inferSchema was not true.
    # To be safe, we will ensure it exists.
    if "_corrupt_record" not in df_raw.columns:
        df_raw = df_raw.withColumn("_corrupt_record", lit(None).cast(StringType()))
        print("💡 Added a null '_corrupt_record' column as none was found during initial load.")

    # Add a row index to the raw DataFrame to track relative order
    df_raw = df_raw.withColumn("raw_df_row_index", monotonically_increasing_id())

    # Identify the header row in df_raw by content matching
    header_row_df_match = None
    # Collect first few rows (e.g., up to lines_to_check * 2 or more) to find the header
    # Ensure to collect enough rows to cover cases where initial lines are boilerplate.
    potential_header_rows = df_raw.limit(max(lines_to_check * 2, 50)).collect() 

    for row in potential_header_rows:
        # We need to compare the actual parsed values of the row (e.g., _c0, _c1, ...)
        # with the `header_parsed_fields` obtained from the RDD scan.
        
        # Build a list of string values from the current row's _cX columns
        row_values_str = []
        for i in range(len(header_parsed_fields)): # Iterate up to the number of expected header fields
            col_name = f"_c{i}"
            if col_name in row.__fields__ and row[col_name] is not None:
                row_values_str.append(str(row[col_name]).strip())
            else:
                row_values_str.append("") # Treat missing/null as empty string for comparison

        # Compare field by field
        is_match = True
        # Compare all fields for an exact match, or a significant portion if the header is very long
        compare_limit = min(len(header_parsed_fields), len(row_values_str), 10) # Compare up to first 10 fields for robustness

        for i in range(compare_limit):
            if row_values_str[i].lower() != header_parsed_fields[i].lower():
                is_match = False
                break
        
        # A simple check: if the lengths of parsed fields roughly match, and is_match is True
        if is_match and row._corrupt_record is None and abs(len(row_values_str) - len(header_parsed_fields)) <= 5: # Allow small diff in col count for headers
            header_row_df_match = row
            break # Found the header row

    if header_row_df_match is None:
        raise ValueError(f"CRITICAL: Could not find detected header row by content in DataFrame for {path}. "
                         "This might indicate an issue with header detection logic or file format consistency. "
                         "Review the first {lines_to_check*2} rows and parsed header: {header_parsed_fields}")

    # Filter out rows that occurred before the header row (using the temp_row_id)
    # And also filter out the header row itself.
    data_df_with_corrupt = df_raw.filter(col("raw_df_row_index") > header_row_df_match.raw_df_row_index)
    
    # Now, separate valid and invalid rows from this filtered data
    valid_df_pre_schema = data_df_with_corrupt.filter(col("_corrupt_record").isNull())
    invalid_df = data_df_with_corrupt.filter(col("_corrupt_record").isNotNull())

    # Select only the relevant columns (_c0, _c1, ...) based on number of expected header fields
    selected_cols_for_data = [f"_c{i}" for i in range(len(clean_header_fields))]
    valid_df_for_schema_inference = valid_df_pre_schema.select(*selected_cols_for_data)

    # Now, infer schema on a sample of this valid data to get correct types
    # This is more robust than trying to manually cast.
    temp_infer_path = f"{output_path}/_temp_infer_schema/{filename}"
    try:
        if valid_df_for_schema_inference.count() > 0: # Ensure there's data to infer from
            print(f"✍️ Writing a sample of data to {temp_infer_path} for schema inference.")
            valid_df_for_schema_inference.limit(1000).write.mode("overwrite").csv(temp_infer_path) # Write a small sample
            
            # Read the sample back with inferSchema=True to get the schema
            print(f"🔍 Inferring schema from sample at {temp_infer_path}")
            inferred_schema_df = spark.read.csv(temp_infer_path, header=False, inferSchema=True)
            inferred_schema = inferred_schema_df.schema
            print("Schema inferred successfully from sample.")
        else:
            print("⚠️ No valid data rows found to infer schema from. Using all StringType schema.")
            inferred_schema = StructType([StructField(f"col_{i}", StringType(), True) for i in range(len(clean_header_fields))])

    except Exception as e:
        print(f"❌ Error during schema inference from sample: {e}. Falling back to all StringType schema.")
        inferred_schema = StructType([StructField(f"col_{i}", StringType(), True) for i in range(len(clean_header_fields))])
    finally:
        # Clean up the temporary infer schema path
        try:
            fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
            hadoop_infer_path = spark._jvm.org.apache.hadoop.fs.Path(temp_infer_path)
            if fs.exists(hadoop_infer_path):
                fs.delete(hadoop_infer_path, True)
                print(f"🧹 Cleaned up temporary schema inference path: {temp_infer_path}")
        except Exception as cleanup_err:
            print(f"⚠️ Could not clean temporary infer schema path {temp_infer_path}: {cleanup_err}")

    # Apply the inferred schema and rename columns
    final_df_cols = []
    for i, field_name in enumerate(clean_header_fields):
        try:
            # Ensure index is within inferred schema bounds
            if i < len(inferred_schema):
                field_type = inferred_schema[i].dataType # Get type from inferred schema
                final_df_cols.append(col(f"_c{i}").cast(field_type).alias(field_name))
            else:
                print(f"⚠️ Warning: Column _c{i} (for {field_name}) is beyond inferred schema length. Casting as string.")
                final_df_cols.append(col(f"_c{i}").cast(StringType()).alias(field_name))
        except Exception as cast_err:
            print(f"⚠️ Error casting column _c{i} ({field_name}): {cast_err}. Casting as string.")
            final_df_cols.append(col(f"_c{i}").cast(StringType()).alias(field_name))
    
    valid_df = valid_df_for_schema_inference.select(*final_df_cols)
    
    # Drop temporary columns from invalid_df before saving
    invalid_df = invalid_df.drop("raw_df_row_index", *[c for c in invalid_df.columns if c.startswith("_c") and c not in selected_cols_for_data])

    total_valid_rows = valid_df.count()
    total_invalid_rows = invalid_df.count()

    print(f"🚨 Malformed records found (via _corrupt_record column): {total_invalid_rows}")
    if total_invalid_rows > 0:
        print(f"   Saving malformed records to: {bad_records_output_path}")
        # Select the corrupt record column and the input file name for audit
        invalid_df.select(col("_corrupt_record"), input_file_name().alias("source_file")) \
                  .write.mode("overwrite").format("text").save(bad_records_output_path)

    print(f"✅ Final row count in loaded DataFrame (valid rows): {total_valid_rows}")
    print(f"Schema for {os.path.basename(path)}:")
    valid_df.printSchema() # Print schema of the valid DataFrame
    print(f"Data preview for {os.path.basename(path)} (valid rows):")
    valid_df.show(2, truncate=False)

    estimated_raw_lines = df_raw.count() # This is the total number of DF rows, not original file lines
    return valid_df, estimated_raw_lines, total_valid_rows, total_invalid_rows

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 16, Finished, Available, Finished)

In [15]:
# --- Main Loop (as provided by you, with minor adjustments for function return) ---

# IMPORTANT: Ensure 'output_path_' is defined before this block of code.
# For example:
# output_path_ = "abfss://<your_lakehouse_guid>@onelake.dfs.fabric.microsoft.com/<your_folder_or_path>/"
# Replace the placeholder with your actual OneLake temporary path.

# Setup
parsed_dfs = {}
successfully_loaded = []
failed_to_load = []

successfully_written = []
failed_to_write = []

TARGET_SCHEMA = "aims_dbo3"  # e.g., 'bronze', 'staging'
TABLE_SAVE_MODE = "overwrite"       # or "append"
output_path = output_path_  # Adjust to the OneLake temp path

# Assuming 'csv_files' is defined from your previous `list_csv_files_recursive` call
# If not, you'll need to define it here, e.g.:
# from notebookutils import mssparkutils
# def list_csv_files_recursive(base_path): ... (your function)
# csv_files = list_csv_files_recursive("Files/") # Or your actual base path for CSVs

# STEP 1: Load all CSVs
load_stats = {} # Initialize load_stats here for the summary
for path in csv_files:
    try:
        print(f"\n🔍 Inspecting: {path}")
        # Call the modified function
        df, raw_count, valid_count, invalid_count = detect_header_and_read_csv(path, output_path)
        parsed_dfs[path] = df
        successfully_loaded.append(path)

        # Store stats for summary
        load_stats[path] = {
            "raw_lines": raw_count,
            "valid_rows": valid_count,
            "invalid_rows": invalid_count
        }

        # df.printSchema() and df.show(2) are already inside the function now
    except Exception as e:
        print(f"❌ Error loading {path}: {e}")
        failed_to_load.append((path, str(e)))

# STEP 2: Write to tables
for path, df in parsed_dfs.items():
    table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    full_table_name = f"{TARGET_SCHEMA}.{table_name}"

    try:
        print(f"\n💾 Writing to table: {full_table_name}")
        df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
        print(f"✅ Successfully written: {full_table_name}")
        successfully_written.append(full_table_name)
    except Exception as e:
        print(f"❌ Failed to write {full_table_name}: {e}")
        failed_to_write.append((full_table_name, str(e)))

# STEP 3: Summary Report
print("\n📊 LOAD SUMMARY")
print(f"✅ Loaded: {len(successfully_loaded)} files")
print(f"❌ Failed to load: {len(failed_to_load)} files")
if failed_to_load:
    for path, err in failed_to_load:
        print(f"   - {path} → {err}")

print("\n📊 WRITE SUMMARY")
print(f"✅ Written to tables: {len(successfully_written)}")
print(f"❌ Failed to write: {len(failed_to_write)}")
if failed_to_write:
    for tbl, err in failed_to_write:
        print(f"   - {tbl} → {err}")

print("\n📋 ROW COUNT STATS")
for path, stats in load_stats.items():
    base = os.path.basename(path)
    print(f"📄 {base} → Raw: {stats['raw_lines']}, Valid: {stats['valid_rows']}, Invalid: {stats['invalid_rows']}")


StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 17, Finished, Available, Finished)


🔍 Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz
📌 Header detected in abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz at raw file line 1 with 76 columns: ['id', 'asset', 'attribute', 'textvalue', 'numericvalue', 'datevalue', 'decimalvalue2dp', 'selectionvalue', 'location', 'validfrom', 'validto', 'kino_csid', 'hs2_kino_cuid', 'kino_cwhen', 'kino_usid', 'kino_uuid', 'kino_uwhen', 'basedon', 'owner', 'phase', 'person', 'organisation', 'textstorefortypetransition', 'decimalstorefortypetransition', 'numericstorefortypetransition', 'startchainage', 'endchainage', 'numericendvalue', 'decimalendvalue2dp', 'isbeingsplit', 'splitatchainage', 'afternumericstartvalue', 'afternumericendvalue', 'afterdecimalstartvalue2dp', 'afterdecimalendvalue2dp', 'aftertextvalue', 'afterdatevalue', 'afterselec

In [16]:
# import os
# from urllib.parse import urlparse
# from pyspark.sql.functions import col, input_file_name, monotonically_increasing_id, lit
# import csv
# from io import StringIO
# from pyspark.sql.types import StringType, StructType, StructField

# def detect_header_and_read_csv(path, output_path, lines_to_check=100):
#     """
#     Detects header in a CSV file, reads it into a Spark DataFrame with inferred schema,
#     handles malformed records, and provides metadata.

#     Args:
#         path (str): The ABFSS path to the CSV or compressed CSV file (e.g., .csv.gz).
#         output_path (str): The base path where temporary files for schema inference
#                                  and bad records will be stored (e.g., "abfss://.../processed_data").
#         lines_to_check (int): Number of initial lines to check for header detection.

#     Returns:
#         tuple: A tuple containing:
#             - pyspark.sql.DataFrame: The loaded and schema-applied DataFrame with valid rows.
#             - int: Estimated total raw lines in the file (including header, corrupt).
#             - int: Count of valid rows in the final DataFrame.
#             - int: Count of malformed (corrupt) rows detected.
#     """
#     print(f"🔍 Inspecting: {path}")

#     # Step 0: Initial read of a small sample for header detection (using RDD)
#     # Using RDD to get raw lines and their 0-based indices
#     raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
#     preview = raw_lines_rdd.take(lines_to_check)

#     # If the file is empty, handle it
#     if not preview:
#         print(f"⚠️ Warning: File {path} is empty or contains no lines to preview.")
#         # Return an empty DataFrame with a generic schema if no lines are found
#         empty_schema = StructType([StructField("col", StringType(), True)])
#         return spark.createDataFrame([], empty_schema), 0, 0, 0

#     # Step 1: Score lines to find best candidate for header
#     # This heuristic tries to identify a header by looking for lines with
#     # many non-empty fields and non-numeric starting fields.
#     def score_line_for_header(line_tuple):
#         line = line_tuple[0] # The raw string content of the line
#         try:
#             fields = next(csv.reader(StringIO(line)))
#             # A simple score: number of non-empty fields
#             score = len([f for f in fields if f.strip() != ''])
#             # Heuristic: Penalize if the first few fields are purely numeric,
#             # as these often indicate data rows, not headers.
#             if score > 0 and all(f.strip().isdigit() for f in fields[:min(3, len(fields))]):
#                 score *= 0.5 # Halve the score if first 3 fields are numeric
#             return score
#         except Exception:
#             # If CSV parsing fails for a line, it's unlikely to be a header
#             return 0

#     # Find the line with the highest score as the potential header
#     header_line_tuple = max(preview, key=score_line_for_header)
#     header_index_rdd = header_line_tuple[1] # 0-based index of the detected header line
#     header_raw_string = header_line_tuple[0] # The raw string content of the detected header line

#     # Parse the header fields from the identified raw string
#     try:
#         # Use csv.reader for robust parsing of the header line (handles quotes, commas within fields)
#         header_parsed_fields = [col_name.strip().strip('"').replace('\ufeff', '') for col_name in next(csv.reader(StringIO(header_raw_string)))]
#     except Exception as e:
#         raise ValueError(f"CRITICAL: Could not parse header line '{header_raw_string}' from {path}. Error: {e}")

#     # Clean header names for Spark DataFrame compatibility (remove special chars, lowercase, handle duplicates)
#     clean_header_fields = []
#     for col_name in header_parsed_fields:
#         clean_name = col_name.replace('.', '_').replace(' ', '_').replace('-', '_').lower() # More robust cleaning
#         counter = 1
#         original_clean_name = clean_name
#         while clean_name in clean_header_fields: # Ensure unique column names
#             clean_name = f"{original_clean_name}_{counter}"
#             counter += 1
#         clean_header_fields.append(clean_name)

#     print(f"📌 Header detected in {path} at raw file line {header_index_rdd} with {len(clean_header_fields)} columns: {clean_header_fields}")

#     # Derive file-specific paths for temporary data
#     file_basename_no_ext = os.path.splitext(os.path.basename(path))[0].split('.')[0].replace(".", "_").lower()
    
#     # Construct paths for temporary files and bad records, using output_path
#     temp_infer_path = f"{output_path}/_temp_infer_schema/{file_basename_no_ext}"
#     bad_records_output_path = f"{output_path}/_bad_records_corrupt_col/{file_basename_no_ext}"

#     # Clean up old temporary directories if they exist
#     try:
#         fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
#         for temp_path in [temp_infer_path, bad_records_output_path]:
#             hadoop_path = spark._jvm.org.apache.hadoop.fs.Path(temp_path)
#             if fs.exists(hadoop_path):
#                 print(f"🧹 Cleaning up existing temporary path: {temp_path}")
#                 fs.delete(hadoop_path, True)
#     except Exception as cleanup_err:
#         print(f"⚠️ Could not clean temporary paths: {cleanup_err}")

#     # Determine the maximum number of columns needed for the raw read
#     # This is crucial to ensure _cN columns are available even if early rows are shorter
#     max_cols_for_raw_read = len(clean_header_fields) if clean_header_fields else 1 # Ensure at least _c0
#     raw_df_schema = StructType([StructField(f"_c{i}", StringType(), True) for i in range(max_cols_for_raw_read)])

#     # Step 2: Read the entire CSV file into a DataFrame using the explicitly defined raw schema.
#     # This fixes the _c1 not found issue by guaranteeing the column presence.
#     print(f"📚 Reading {path} directly with Spark's CSV reader using explicit schema (header=false, inferSchema=false).")
#     print("   Malformed records will be captured in the '_corrupt_record' column if detected.")

#     df_raw = (spark.read
#               .format("csv")
#               .schema(raw_df_schema) # Use the explicit schema here
#               .option("header", "false")
#               .option("inferSchema", "false") # Still false, as schema is provided explicitly
#               .option("sep", ",")
#               .option("quote", '"')
#               .option("escape", '"')
#               .option("mode", "PERMISSIVE")
#               .option("multiLine", "true") # Important for handling fields with newlines
#               .load(path))

#     # Add a _corrupt_record column if Spark didn't automatically add it (e.g., no corrupt records found yet)
#     if "_corrupt_record" not in df_raw.columns:
#         df_raw = df_raw.withColumn("_corrupt_record", lit(None).cast(StringType()))
#         print("💡 Added a null '_corrupt_record' column as none was found during initial load.")

#     # Add a row index to the raw DataFrame to easily filter out the header row later
#     df_raw = df_raw.withColumn("raw_df_row_index", monotonically_increasing_id())

#     # Step 3: Find the detected header row by content within the DataFrame
#     # Collect enough rows from the raw DataFrame to cover the RDD-detected header index
#     # and a buffer to find potential matches.
#     num_rows_to_collect = max(header_index_rdd + 20, 200) # Collect up to 20 lines after RDD header, or 200 total
#     potential_header_rows = df_raw.limit(num_rows_to_collect).collect()

#     header_row_df_match = None
#     best_match_score_percentage = -1.0

#     print(f"🕵️‍♀️ Attempting to find header row by content within the first {len(potential_header_rows)} rows of the DataFrame.")

#     # Iterate through collected rows to find the one whose content matches the parsed header fields
#     for row_idx, row in enumerate(potential_header_rows):
#         if row._corrupt_record is not None:
#             continue # Skip corrupt rows when searching for the header

#         row_values_str = []
#         # Extract values for comparison, handling potential missing columns if the row is shorter than header
#         for i in range(len(header_parsed_fields)):
#             col_name = f"_c{i}" # Spark uses _cN for columns when header=false
#             if col_name in row.__fields__ and row[col_name] is not None:
#                 row_values_str.append(str(row[col_name]).strip())
#             else:
#                 # If a _cN column isn't present in this specific row's schema (shouldn't happen with explicit schema now, but for safety)
#                 # or if the value is null, treat as empty for comparison
#                 row_values_str.append("")

#         current_match_score = 0
#         num_fields_to_compare = min(len(header_parsed_fields), len(row_values_str))

#         for i in range(num_fields_to_compare):
#             # Case-insensitive and whitespace-stripped comparison
#             if row_values_str[i].lower() == header_parsed_fields[i].lower():
#                 current_match_score += 1

#         match_percentage = current_match_score / len(header_parsed_fields) if len(header_parsed_fields) > 0 else 0

#         # Prioritize exact RDD index match if its content match is high (e.g., >= 90%)
#         # This prevents picking a data row that might coincidentally match a few header fields.
#         if row.raw_df_row_index == header_index_rdd and match_percentage >= 0.90:
#             header_row_df_match = row
#             break # Found the most reliable match, exit loop

#         # Otherwise, keep track of the row with the highest match percentage
#         if match_percentage > best_match_score_percentage:
#             best_match_score_percentage = match_percentage
#             header_row_df_match = row

#     if header_row_df_match is None:
#         raise ValueError(f"CRITICAL: Could not find detected header row by content in DataFrame for {path}. "
#                          f"RDD detected header at line {header_index_rdd}. "
#                          "This indicates an issue with header detection or file format consistency. "
#                          f"Review the first {num_rows_to_collect} rows and parsed header: {header_parsed_fields}")

#     print(f"Header row identified in DataFrame at raw_df_row_index: {header_row_df_match.raw_df_row_index}")

#     # Step 4: Separate header, valid data, and malformed records
#     # Filter out the header row and subsequent rows from the raw DataFrame
#     data_df_with_corrupt = df_raw.filter(col("raw_df_row_index") > header_row_df_match.raw_df_row_index)
#     valid_df_pre_schema = data_df_with_corrupt.filter(col("_corrupt_record").isNull())
#     invalid_df = data_df_with_corrupt.filter(col("_corrupt_record").isNotNull())

#     # Select only the relevant _cN columns for valid data based on the number of header fields
#     selected_cols_for_data = [f"_c{i}" for i in range(len(clean_header_fields))]
#     valid_df_for_schema_inference = valid_df_pre_schema.select(*selected_cols_for_data)

#     # Step 5: Infer schema from a sample of the valid data
#     inferred_schema = None
#     try:
#         if valid_df_for_schema_inference.count() > 0:
#             print(f"✍️ Writing a sample of data to {temp_infer_path} for schema inference.")
#             # Write a small sample to disk for Spark to infer schema reliably
#             valid_df_for_schema_inference.limit(10000).write.mode("overwrite").csv(temp_infer_path, header=False) # Write without header

#             print(f"🔍 Inferring schema from sample at {temp_infer_path}")
#             # Read the sample with inferSchema=True. Crucially, set header=False as we didn't write one.
#             inferred_schema_df = spark.read.csv(temp_infer_path, header=False, inferSchema=True, sep=",", quote='"', escape='"', multiLine=True)
#             inferred_schema = inferred_schema_df.schema
#             print("Schema inferred successfully from sample.")
#         else:
#             print("⚠️ No valid data rows found to infer schema from. Using all StringType schema.")
#             # If no valid data, create a schema with all StringType based on header count
#             inferred_schema = StructType([StructField(field_name, StringType(), True) for field_name in clean_header_fields])

#     except Exception as e:
#         print(f"❌ Error during schema inference from sample: {e}. Falling back to all StringType schema.")
#         inferred_schema = StructType([StructField(field_name, StringType(), True) for field_name in clean_header_fields])
#     finally:
#         # Clean up temporary schema inference path
#         try:
#             fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
#             hadoop_infer_path = spark._jvm.org.apache.hadoop.fs.Path(temp_infer_path)
#             if fs.exists(hadoop_infer_path):
#                 fs.delete(hadoop_infer_path, True)
#                 print(f"🧹 Cleaned up temporary schema inference path: {temp_infer_path}")
#         except Exception as cleanup_err:
#             print(f"⚠️ Could not clean temporary infer schema path {temp_infer_path}: {cleanup_err}")

#     # Step 6: Apply the inferred schema and clean column names to the valid data DataFrame
#     final_df_cols = []
#     for i, field_name in enumerate(clean_header_fields):
#         # Apply inferred type or default to StringType if inference failed or column index out of bounds
#         try:
#             if inferred_schema and i < len(inferred_schema):
#                 field_type = inferred_schema[i].dataType
#                 final_df_cols.append(col(f"_c{i}").cast(field_type).alias(field_name))
#             else:
#                 final_df_cols.append(col(f"_c{i}").cast(StringType()).alias(field_name))
#         except Exception as cast_err:
#             print(f"⚠️ Error casting column _c{i} ({field_name}): {cast_err}. Casting as string.")
#             final_df_cols.append(col(f"_c{i}").cast(StringType()).alias(field_name))
    
#     # Select columns with their new names and types
#     valid_df = valid_df_for_schema_inference.select(*final_df_cols)
    
#     # Clean up the invalid DataFrame to keep only _corrupt_record and source_file for logging
#     invalid_df = invalid_df.drop("raw_df_row_index", *[c for c in invalid_df.columns if c.startswith("_c") and c != "_corrupt_record"])

#     total_valid_rows = valid_df.count()
#     total_invalid_rows = invalid_df.count()
#     estimated_raw_lines = df_raw.count()

#     print(f"🚨 Malformed records found (via _corrupt_record column): {total_invalid_rows}")
#     if total_invalid_rows > 0:
#         print(f"   Saving malformed records to: {bad_records_output_path}")
#         # Save malformed records as text to avoid further parsing issues
#         invalid_df.select(col("_corrupt_record"), input_file_name().alias("source_file")) \
#                   .write.mode("overwrite").format("text").save(bad_records_output_path)

#     print(f"✅ Final row count in loaded DataFrame (valid rows): {total_valid_rows}")
#     print(f"Schema for {os.path.basename(path)}:")
#     valid_df.printSchema()
#     print(f"Data preview for {os.path.basename(path)} (valid rows):")
#     valid_df.show(2, truncate=False)

#     return valid_df, estimated_raw_lines, total_valid_rows, total_invalid_rows

# # --- Example Usage (place this in a PySpark environment) ---
# # Assuming 'spark' session is already available. If not, initialize it:
# # from pyspark.sql import SparkSession
# # spark = SparkSession.builder.appName("CSVLoader").getOrCreate()

# # Define your input and a base output path for temporary/bad records
# base_input_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/"
# base_output_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/v2_processed_raw_source_data_aims2025/"

# files_to_process = [
#     "AssetAttributes.csv.gz",
#     "AssetClassAttributes.csv",
#     "AssetClassChangeLogs.csv",
#     "AssetClassRelationships.csv",
#     "AssetClasses.csv",
#     "AssetHierarchyMap.csv.gz",
#     "AssetLocations.csv.gz",
#     "AssetStatuses.csv",
#     "Assets.csv.gz",
#     "AttributeDomainValues.csv",
#     "AttributeDomains.csv"
# ]

# # Dictionary to store results (DataFrames and metadata)
# processed_files_summary = {}

# for file_name in files_to_process:
#     full_path = os.path.join(base_input_path, file_name)
#     try:
#         df, raw_lines, valid_rows, invalid_rows = detect_header_and_read_csv(full_path, base_output_path)
#         processed_files_summary[file_name] = {
#             "status": "SUCCESS",
#             "dataframe": df,
#             "raw_lines": raw_lines,
#             "valid_rows": valid_rows,
#             "invalid_rows": invalid_rows
#         }
#     except Exception as e:
#         print(f"❌ Failed to process {file_name}: {e}")
#         processed_files_summary[file_name] = {
#             "status": "FAILED",
#             "error": str(e)
#         }

# print("\n--- Processing Summary ---")
# for file_name, result in processed_files_summary.items():
#     if result["status"] == "SUCCESS":
#         print(f"✅ {file_name}: Successfully loaded. Valid Rows: {result['valid_rows']}, Malformed Rows: {result['invalid_rows']}")
#     else:
#         print(f"❌ {file_name}: Failed. Error: {result['error']}")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 18, Finished, Available, Finished)

In [17]:
import os
from urllib.parse import urlparse
from pyspark.sql.functions import col, input_file_name, monotonically_increasing_id, lit
import csv
from io import StringIO
from pyspark.sql.types import StringType, StructType, StructField

def detect_header_and_read_csv(path, output_path, lines_to_check=100):
    """
    Detects header in a CSV file, reads it into a Spark DataFrame with inferred schema,
    handles malformed records, and provides metadata.

    Args:
        path (str): The ABFSS path to the CSV or compressed CSV file (e.g., .csv.gz).
        output_path (str): The base path where temporary files for schema inference
                                 and bad records will be stored (e.g., "abfss://.../processed_data").
        lines_to_check (int): Number of initial lines to check for header detection.

    Returns:
        tuple: A tuple containing:
            - pyspark.sql.DataFrame: The loaded and schema-applied DataFrame with valid rows.
            - int: Estimated total raw lines in the file (including header, corrupt).
            - int: Count of valid rows in the final DataFrame.
            - int: Count of malformed (corrupt) rows detected.
    """
    print(f"🔍 Inspecting: {path}")

    # Step 0: Initial read of a small sample for header detection (using RDD)
    # Using RDD to get raw lines and their 0-based indices
    raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
    preview = raw_lines_rdd.take(lines_to_check)

    # If the file is empty, handle it
    if not preview:
        print(f"⚠️ Warning: File {path} is empty or contains no lines to preview.")
        # Return an empty DataFrame with a generic schema if no lines are found
        empty_schema = StructType([StructField("col", StringType(), True)])
        return spark.createDataFrame([], empty_schema), 0, 0, 0

    # Step 1: Score lines to find best candidate for header
    # This heuristic tries to identify a header by looking for lines with
    # many non-empty fields and non-numeric starting fields.
    def score_line_for_header(line_tuple):
        line = line_tuple[0] # The raw string content of the line
        try:
            fields = next(csv.reader(StringIO(line)))
            # A simple score: number of non-empty fields
            score = len([f for f in fields if f.strip() != ''])
            # Heuristic: Penalize if the first few fields are purely numeric,
            # as these often indicate data rows, not headers.
            if score > 0 and all(f.strip().isdigit() for f in fields[:min(3, len(fields))]):
                score *= 0.5 # Halve the score if first 3 fields are numeric
            return score
        except Exception:
            # If CSV parsing fails for a line, it's unlikely to be a header
            return 0

    # Find the line with the highest score as the potential header
    header_line_tuple = max(preview, key=score_line_for_header)
    header_index_rdd = header_line_tuple[1] # 0-based index of the detected header line
    header_raw_string = header_line_tuple[0] # The raw string content of the detected header line

    # Parse the header fields from the identified raw string
    try:
        # Use csv.reader for robust parsing of the header line (handles quotes, commas within fields)
        header_parsed_fields = [col_name.strip().strip('"').replace('\ufeff', '') for col_name in next(csv.reader(StringIO(header_raw_string)))]
    except Exception as e:
        raise ValueError(f"CRITICAL: Could not parse header line '{header_raw_string}' from {path}. Error: {e}")

    # Clean header names for Spark DataFrame compatibility (remove special chars, lowercase, handle duplicates)
    clean_header_fields = []
    for col_name in header_parsed_fields:
        clean_name = col_name.replace('.', '_').replace(' ', '_').replace('-', '_').lower() # More robust cleaning
        counter = 1
        original_clean_name = clean_name
        while clean_name in clean_header_fields: # Ensure unique column names
            clean_name = f"{original_clean_name}_{counter}"
            counter += 1
        clean_header_fields.append(clean_name)

    print(f"📌 Header detected in {path} at raw file line {header_index_rdd} with {len(clean_header_fields)} columns: {clean_header_fields}")

    # Derive file-specific paths for temporary data
    file_basename_no_ext = os.path.splitext(os.path.basename(path))[0].split('.')[0].replace(".", "_").lower()
    
    # Construct paths for temporary files and bad records, using output_path
    temp_infer_path = f"{output_path}/_temp_infer_schema/{file_basename_no_ext}"
    bad_records_output_path = f"{output_path}/_bad_records_corrupt_col/{file_basename_no_ext}"

    # Clean up old temporary directories if they exist
    try:
        fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
        for temp_path in [temp_infer_path, bad_records_output_path]:
            hadoop_path = spark._jvm.org.apache.hadoop.fs.Path(temp_path)
            if fs.exists(hadoop_path):
                print(f"🧹 Cleaning up existing temporary path: {temp_path}")
                fs.delete(hadoop_path, True)
    except Exception as cleanup_err:
        print(f"⚠️ Could not clean temporary paths: {cleanup_err}")

    # Determine the maximum number of columns needed for the raw read
    # This is crucial to ensure _cN columns are available even if early rows are shorter
    max_cols_for_raw_read = len(clean_header_fields) if clean_header_fields else 1 # Ensure at least _c0
    raw_df_schema = StructType([StructField(f"_c{i}", StringType(), True) for i in range(max_cols_for_raw_read)])

    # Step 2: Read the entire CSV file into a DataFrame using the explicitly defined raw schema.
    # This fixes the _c1 not found issue by guaranteeing the column presence.
    print(f"📚 Reading {path} directly with Spark's CSV reader using explicit schema (header=false, inferSchema=false).")
    print("   Malformed records will be captured in the '_corrupt_record' column if detected.")

    df_raw = (spark.read
              .format("csv")
              .schema(raw_df_schema) # Use the explicit schema here
              .option("header", "false")
              .option("inferSchema", "false") # Still false, as schema is provided explicitly
              .option("sep", ",")
              .option("quote", '"')
              .option("escape", '"')
              .option("mode", "PERMISSIVE")
              .option("multiLine", "true") # Important for handling fields with newlines
              .load(path))

    # Add a _corrupt_record column if Spark didn't automatically add it (e.g., no corrupt records found yet)
    if "_corrupt_record" not in df_raw.columns:
        df_raw = df_raw.withColumn("_corrupt_record", lit(None).cast(StringType()))
        print("💡 Added a null '_corrupt_record' column as none was found during initial load.")

    # Add a row index to the raw DataFrame to easily filter out the header row later
    df_raw = df_raw.withColumn("raw_df_row_index", monotonically_increasing_id())

    # Step 3: Find the detected header row by content within the DataFrame
    # Collect enough rows from the raw DataFrame to cover the RDD-detected header index
    # and a buffer to find potential matches.
    num_rows_to_collect = max(header_index_rdd + 20, 200) # Collect up to 20 lines after RDD header, or 200 total
    potential_header_rows = df_raw.limit(num_rows_to_collect).collect()

    header_row_df_match = None
    best_match_score_percentage = -1.0

    print(f"🕵️‍♀️ Attempting to find header row by content within the first {len(potential_header_rows)} rows of the DataFrame.")

    # Iterate through collected rows to find the one whose content matches the parsed header fields
    for row_idx, row in enumerate(potential_header_rows):
        if row._corrupt_record is not None:
            continue # Skip corrupt rows when searching for the header

        row_values_str = []
        # Extract values for comparison, handling potential missing columns if the row is shorter than header
        for i in range(len(header_parsed_fields)):
            col_name = f"_c{i}" # Spark uses _cN for columns when header=false
            if col_name in row.__fields__ and row[col_name] is not None:
                row_values_str.append(str(row[col_name]).strip())
            else:
                # If a _cN column isn't present in this specific row's schema (shouldn't happen with explicit schema now, but for safety)
                # or if the value is null, treat as empty for comparison
                row_values_str.append("")

        current_match_score = 0
        num_fields_to_compare = min(len(header_parsed_fields), len(row_values_str))

        for i in range(num_fields_to_compare):
            # Case-insensitive and whitespace-stripped comparison
            if row_values_str[i].lower() == header_parsed_fields[i].lower():
                current_match_score += 1

        match_percentage = current_match_score / len(header_parsed_fields) if len(header_parsed_fields) > 0 else 0

        # Prioritize exact RDD index match if its content match is high (e.g., >= 90%)
        # This prevents picking a data row that might coincidentally match a few header fields.
        if row.raw_df_row_index == header_index_rdd and match_percentage >= 0.90:
            header_row_df_match = row
            break # Found the most reliable match, exit loop

        # Otherwise, keep track of the row with the highest match percentage
        if match_percentage > best_match_score_percentage:
            best_match_score_percentage = match_percentage
            header_row_df_match = row

    if header_row_df_match is None:
        raise ValueError(f"CRITICAL: Could not find detected header row by content in DataFrame for {path}. "
                         f"RDD detected header at line {header_index_rdd}. "
                         "This indicates an issue with header detection or file format consistency. "
                         f"Review the first {num_rows_to_collect} rows and parsed header: {header_parsed_fields}")

    print(f"Header row identified in DataFrame at raw_df_row_index: {header_row_df_match.raw_df_row_index}")

    # Step 4: Separate header, valid data, and malformed records
    # Filter out the header row and subsequent rows from the raw DataFrame
    data_df_with_corrupt = df_raw.filter(col("raw_df_row_index") > header_row_df_match.raw_df_row_index)
    valid_df_pre_schema = data_df_with_corrupt.filter(col("_corrupt_record").isNull())
    invalid_df = data_df_with_corrupt.filter(col("_corrupt_record").isNotNull())

    # Select only the relevant _cN columns for valid data based on the number of header fields
    selected_cols_for_data = [f"_c{i}" for i in range(len(clean_header_fields))]
    valid_df_for_schema_inference = valid_df_pre_schema.select(*selected_cols_for_data)

    # Step 5: Infer schema from a sample of the valid data
    inferred_schema = None
    try:
        if valid_df_for_schema_inference.count() > 0:
            print(f"✍️ Writing a sample of data to {temp_infer_path} for schema inference.")
            # Write a small sample to disk for Spark to infer schema reliably
            valid_df_for_schema_inference.limit(10000).write.mode("overwrite").csv(temp_infer_path, header=False) # Write without header

            print(f"🔍 Inferring schema from sample at {temp_infer_path}")
            # Read the sample with inferSchema=True. Crucially, set header=False as we didn't write one.
            inferred_schema_df = spark.read.csv(temp_infer_path, header=False, inferSchema=True, sep=",", quote='"', escape='"', multiLine=True)
            inferred_schema = inferred_schema_df.schema
            print("Schema inferred successfully from sample.")
        else:
            print("⚠️ No valid data rows found to infer schema from. Using all StringType schema.")
            # If no valid data, create a schema with all StringType based on header count
            inferred_schema = StructType([StructField(field_name, StringType(), True) for field_name in clean_header_fields])

    except Exception as e:
        print(f"❌ Error during schema inference from sample: {e}. Falling back to all StringType schema.")
        inferred_schema = StructType([StructField(field_name, StringType(), True) for field_name in clean_header_fields])
    finally:
        # Clean up temporary schema inference path
        try:
            fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
            hadoop_infer_path = spark._jvm.org.apache.hadoop.fs.Path(temp_infer_path)
            if fs.exists(hadoop_infer_path):
                fs.delete(hadoop_infer_path, True)
                print(f"🧹 Cleaned up temporary schema inference path: {temp_infer_path}")
        except Exception as cleanup_err:
            print(f"⚠️ Could not clean temporary infer schema path {temp_infer_path}: {cleanup_err}")

    # Step 6: Apply the inferred schema and clean column names to the valid data DataFrame
    final_df_cols = []
    for i, field_name in enumerate(clean_header_fields):
        # Apply inferred type or default to StringType if inference failed or column index out of bounds
        try:
            if inferred_schema and i < len(inferred_schema):
                field_type = inferred_schema[i].dataType
                final_df_cols.append(col(f"_c{i}").cast(field_type).alias(field_name))
            else:
                final_df_cols.append(col(f"_c{i}").cast(StringType()).alias(field_name))
        except Exception as cast_err:
            print(f"⚠️ Error casting column _c{i} ({field_name}): {cast_err}. Casting as string.")
            final_df_cols.append(col(f"_c{i}").cast(StringType()).alias(field_name))
    
    # Select columns with their new names and types
    valid_df = valid_df_for_schema_inference.select(*final_df_cols)
    
    # Clean up the invalid DataFrame to keep only _corrupt_record and source_file for logging
    invalid_df = invalid_df.drop("raw_df_row_index", *[c for c in invalid_df.columns if c.startswith("_c") and c != "_corrupt_record"])

    total_valid_rows = valid_df.count()
    total_invalid_rows = invalid_df.count()
    estimated_raw_lines = df_raw.count()

    print(f"🚨 Malformed records found (via _corrupt_record column): {total_invalid_rows}")
    if total_invalid_rows > 0:
        print(f"   Saving malformed records to: {bad_records_output_path}")
        # Save malformed records as text to avoid further parsing issues
        invalid_df.select(col("_corrupt_record"), input_file_name().alias("source_file")) \
                  .write.mode("overwrite").format("text").save(bad_records_output_path)

    print(f"✅ Final row count in loaded DataFrame (valid rows): {total_valid_rows}")
    print(f"Schema for {os.path.basename(path)}:")
    valid_df.printSchema()
    print(f"Data preview for {os.path.basename(path)} (valid rows):")
    valid_df.show(2, truncate=False)

    return valid_df, estimated_raw_lines, total_valid_rows, total_invalid_rows

# --- Example Usage (place this in a PySpark environment) ---
# Assuming 'spark' session is already available. If not, initialize it:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("CSVLoader").getOrCreate()

# Define your input and a base output path for temporary/bad records
base_input_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/"
base_output_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/v2_processed_raw_source_data_aims2025/"

files_to_process = [
    "AssetAttributes.csv.gz",
    "AssetClassAttributes.csv",
    "AssetClassChangeLogs.csv",
    "AssetClassRelationships.csv",
    "AssetClasses.csv",
    "AssetHierarchyMap.csv.gz",
    "AssetLocations.csv.gz",
    "AssetStatuses.csv",
    "Assets.csv.gz",
    "AttributeDomainValues.csv",
    "AttributeDomains.csv"
]

# Dictionary to store results (DataFrames and metadata)
processed_files_summary = {}

for file_name in files_to_process:
    full_path = os.path.join(base_input_path, file_name)
    try:
        df, raw_lines, valid_rows, invalid_rows = detect_header_and_read_csv(full_path, base_output_path)
        processed_files_summary[file_name] = {
            "status": "SUCCESS",
            "dataframe": df,
            "raw_lines": raw_lines,
            "valid_rows": valid_rows,
            "invalid_rows": invalid_rows
        }
    except Exception as e:
        print(f"❌ Failed to process {file_name}: {e}")
        processed_files_summary[file_name] = {
            "status": "FAILED",
            "error": str(e)
        }

print("\n--- Processing Summary ---")
for file_name, result in processed_files_summary.items():
    if result["status"] == "SUCCESS":
        print(f"✅ {file_name}: Successfully loaded. Valid Rows: {result['valid_rows']}, Malformed Rows: {result['invalid_rows']}")
    else:
        print(f"❌ {file_name}: Failed. Error: {result['error']}")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 19, Finished, Available, Finished)

🔍 Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz
📌 Header detected in abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz at raw file line 1 with 76 columns: ['id', 'asset', 'attribute', 'textvalue', 'numericvalue', 'datevalue', 'decimalvalue2dp', 'selectionvalue', 'location', 'validfrom', 'validto', 'kino_csid', 'hs2_kino_cuid', 'kino_cwhen', 'kino_usid', 'kino_uuid', 'kino_uwhen', 'basedon', 'owner', 'phase', 'person', 'organisation', 'textstorefortypetransition', 'decimalstorefortypetransition', 'numericstorefortypetransition', 'startchainage', 'endchainage', 'numericendvalue', 'decimalendvalue2dp', 'isbeingsplit', 'splitatchainage', 'afternumericstartvalue', 'afternumericendvalue', 'afterdecimalstartvalue2dp', 'afterdecimalendvalue2dp', 'aftertextvalue', 'afterdatevalue', 'afterselect

##### This code appears to load all correctly. 

In [18]:
import os
from urllib.parse import urlparse
from pyspark.sql.functions import col, input_file_name, monotonically_increasing_id, lit
import csv
from io import StringIO
from pyspark.sql.types import StringType, StructType, StructField

def detect_header_and_read_csv(path, output_path, lines_to_check=5000):
    """
    Detects header in a CSV file, reads it into a Spark DataFrame with inferred schema,
    handles malformed records, and provides metadata.

    Args:
        path (str): The ABFSS path to the CSV or compressed CSV file (e.g., .csv.gz).
        output_path (str): The base path where temporary files for schema inference
                                 and bad records will be stored (e.g., "abfss://.../processed_data").
        lines_to_check (int): Number of initial lines to check for header detection.

    Returns:
        tuple: A tuple containing:
            - pyspark.sql.DataFrame: The loaded and schema-applied DataFrame with valid rows.
            - int: Estimated total raw lines in the file (including header, corrupt).
            - int: Count of rows in the DataFrame after header removal, before validation split.
            - int: Count of valid rows in the final DataFrame.
            - int: Count of malformed (corrupt) rows detected.
    """
    print(f"🔍 Inspecting: {path}")

    # Step 0: Initial read of a small sample for header detection (using RDD)
    # Using RDD to get raw lines and their 0-based indices
    raw_lines_rdd = spark.sparkContext.textFile(path).zipWithIndex()
    preview = raw_lines_rdd.take(lines_to_check)

    # If the file is empty, handle it
    if not preview:
        print(f"⚠️ Warning: File {path} is empty or contains no lines to preview.")
        # Return an empty DataFrame with a generic schema if no lines are found
        empty_schema = StructType([StructField("col", StringType(), True)])
        # Updated return to include rows_after_header_count
        return spark.createDataFrame([], empty_schema), 0, 0, 0, 0

    # Step 1: Score lines to find best candidate for header
    # This heuristic tries to identify a header by looking for lines with
    # many non-empty fields and non-numeric starting fields.
    def score_line_for_header(line_tuple):
        line = line_tuple[0] # The raw string content of the line
        try:
            fields = next(csv.reader(StringIO(line)))
            # A simple score: number of non-empty fields
            score = len([f for f in fields if f.strip() != ''])
            # Heuristic: Penalize if the first few fields are purely numeric,
            # as these often indicate data rows, not headers.
            if score > 0 and all(f.strip().isdigit() for f in fields[:min(3, len(fields))]):
                score *= 0.5 # Halve the score if first 3 fields are numeric
            return score
        except Exception:
            # If CSV parsing fails for a line, it's unlikely to be a header
            return 0

    # Find the line with the highest score as the potential header
    header_line_tuple = max(preview, key=score_line_for_header)
    header_index_rdd = header_line_tuple[1] # 0-based index of the detected header line
    header_raw_string = header_line_tuple[0] # The raw string content of the detected header line

    # Parse the header fields from the identified raw string
    try:
        # Use csv.reader for robust parsing of the header line (handles quotes, commas within fields)
        header_parsed_fields = [col_name.strip().strip('"').replace('\ufeff', '') for col_name in next(csv.reader(StringIO(header_raw_string)))]
    except Exception as e:
        raise ValueError(f"CRITICAL: Could not parse header line '{header_raw_string}' from {path}. Error: {e}")

    # Clean header names for Spark DataFrame compatibility (remove special chars, lowercase, handle duplicates)
    clean_header_fields = []
    for col_name in header_parsed_fields:
        clean_name = col_name.replace('.', '_').replace(' ', '_').replace('-', '_').lower() # More robust cleaning
        counter = 1
        original_clean_name = clean_name
        while clean_name in clean_header_fields: # Ensure unique column names
            clean_name = f"{original_clean_name}_{counter}"
            counter += 1
        clean_header_fields.append(clean_name)

    print(f"📌 Header detected in {path} at raw file line {header_index_rdd} with {len(clean_header_fields)} columns: {clean_header_fields}")

    # Derive file-specific paths for temporary data
    file_basename_no_ext = os.path.splitext(os.path.basename(path))[0].split('.')[0].replace(".", "_").lower()
    
    # Construct paths for temporary files and bad records, using output_path
    temp_infer_path = f"{output_path}/_temp_infer_schema/{file_basename_no_ext}"
    bad_records_output_path = f"{output_path}/_bad_records_corrupt_col/{file_basename_no_ext}"

    # Clean up old temporary directories if they exist
    try:
        fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
        for temp_path in [temp_infer_path, bad_records_output_path]:
            hadoop_path = spark._jvm.org.apache.hadoop.fs.Path(temp_path)
            if fs.exists(hadoop_path):
                print(f"🧹 Cleaning up existing temporary path: {temp_path}")
                fs.delete(hadoop_path, True)
    except Exception as cleanup_err:
        print(f"⚠️ Could not clean temporary paths: {cleanup_err}")

    # Determine the maximum number of columns needed for the raw read
    # This is crucial to ensure _cN columns are available even if early rows are shorter
    max_cols_for_raw_read = len(clean_header_fields) if clean_header_fields else 1 # Ensure at least _c0
    raw_df_schema = StructType([StructField(f"_c{i}", StringType(), True) for i in range(max_cols_for_raw_read)])

    # Step 2: Read the entire CSV file into a DataFrame using the explicitly defined raw schema.
    # This fixes the _c1 not found issue by guaranteeing the column presence.
    print(f"📚 Reading {path} directly with Spark's CSV reader using explicit schema (header=false, inferSchema=false).")
    print("   Malformed records will be captured in the '_corrupt_record' column if detected.")

    df_raw = (spark.read
              .format("csv")
              .schema(raw_df_schema) # Use the explicit schema here
              .option("header", "false")
              .option("inferSchema", "false") # Still false, as schema is provided explicitly
              .option("sep", ",")
              .option("quote", '"')
              .option("escape", '"')
              .option("mode", "PERMISSIVE")
              .option("multiLine", "true") # Important for handling fields with newlines
              .load(path))

    # Add a _corrupt_record column if Spark didn't automatically add it (e.g., no corrupt records found yet)
    if "_corrupt_record" not in df_raw.columns:
        df_raw = df_raw.withColumn("_corrupt_record", lit(None).cast(StringType()))
        print("💡 Added a null '_corrupt_record' column as none was found during initial load.")

    # Add a row index to the raw DataFrame to easily filter out the header row later
    df_raw = df_raw.withColumn("raw_df_row_index", monotonically_increasing_id())

    # Step 3: Find the detected header row by content within the DataFrame
    # Collect enough rows from the raw DataFrame to cover the RDD-detected header index
    # and a buffer to find potential matches.
    num_rows_to_collect = max(header_index_rdd + 200, 2000) # original -> Collect up to 20 lines after RDD header, or 200 total
    potential_header_rows = df_raw.limit(num_rows_to_collect).collect()

    header_row_df_match = None
    best_match_score_percentage = -1.0

    print(f"🕵️‍♀️ Attempting to find header row by content within the first {len(potential_header_rows)} rows of the DataFrame.")

    # Iterate through collected rows to find the one whose content matches the parsed header fields
    for row_idx, row in enumerate(potential_header_rows):
        if row._corrupt_record is not None:
            continue # Skip corrupt rows when searching for the header

        row_values_str = []
        # Extract values for comparison, handling potential missing columns if the row is shorter than header
        for i in range(len(header_parsed_fields)):
            col_name = f"_c{i}" # Spark uses _cN for columns when header=false
            if col_name in row.__fields__ and row[col_name] is not None:
                row_values_str.append(str(row[col_name]).strip())
            else:
                # If a _cN column isn't present in this specific row's schema (shouldn't happen with explicit schema now, but for safety)
                # or if the value is null, treat as empty for comparison
                row_values_str.append("")

        current_match_score = 0
        num_fields_to_compare = min(len(header_parsed_fields), len(row_values_str))

        for i in range(num_fields_to_compare):
            # Case-insensitive and whitespace-stripped comparison
            if row_values_str[i].lower() == header_parsed_fields[i].lower():
                current_match_score += 1

        match_percentage = current_match_score / len(header_parsed_fields) if len(header_parsed_fields) > 0 else 0

        # Prioritize exact RDD index match if its content match is high (e.g., >= 90%)
        # This prevents picking a data row that might coincidentally match a few header fields.
        if row.raw_df_row_index == header_index_rdd and match_percentage >= 0.90:
            header_row_df_match = row
            break # Found the most reliable match, exit loop

        # Otherwise, keep track of the row with the highest match percentage
        if match_percentage > best_match_score_percentage:
            best_match_score_percentage = match_percentage
            header_row_df_match = row

    if header_row_df_match is None:
        raise ValueError(f"CRITICAL: Could not find detected header row by content in DataFrame for {path}. "
                         f"RDD detected header at line {header_index_rdd}. "
                         "This indicates an issue with header detection or file format consistency. "
                         f"Review the first {num_rows_to_collect} rows and parsed header: {header_parsed_fields}")

    print(f"Header row identified in DataFrame at raw_df_row_index: {header_row_df_match.raw_df_row_index}")

    # Step 4: Separate header, valid data, and malformed records
    # Filter out the header row and subsequent rows from the raw DataFrame
    data_df_with_corrupt = df_raw.filter(col("raw_df_row_index") > header_row_df_match.raw_df_row_index)
    
    # NEW: Get the count of rows after the header but before splitting into valid/invalid
    rows_after_header_count = data_df_with_corrupt.count()
    print(f"ℹ️ Rows in CSV after header (before validation split): {rows_after_header_count}")

    valid_df_pre_schema = data_df_with_corrupt.filter(col("_corrupt_record").isNull())
    invalid_df = data_df_with_corrupt.filter(col("_corrupt_record").isNotNull())

    # Select only the relevant _cN columns for valid data based on the number of header fields
    selected_cols_for_data = [f"_c{i}" for i in range(len(clean_header_fields))]
    valid_df_for_schema_inference = valid_df_pre_schema.select(*selected_cols_for_data)

    # Step 5: Infer schema from a sample of the valid data
    inferred_schema = None
    try:
        if valid_df_for_schema_inference.count() > 0:
            print(f"✍️ Writing a sample of data to {temp_infer_path} for schema inference.")
            # Write a small sample to disk for Spark to infer schema reliably
            valid_df_for_schema_inference.limit(10000).write.mode("overwrite").csv(temp_infer_path, header=False) # Write without header

            print(f"🔍 Inferring schema from sample at {temp_infer_path}")
            # Read the sample with inferSchema=True. Crucially, set header=False as we didn't write one.
            inferred_schema_df = spark.read.csv(temp_infer_path, header=False, inferSchema=True, sep=",", quote='"', escape='"', multiLine=True)
            inferred_schema = inferred_schema_df.schema
            print("Schema inferred successfully from sample.")
        else:
            print("⚠️ No valid data rows found to infer schema from. Using all StringType schema.")
            # If no valid data, create a schema with all StringType based on header count
            inferred_schema = StructType([StructField(field_name, StringType(), True) for field_name in clean_header_fields])

    except Exception as e:
        print(f"❌ Error during schema inference from sample: {e}. Falling back to all StringType schema.")
        inferred_schema = StructType([StructField(field_name, StringType(), True) for field_name in clean_header_fields])
    finally:
        # Clean up temporary schema inference path
        try:
            fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
            hadoop_infer_path = spark._jvm.org.apache.hadoop.fs.Path(temp_infer_path)
            if fs.exists(hadoop_infer_path):
                fs.delete(hadoop_infer_path, True)
                print(f"🧹 Cleaned up temporary schema inference path: {temp_infer_path}")
        except Exception as cleanup_err:
            print(f"⚠️ Could not clean temporary infer schema path {temp_infer_path}: {cleanup_err}")

    # Step 6: Apply the inferred schema and clean column names to the valid data DataFrame
    final_df_cols = []
    for i, field_name in enumerate(clean_header_fields):
        # Apply inferred type or default to StringType if inference failed or column index out of bounds
        try:
            if inferred_schema and i < len(inferred_schema):
                field_type = inferred_schema[i].dataType
                final_df_cols.append(col(f"_c{i}").cast(field_type).alias(field_name))
            else:
                final_df_cols.append(col(f"_c{i}").cast(StringType()).alias(field_name))
        except Exception as cast_err:
            print(f"⚠️ Error casting column _c{i} ({field_name}): {cast_err}. Casting as string.")
            final_df_cols.append(col(f"_c{i}").cast(StringType()).alias(field_name))
    
    # Select columns with their new names and types
    valid_df = valid_df_for_schema_inference.select(*final_df_cols)
    
    # Clean up the invalid DataFrame to keep only _corrupt_record and source_file for logging
    invalid_df = invalid_df.drop("raw_df_row_index", *[c for c in invalid_df.columns if c.startswith("_c") and c != "_corrupt_record"])

    total_valid_rows = valid_df.count()
    total_invalid_rows = invalid_df.count()
    estimated_raw_lines = df_raw.count() # This is the count of rows in the raw DataFrame, including header and corrupt rows

    print(f"🚨 Malformed records found (via _corrupt_record column): {total_invalid_rows}")
    if total_invalid_rows > 0:
        print(f"   Saving malformed records to: {bad_records_output_path}")
        # Save malformed records as text to avoid further parsing issues
        invalid_df.select(col("_corrupt_record"), input_file_name().alias("source_file")) \
                  .write.mode("overwrite").format("text").save(bad_records_output_path)

    print(f"✅ Final row count in loaded DataFrame (valid rows): {total_valid_rows}")
    print(f"Schema for {os.path.basename(path)}:")
    valid_df.printSchema()
    print(f"Data preview for {os.path.basename(path)} (valid rows):")
    valid_df.show(2, truncate=False)

    # Updated return value to include rows_after_header_count
    return valid_df, estimated_raw_lines, rows_after_header_count, total_valid_rows, total_invalid_rows

# --- Main Execution Block ---
# Assuming 'spark' session is already available. If not, initialize it:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("CSVLoader").getOrCreate()

# Define your input and a base output path for temporary/bad records
base_input_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/"
base_output_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/v2_processed_raw_source_data_aims2025/"

# List of files to process (replace with your actual csv_files list if dynamically generated)
files_to_process =  [
    "AssetAttributes.csv.gz",
    "AssetClassAttributes.csv",
    "AssetClassChangeLogs.csv",
    "AssetClassRelationships.csv",
    "AssetClasses.csv",
    "AssetHierarchyMap.csv.gz", # Note: Changed back to .gz as per your initial list
    "AssetLocations.csv.gz",
    "AssetStatuses.csv",
    "Assets.csv.gz",
    "AttributeDomainValues.csv",
    "AttributeDomains.csv",
    "AttributeGroups.csv",
    "Attributes.csv",
    "ChainageBaselines.csv",
    "LinkTypes.csv",
    "Links.csv",
    "Organisations.csv",
    "Owners.csv",
    "People.csv",
    "Phases.csv",
    "RelationshipTypes.csv",
    "Relationships.csv.gz",
    "Routes.csv",
    "SecondaryAssetClassCodes.csv",
    "SecondaryCodingSystems.csv",
    "Stages.csv",
    "Tracks.csv",
    "Users.csv"
]

# Dictionary to store results (DataFrames and metadata)
processed_files_summary = {}

for file_name in files_to_process:
    full_path = os.path.join(base_input_path, file_name)
    try:
        # Unpack the 5 return values
        df, raw_lines, rows_after_header, valid_rows, invalid_rows = detect_header_and_read_csv(full_path, base_output_path)
        processed_files_summary[file_name] = {
            "status": "SUCCESS",
            "dataframe": df,
            "raw_lines": raw_lines,
            "rows_after_header": rows_after_header, # Storing the new metric
            "valid_rows": valid_rows,
            "invalid_rows": invalid_rows
        }
    except Exception as e:
        print(f"❌ Failed to process {file_name}: {e}")
        processed_files_summary[file_name] = {
            "status": "FAILED",
            "error": str(e)
        }

print("\n--- Processing Summary ---")
# Updated header to include "Rows After Header"
print(f"{'File Name':<40} | {'Raw Rows':>12} | {'Rows After Header':>18} | {'Valid Rows':>12} | {'Invalid Rows':>12} | {'Delta (%)':>10} | Status")
print("-" * 130) # Adjusted line length

for file_name, result in processed_files_summary.items():
    if result["status"] == "SUCCESS":
        raw = result['raw_lines']
        rows_after_header = result['rows_after_header'] # Retrieve the new metric
        valid = result['valid_rows']
        invalid = result['invalid_rows']
        
        # Calculate Delta (%) based on Rows After Header vs Valid Rows
        delta_percentage = ((rows_after_header - valid) / rows_after_header * 100) if rows_after_header > 0 else 0
        
        print(f"{file_name:<40} | {raw:>12} | {rows_after_header:>18} | {valid:>12} | {invalid:>12} | {delta_percentage:>9.2f}% | SUCCESS")
    else:
        print(f"{file_name:<40} | {'N/A':>12} | {'N/A':>18} | {'N/A':>12} | {'N/A':>12} | {'N/A':>10} | FAILED - {result['error']}")

print("\n--- End of Summary ---")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 20, Finished, Available, Finished)

🔍 Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz
📌 Header detected in abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz at raw file line 1 with 76 columns: ['id', 'asset', 'attribute', 'textvalue', 'numericvalue', 'datevalue', 'decimalvalue2dp', 'selectionvalue', 'location', 'validfrom', 'validto', 'kino_csid', 'hs2_kino_cuid', 'kino_cwhen', 'kino_usid', 'kino_uuid', 'kino_uwhen', 'basedon', 'owner', 'phase', 'person', 'organisation', 'textstorefortypetransition', 'decimalstorefortypetransition', 'numericstorefortypetransition', 'startchainage', 'endchainage', 'numericendvalue', 'decimalendvalue2dp', 'isbeingsplit', 'splitatchainage', 'afternumericstartvalue', 'afternumericendvalue', 'afterdecimalstartvalue2dp', 'afterdecimalendvalue2dp', 'aftertextvalue', 'afterdatevalue', 'afterselect

In [19]:
# from notebookutils import mssparkutils
# import os
# from urllib.parse import urlparse

# def list_csv_files_recursive(base_path):
#     csv_files = []

#     def walk(path):
#         for item in mssparkutils.fs.ls(path):
#             if item.isDir:
#                 walk(item.path)
#             elif item.path.endswith(".csv") or item.path.endswith(".csv.gz"):
#                 csv_files.append(item.path)

#     walk(base_path)
#     return csv_files

# # ✅ Use this to get all CSV files
# csv_files = list_csv_files_recursive("Files/")
# print(f"📁 Found {len(csv_files)} CSV files")

# def detect_header_and_read_csv(path, lines_to_check=20):
#     # Step 1: Read file as raw lines with line numbers
#     raw_lines_rdd = spark.read.text(path).rdd.map(lambda r: r[0])
#     total_raw_rows = raw_lines_rdd.count() # Count total raw rows
    
#     # Add line numbers to raw lines for header detection
#     raw_lines_with_index = raw_lines_rdd.zipWithIndex()
#     preview = raw_lines_with_index.take(lines_to_check)

#     # Step 2: Score lines to find best candidate for header
#     def score(line_tuple):
#         line = line_tuple[0]
#         return line.count(",") + line.count('"')

#     header_line = max(preview, key=score)
#     header_index = header_line[1]
#     header_raw = header_line[0]

#     # Step 3: Parse header fields
#     header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in header_raw.split(",")]
#     num_fields = len(header_fields)

#     print(f"📌 Header detected in {path} at line {header_index}: {header_fields}")

#     # Step 4: Get only data lines AFTER the header
#     data_lines_after_header_rdd = (
#         raw_lines_with_index
#         .filter(lambda x: x[1] > header_index)
#         .map(lambda x: x[0])
#     )
#     rows_after_header = data_lines_after_header_rdd.count() # Count lines after header

#     # Filter for valid lines based on number of fields
#     valid_data_lines_rdd = data_lines_after_header_rdd.filter(lambda line: line.count(",") >= num_fields - 1)
#     valid_rows = valid_data_lines_rdd.count()

#     invalid_rows = rows_after_header - valid_rows

#     # Step 5: Save cleaned lines to OneLake temp location
#     filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
#     temp_path = f"{output_path}/{filename}"

#     try:
#         fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
#         hadoop_path = spark._jvm.org.apache.hadoop.fs.Path(temp_path)
#         if fs.exists(hadoop_path):
#             fs.delete(hadoop_path, True)
#     except Exception as cleanup_err:
#         print(f"⚠️ Could not clean temp path: {cleanup_err}")

#     valid_data_lines_rdd.saveAsTextFile(temp_path)

#     # Step 6: Read back the cleaned lines
#     df = spark.read.option("header", "false").csv(temp_path)
    
#     # Step 7: Force column count alignment
#     actual_fields = len(df.columns)
#     if actual_fields != num_fields:
#         print(f"⚠️ Mismatch: {actual_fields} fields in data vs {num_fields} in header. Trimming/padding...")

#         # Pad with dummy columns or trim extra columns
#         if actual_fields > num_fields:
#             df = df.select(df.columns[:num_fields])
#         else:
#             df = df.selectExpr("*", *["null as dummy_col_" + str(i) for i in range(num_fields - actual_fields)])

#     df = df.toDF(*header_fields)

#     return df, {
#         "raw_rows": total_raw_rows,
#         "rows_after_header": rows_after_header,
#         "valid_rows": valid_rows,
#         "invalid_rows": invalid_rows,
#         "filename": os.path.basename(path)
#     }

# # Setup
# parsed_dfs = {}
# successfully_loaded = []
# failed_to_load = []

# successfully_written = []
# failed_to_write = []

# processing_summary = [] # New list to store summary data

# TARGET_SCHEMA = "aims_dbo3"  # e.g., 'bronze', 'staging'
# TABLE_SAVE_MODE = "overwrite"       # or "append"
# # Assuming 'output_path_' is defined elsewhere in your notebook environment
# output_path = "abfss://f0b2f0a1-d70c-43f1-9c60-a8d67287955c@onelake.dfs.fabric.microsoft.com/83c8ed30-e379-45f8-8098-b80c102a9009/Files/temp_output"  # Placeholder, replace with actual output_path_

# # STEP 1: Load all CSVs
# for path in csv_files:
#     file_name = os.path.basename(path)
#     try:
#         print(f"\n🔍 Inspecting: {path}")
#         df, stats = detect_header_and_read_csv(path)
#         parsed_dfs[path] = df
#         successfully_loaded.append(path)
#         df.printSchema()
#         df.show(2)
        
#         # Add status to stats for summary
#         stats["status"] = "SUCCESS"
#         processing_summary.append(stats)

#     except Exception as e:
#         print(f"❌ Error loading {path}: {e}")
#         failed_to_load.append((path, str(e)))
#         # Add failed status to stats for summary, with default counts
#         processing_summary.append({
#             "raw_rows": 0,
#             "rows_after_header": 0,
#             "valid_rows": 0,
#             "invalid_rows": 0,
#             "filename": file_name,
#             "status": "FAILED"
#         })


# # STEP 2: Write to tables
# for path, df in parsed_dfs.items():
#     table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
#     full_table_name = f"{TARGET_SCHEMA}.{table_name}"

#     try:
#         print(f"\n💾 Writing to table: {full_table_name}")
#         df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
#         print(f"✅ Successfully written: {full_table_name}")
#         successfully_written.append(full_table_name)
#     except Exception as e:
#         print(f"❌ Failed to write {full_table_name}: {e}")
#         failed_to_write.append((full_table_name, str(e)))

# # STEP 3: Summary Report
# print("\n📊 LOAD SUMMARY")
# print(f"✅ Loaded: {len(successfully_loaded)} files")
# print(f"❌ Failed to load: {len(failed_to_load)} files")
# if failed_to_load:
#     for path, err in failed_to_load:
#         print(f"   - {path} → {err}")

# print("\n📊 WRITE SUMMARY")
# print(f"✅ Written to tables: {len(successfully_written)}")
# print(f"❌ Failed to write: {len(failed_to_write)}")
# if failed_to_write:
#     for tbl, err in failed_to_write:
#         print(f"   - {tbl} → {err}")

# # STEP 4: Processing Summary Table
# print("\n--- Processing Summary ---")
# header = [
#     "File Name",
#     "Raw Rows",
#     "Rows After Header",
#     "Valid Rows",
#     "Invalid Rows",
#     "Delta (%)",
#     "Status"
# ]
# # Determine maximum column widths for formatting
# col_widths = {
#     "File Name": max(len(s["filename"]) for s in processing_summary) if processing_summary else len("File Name"),
#     "Raw Rows": max(len(str(s["raw_rows"])) for s in processing_summary) if processing_summary else len("Raw Rows"),
#     "Rows After Header": max(len(str(s["rows_after_header"])) for s in processing_summary) if processing_summary else len("Rows After Header"),
#     "Valid Rows": max(len(str(s["valid_rows"])) for s in processing_summary) if processing_summary else len("Valid Rows"),
#     "Invalid Rows": max(len(str(s["invalid_rows"])) for s in processing_summary) if processing_summary else len("Invalid Rows"),
#     "Delta (%)": len("Delta (%)"), # Fixed width for percentage
#     "Status": max(len(s["status"]) for s in processing_summary) if processing_summary else len("Status")
# }

# # Adjust header widths if they are longer than any data
# for h in header:
#     col_widths[h] = max(col_widths[h], len(h))

# # Print header
# header_str = " | ".join([f"{h:<{col_widths[h]}}" for h in header])
# print(header_str)
# print("-" * len(header_str))

# # Print data rows
# for stats in processing_summary:
#     delta_percent = 0.0
#     if stats["rows_after_header"] > 0:
#         delta_percent = ((stats["rows_after_header"] - stats["valid_rows"]) / stats["rows_after_header"]) * 100
    
#     row_data = [
#         f"{stats['filename']:<{col_widths['File Name']}}",
#         f"{stats['raw_rows']:>{col_widths['Raw Rows']}}",
#         f"{stats['rows_after_header']:>{col_widths['Rows After Header']}}",
#         f"{stats['valid_rows']:>{col_widths['Valid Rows']}}",
#         f"{stats['invalid_rows']:>{col_widths['Invalid Rows']}}",
#         f"{delta_percent:>{col_widths['Delta (%)']}-.2f}%",
#         f"{stats['status']:<{col_widths['Status']}}"
#     ]
#     print(" | ".join(row_data))

# print("\n--- End of Summary ---")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 21, Finished, Available, Finished)

final attempt is to provide visibility.

In [20]:
from notebookutils import mssparkutils
import os
from urllib.parse import urlparse

def list_csv_files_recursive(base_path):
    csv_files = []

    def walk(path):
        for item in mssparkutils.fs.ls(path):
            if item.isDir:
                walk(item.path)
            elif item.path.endswith(".csv") or item.path.endswith(".csv.gz"):
                csv_files.append(item.path)

    walk(base_path)
    return csv_files

# ✅ Use this to get all CSV files
csv_files = list_csv_files_recursive("Files/")
print(f"📁 Found {len(csv_files)} CSV files")

def detect_header_and_read_csv(path, lines_to_check=20):
    # Step 1: Read file as raw lines with line numbers
    raw_lines_rdd = spark.read.text(path).rdd.map(lambda r: r[0])
    total_raw_rows = raw_lines_rdd.count() # Count total raw rows
    
    # Add line numbers to raw lines for header detection
    raw_lines_with_index = raw_lines_rdd.zipWithIndex()
    preview = raw_lines_with_index.take(lines_to_check)

    # Step 2: Score lines to find best candidate for header
    def score(line_tuple):
        line = line_tuple[0]
        return line.count(",") + line.count('"')

    header_line = max(preview, key=score)
    header_index = header_line[1]
    header_raw = header_line[0]

    # Step 3: Parse header fields
    header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in header_raw.split(",")]
    num_fields = len(header_fields)

    print(f"📌 Header detected in {path} at line {header_index}: {header_fields}")

    # Step 4: Get only data lines AFTER the header
    data_lines_after_header_rdd = (
        raw_lines_with_index
        .filter(lambda x: x[1] > header_index)
        .map(lambda x: x[0])
    )
    rows_after_header = data_lines_after_header_rdd.count() # Count lines after header

    # Filter for valid lines based on number of fields
    valid_data_lines_rdd = data_lines_after_header_rdd.filter(lambda line: line.count(",") >= num_fields - 1)
    valid_rows = valid_data_lines_rdd.count()

    invalid_rows = rows_after_header - valid_rows

    # Step 5: Save cleaned lines to OneLake temp location
    filename = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    temp_path = f"{output_path}/{filename}"

    try:
        fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
        hadoop_path = spark._jvm.org.apache.hadoop.fs.Path(temp_path)
        if fs.exists(hadoop_path):
            fs.delete(hadoop_path, True)
    except Exception as cleanup_err:
        print(f"⚠️ Could not clean temp path: {cleanup_err}")

    valid_data_lines_rdd.saveAsTextFile(temp_path)

    # Step 6: Read back the cleaned lines
    df = spark.read.option("header", "false").csv(temp_path)
    
    # Step 7: Force column count alignment
    actual_fields = len(df.columns)
    if actual_fields != num_fields:
        print(f"⚠️ Mismatch: {actual_fields} fields in data vs {num_fields} in header. Trimming/padding...")

        # Pad with dummy columns or trim extra columns
        if actual_fields > num_fields:
            df = df.select(df.columns[:num_fields])
        else:
            df = df.selectExpr("*", *["null as dummy_col_" + str(i) for i in range(num_fields - actual_fields)])

    df = df.toDF(*header_fields)

    return df, {
        "raw_rows": total_raw_rows,
        "rows_after_header": rows_after_header,
        "valid_rows": valid_rows,
        "invalid_rows": invalid_rows,
        "filename": os.path.basename(path)
    }

# Setup
parsed_dfs = {}
successfully_loaded = []
failed_to_load = []

successfully_written = []
failed_to_write = []

processing_summary = [] # New list to store summary data

TARGET_SCHEMA = "aims_dbo3"  # e.g., 'bronze', 'staging'
TABLE_SAVE_MODE = "overwrite"       # or "append"
# Assuming 'output_path_' is defined elsewhere in your notebook environment
output_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/v2_processed_raw_source_data_aims2025/"  # Placeholder, replace with actual output_path_

# STEP 1: Load all CSVs
for path in csv_files:
    file_name = os.path.basename(path)
    try:
        print(f"\n🔍 Inspecting: {path}")
        df, stats = detect_header_and_read_csv(path)
        parsed_dfs[path] = df
        successfully_loaded.append(path)
        df.printSchema()
        df.show(2)
        
        # Add status to stats for summary
        stats["status"] = "SUCCESS"
        processing_summary.append(stats)

    except Exception as e:
        print(f"❌ Error loading {path}: {e}")
        failed_to_load.append((path, str(e)))
        # Add failed status to stats for summary, with default counts
        processing_summary.append({
            "raw_rows": 0,
            "rows_after_header": 0,
            "valid_rows": 0,
            "invalid_rows": 0,
            "filename": file_name,
            "status": "FAILED"
        })


# STEP 2: Write to tables
for path, df in parsed_dfs.items():
    table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    full_table_name = f"{TARGET_SCHEMA}.{table_name}"

    try:
        print(f"\n💾 Writing to table: {full_table_name}")
        df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
        print(f"✅ Successfully written: {full_table_name}")
        successfully_written.append(full_table_name)
    except Exception as e:
        print(f"❌ Failed to write {full_table_name}: {e}")
        failed_to_write.append((full_table_name, str(e)))

# STEP 3: Summary Report
print("\n📊 LOAD SUMMARY")
print(f"✅ Loaded: {len(successfully_loaded)} files")
print(f"❌ Failed to load: {len(failed_to_load)} files")
if failed_to_load:
    for path, err in failed_to_load:
        print(f"   - {path} → {err}")

print("\n📊 WRITE SUMMARY")
print(f"✅ Written to tables: {len(successfully_written)}")
print(f"❌ Failed to write: {len(failed_to_write)}")
if failed_to_write:
    for tbl, err in failed_to_write:
        print(f"   - {tbl} → {err}")

# STEP 4: Processing Summary Table
print("\n--- Processing Summary ---")
header = [
    "File Name",
    "Raw Rows",
    "Rows After Header",
    "Valid Rows",
    "Invalid Rows",
    "Delta (%)",
    "Status"
]
# Determine maximum column widths for formatting
col_widths = {
    "File Name": max(len(s["filename"]) for s in processing_summary) if processing_summary else len("File Name"),
    "Raw Rows": max(len(str(s["raw_rows"])) for s in processing_summary) if processing_summary else len("Raw Rows"),
    "Rows After Header": max(len(str(s["rows_after_header"])) for s in processing_summary) if processing_summary else len("Rows After Header"),
    "Valid Rows": max(len(str(s["valid_rows"])) for s in processing_summary) if processing_summary else len("Valid Rows"),
    "Invalid Rows": max(len(str(s["invalid_rows"])) for s in processing_summary) if processing_summary else len("Invalid Rows"),
    "Delta (%)": len("Delta (%)"), # Fixed width for percentage
    "Status": max(len(s["status"]) for s in processing_summary) if processing_summary else len("Status")
}

# Adjust header widths if they are longer than any data
for h in header:
    col_widths[h] = max(col_widths[h], len(h))

# Print header
header_str = " | ".join([f"{h:<{col_widths[h]}}" for h in header])
print(header_str)
print("-" * len(header_str))

# Print data rows
for stats in processing_summary:
    delta_percent = 0.0
    if stats["rows_after_header"] > 0:
        delta_percent = ((stats["rows_after_header"] - stats["valid_rows"]) / stats["rows_after_header"]) * 100
    
    row_data = [
        f"{stats['filename']:<{col_widths['File Name']}}",
        f"{stats['raw_rows']:>{col_widths['Raw Rows']}}",
        f"{stats['rows_after_header']:>{col_widths['Rows After Header']}}",
        f"{stats['valid_rows']:>{col_widths['Valid Rows']}}",
        f"{stats['invalid_rows']:>{col_widths['Invalid Rows']}}",
        f"{delta_percent:>{col_widths['Delta (%)']}.2f}%", # CORRECTED LINE
        f"{stats['status']:<{col_widths['Status']}}"
    ]
    print(" | ".join(row_data))

print("\n--- End of Summary ---")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 22, Finished, Available, Finished)

📁 Found 28 CSV files

🔍 Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz
📌 Header detected in abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz at line 1: ['ID', 'ASSET', 'ATTRIBUTE', 'TEXTVALUE', 'NUMERICVALUE', 'DATEVALUE', 'DECIMALVALUE2DP', 'SELECTIONVALUE', 'LOCATION', 'VALIDFROM', 'VALIDTO', 'KINO_CSID', 'HS2_KINO_CUID', 'KINO_CWHEN', 'KINO_USID', 'KINO_UUID', 'KINO_UWHEN', 'BASEDON', 'OWNER', 'PHASE', 'PERSON', 'ORGANISATION', 'TEXTSTOREFORTYPETRANSITION', 'DECIMALSTOREFORTYPETRANSITION', 'NUMERICSTOREFORTYPETRANSITION', 'STARTCHAINAGE', 'ENDCHAINAGE', 'NUMERICENDVALUE', 'DECIMALENDVALUE2DP', 'ISBEINGSPLIT', 'SPLITATCHAINAGE', 'AFTERNUMERICSTARTVALUE', 'AFTERNUMERICENDVALUE', 'AFTERDECIMALSTARTVALUE2DP', 'AFTERDECIMALENDVALUE2DP', 'AFTERTEXTVALUE', 'AFTERDATEVALUE', 'AFTERSELECTION

##### Enforced Schema Approach as recommended by COPILOT.
problem remains, malformed rows will not be loaded.

In [21]:
from notebookutils import mssparkutils
import os
from urllib.parse import urlparse
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType

# Define schemas for specific files
# All fields are initially StringType as per your provided root schema output.
# You can refine these types (e.g., to IntegerType, DoubleType, DateType)
# if you know the data quality is consistent for those columns.
# A '_corrupt_record' column is added to each schema to capture malformed rows.

FILE_SCHEMAS = {
    "AssetAttributes.csv.gz": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSET", StringType(), True),
        StructField("ATTRIBUTE", StringType(), True),
        StructField("TEXTVALUE", StringType(), True),
        StructField("NUMERICVALUE", StringType(), True),
        StructField("DATEVALUE", StringType(), True),
        StructField("DECIMALVALUE2DP", StringType(), True),
        StructField("SELECTIONVALUE", StringType(), True),
        StructField("LOCATION", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("HS2_KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("BASEDON", StringType(), True),
        StructField("OWNER", StringType(), True),
        StructField("PHASE", StringType(), True),
        StructField("PERSON", StringType(), True),
        StructField("ORGANISATION", StringType(), True),
        StructField("TEXTSTOREFORTYPETRANSITION", StringType(), True),
        StructField("DECIMALSTOREFORTYPETRANSITION", StringType(), True),
        StructField("NUMERICSTOREFORTYPETRANSITION", StringType(), True),
        StructField("STARTCHAINAGE", StringType(), True),
        StructField("ENDCHAINAGE", StringType(), True),
        StructField("NUMERICENDVALUE", StringType(), True),
        StructField("DECIMALENDVALUE2DP", StringType(), True),
        StructField("ISBEINGSPLIT", StringType(), True),
        StructField("SPLITATCHAINAGE", StringType(), True),
        StructField("AFTERNUMERICSTARTVALUE", StringType(), True),
        StructField("AFTERNUMERICENDVALUE", StringType(), True),
        StructField("AFTERDECIMALSTARTVALUE2DP", StringType(), True),
        StructField("AFTERDECIMALENDVALUE2DP", StringType(), True),
        StructField("AFTERTEXTVALUE", StringType(), True),
        StructField("AFTERDATEVALUE", StringType(), True),
        StructField("AFTERSELECTIONVALUE", StringType(), True),
        StructField("ROUTE", StringType(), True),
        StructField("DECIMALVALUE3DP", StringType(), True),
        StructField("DECIMALENDVALUE3DP", StringType(), True),
        StructField("AFTERDECIMALSTARTVALUE3DP", StringType(), True),
        StructField("AFTERDECIMALENDVALUE3DP", StringType(), True),
        StructField("COPIEDFROM", StringType(), True),
        StructField("ASSETCLASS", StringType(), True),
        StructField("DECIMALENDSTOREFORTYPETRANS", StringType(), True),
        StructField("NUMERICENDSTOREFORTYPETRANS", StringType(), True),
        StructField("DECIMALVALUE4DP", StringType(), True),
        StructField("CONSTRAINTOGROUP", StringType(), True),
        StructField("ADDRESSLINE1", StringType(), True),
        StructField("ADDRESSLINE2", StringType(), True),
        StructField("ADDRESSLINE3", StringType(), True),
        StructField("TOWNCITY", StringType(), True),
        StructField("POSTCODE", StringType(), True),
        StructField("STAGE", StringType(), True),
        StructField("STATUS", StringType(), True),
        StructField("CHAINAGEBASELINE", StringType(), True),
        StructField("BOOLEANVALUE", StringType(), True),
        StructField("TIMEVALUE", StringType(), True),
        StructField("ACCEPTEDBY", StringType(), True),
        StructField("NEWCONTRACTSTAGECODE", StringType(), True),
        StructField("MEMOVALUE", StringType(), True),
        StructField("MEMOSTOREFORTYPETRANSITION", StringType(), True),
        StructField("MATRIXUPDATEWIZARD", StringType(), True),
        StructField("REQUIRESATTRIBUTEDERIVATION", StringType(), True),
        StructField("TEXTDOMAINVALUE", StringType(), True),
        StructField("AFTERTEXTDOMAINVALUE", StringType(), True),
        StructField("VALUESTATUS", StringType(), True),
        StructField("DEFAULTEDFROM", StringType(), True),
        StructField("ACCEPTEDOUTSIDERANGE", StringType(), True),
        StructField("INFORMATIONPACKAGE", StringType(), True),
        StructField("PUBLISHEDATSTAGE", StringType(), True),
        StructField("ISLATESTPUBLISHED", StringType(), True),
        StructField("WORKORDER", StringType(), True),
        StructField("TRACK", StringType(), True),
        StructField("TYPECONVERSIONFAILED", StringType(), True),
        StructField("_corrupt_record", StringType(), True) # Column to capture malformed rows
    ]),
    "AssetClassAttributes.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSETCLASS", StringType(), True),
        StructField("ATTRIBUTE", StringType(), True),
        StructField("REQUIREDATSTAGEOLD", StringType(), True),
        StructField("AVAILABLEFROMSTAGEOLD", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("EXPECTED", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("AVAILABLEFROMSTAGE", StringType(), True),
        StructField("REQUIREDATSTAGE", StringType(), True),
        StructField("CREATEDBYUPDATE", StringType(), True),
        StructField("OWNER", StringType(), True),
        StructField("MODELLINGPROPERTY", StringType(), True),
        StructField("STATUS", StringType(), True),
        StructField("LOCATIONSSETDENORM", StringType(), True),
        StructField("LOCATIONSUNSETDENORM", StringType(), True),
        StructField("ATTRIBUTEDOMAIN", StringType(), True),
        StructField("PHASEAPPLICABILITY", StringType(), True),
        StructField("PHASEAPPLICABILITYDENORM", StringType(), True),
        StructField("SETFROMEXEMPLAR", StringType(), True),
        StructField("IGNOREINTRANSITIONCHECK", StringType(), True),
        StructField("INHERIT", StringType(), True),
        StructField("INHERITEDFROM", StringType(), True),
        StructField("MOVEDTOLIVEBY", StringType(), True),
        StructField("MOVEDTOLIVEDATE", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClassChangeLogs.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSETCLASS", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("CHAINAGE", StringType(), True),
        StructField("POSTALADDRESS", StringType(), True),
        StructField("GRIDREFERENCE", StringType(), True),
        StructField("CHANGEDATE", StringType(), True),
        StructField("CHANGES", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("PHASE", StringType(), True),
        StructField("IDENTIFICATIONPLATEINFO", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClassRelationships.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("SOURCEASSETCLASS", StringType(), True),
        StructField("TARGETASSETCLASS", StringType(), True),
        StructField("RELATIONSHIPTYPE", StringType(), True),
        StructField("REQUIREDATSTAGEOLD", StringType(), True),
        StructField("EXPECTED", StringType(), True),
        StructField("ADDITIONALDESCRIPTION", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("REQUIREDATSTAGE", StringType(), True),
        StructField("CREATEDBYUPDATE", StringType(), True),
        StructField("OWNER", StringType(), True),
        StructField("UUID", StringType(), True),
        StructField("STATUS", StringType(), True),
        StructField("MOVEDTOLIVEBY", StringType(), True),
        StructField("MOVETOLIVEDATE", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClasses.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("PARENT", StringType(), True),
        StructField("LEVELNUMBER", StringType(), True),
        StructField("GROUPINGCLASSFORHIERARCHYREPOR", StringType(), True),
        StructField("OLDHIERARCHYENTITY", StringType(), True),
        StructField("DISPLAYCOLOUR", StringType(), True),
        StructField("CONTAINER", StringType(), True),
        StructField("BACKGROUNDCOLOUR", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("PARENTINHERITEDKEY", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("DOCUMENTNUMBER", StringType(), True),
        StructField("CURRENTVERSION", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("CHAINAGEAPPLICABILITY", StringType(), True),
        StructField("POSTALADDRESSAPPLICABILITY", StringType(), True),
        StructField("GRIDCOORDINATES", StringType(), True),
        StructField("ASSETLEVEL", StringType(), True),
        StructField("INCLUDEIDENTIFCATIONPLATES", StringType(), True),
        StructField("IDENTIFICATIONPLATEDESCRIPTION", StringType(), True),
        StructField("DEFAULTVISUALISATION", StringType(), True),
        StructField("INTANGIBLEGROUPINGCODECHARACTE", StringType(), True),
        StructField("INTANGIBLENLR", StringType(), True),
        StructField("SHOWGISVIEWERPANEL", StringType(), True),
        StructField("OWNER", StringType(), True),
        StructField("ASSETFORMTITLE", StringType(), True),
        StructField("UUID", StringType(), True),
        StructField("ALLOWATTRIBUTEGRAPHING", StringType(), True),
        StructField("NEARESTAUTHORITATIVECLASS", StringType(), True),
        StructField("SCOPEBOOKDEFINITION", StringType(), True),
        StructField("SENSITIVITYLEVEL", StringType(), True),
        StructField("RELSHIPSECTIONHEADEROVERRIDE", StringType(), True),
        StructField("PRIMARYEXEMPLAR", StringType(), True),
        StructField("PHASEAPPLICABILITY", StringType(), True),
        StructField("LATESTLOG", StringType(), True),
        StructField("HIDEINHIERARCHYREPORT", StringType(), True),
        StructField("DISALLOWSELECTIONFORASSETS", StringType(), True),
        StructField("INSTRUCTEDDATE", StringType(), True),
        StructField("STATUS", StringType(), True),
        StructField("HASDRAFTITEM", StringType(), True),
        StructField("CANHAVEASSETPLANS", StringType(), True),
        StructField("MOVEDTOLIVEBY", StringType(), True),
        StructField("MOVEDTOLIVEDATE", StringType(), True),
        StructField("UAIDOVERRIDECHARACTER", StringType(), True),
        StructField("ALLOWASSIGNMENTOFPRODUCT", StringType(), True),
        StructField("DISPLAYALERTONASSETFORM", StringType(), True),
        StructField("ALERTTEXT", StringType(), True),
        StructField("REMOVALREASON", StringType(), True),
        StructField("INTANGIBLE", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ])
    # Add other schemas here as needed for other CSV files
}


def list_csv_files_recursive(base_path):
    csv_files = []

    def walk(path):
        for item in mssparkutils.fs.ls(path):
            if item.isDir:
                walk(item.path)
            elif item.path.endswith(".csv") or item.path.endswith(".csv.gz"):
                csv_files.append(item.path)

    walk(base_path)
    return csv_files

# ✅ Use this to get all CSV files
csv_files = list_csv_files_recursive("Files/")
print(f"📁 Found {len(csv_files)} CSV files")

def detect_header_and_read_csv(path, predefined_schema=None):
    file_name = os.path.basename(path)

    # Step 1: Count total raw lines in the file
    # This is done before reading with schema to get the absolute raw count
    total_raw_rows = spark.read.text(path).count()

    # Step 2: Read file with schema (if provided) or infer schema if not
    if predefined_schema:
        print(f"📌 Reading {file_name} with predefined schema.")
        # Use header=True because the schema assumes the first row is the header
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("mode", "PERMISSIVE") \
            .option("delimiter", ",") \
            .schema(predefined_schema) \
            .load(path)
        
        # Cache the DataFrame to allow querying _corrupt_record
        df.cache()
        
        # Determine header index (assuming it's the first row if header=True)
        # For accurate 'rows_after_header', we assume 1 header row if header=True
        header_index = 0 
        rows_after_header = total_raw_rows - 1 if total_raw_rows > 0 else 0

    else:
        print(f"📌 No predefined schema for {file_name}. Inferring header and schema.")
        # Fallback to original header detection if no predefined schema
        raw_lines_with_index = spark.read.text(path).rdd.map(lambda r: r[0]).zipWithIndex()
        preview = raw_lines_with_index.take(20) # Check first 20 lines for header

        def score(line_tuple):
            line = line_tuple[0]
            return line.count(",") + line.count('"')

        header_line = max(preview, key=score)
        header_index = header_line[1]
        header_raw = header_line[0]
        header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in header_raw.split(",")]
        num_fields = len(header_fields)

        print(f"📌 Header detected in {file_name} at line {header_index}: {header_fields}")

        # Read the data, skipping the header lines found
        # We need to re-read the file without the header
        df = spark.read.format("csv") \
            .option("header", "false") \
            .option("mode", "PERMISSIVE") \
            .option("delimiter", ",") \
            .load(path)
        
        # Cache the DataFrame to allow querying _corrupt_record
        df.cache()

        # Filter out header lines if detected at a non-zero index
        if header_index > 0:
            # Re-read raw lines to filter out header rows
            raw_data_rdd = spark.read.text(path).rdd.map(lambda r: r[0]).zipWithIndex()
            data_lines_after_header_rdd = raw_data_rdd.filter(lambda x: x[1] > header_index).map(lambda x: x[0])
            
            # Save to a temporary location to read as CSV
            temp_path_for_filtered_data = f"{output_path}/temp_filtered_{os.path.splitext(file_name)[0].replace('.', '_').lower()}"
            try:
                fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
                hadoop_path = spark._jvm.org.apache.hadoop.fs.Path(temp_path_for_filtered_data)
                if fs.exists(hadoop_path):
                    fs.delete(hadoop_path, True)
            except Exception as cleanup_err:
                print(f"⚠️ Could not clean temp path for filtered data: {cleanup_err}")
            
            data_lines_after_header_rdd.saveAsTextFile(temp_path_for_filtered_data)
            
            df = spark.read.option("header", "false").csv(temp_path_for_filtered_data)
            
            # Cache the DataFrame to allow querying _corrupt_record
            df.cache()

            # Re-apply header fields if they were dynamically detected
            actual_fields = len(df.columns)
            if actual_fields != num_fields:
                print(f"⚠️ Mismatch: {actual_fields} fields in data vs {num_fields} in header. Trimming/padding...")
                if actual_fields > num_fields:
                    df = df.select(df.columns[:num_fields])
                else:
                    df = df.selectExpr("*", *["null as dummy_col_" + str(i) for i in range(num_fields - actual_fields)])
            df = df.toDF(*header_fields)
            
            rows_after_header = total_raw_rows - (header_index + 1)
        else:
            rows_after_header = total_raw_rows # If header is at line 0 or no header detected

    # Calculate valid and invalid rows using the _corrupt_record column
    # If the schema was not predefined and _corrupt_record wasn't added, this will be null
    valid_rows = df.filter(df["_corrupt_record"].isNull()).count()
    invalid_rows = df.filter(df["_corrupt_record"].isNotNull()).count()

    # Unpersist the DataFrame to release cached data
    df.unpersist()

    # Drop the _corrupt_record column before returning the DataFrame
    # This ensures the final DataFrame schema matches the original desired output
    if "_corrupt_record" in df.columns:
        df = df.drop("_corrupt_record")

    return df, {
        "raw_rows": total_raw_rows,
        "rows_after_header": rows_after_header,
        "valid_rows": valid_rows,
        "invalid_rows": invalid_rows,
        "filename": file_name
    }

# Setup
parsed_dfs = {}
successfully_loaded = []
failed_to_load = []

successfully_written = []
failed_to_write = []

processing_summary = [] # New list to store summary data

TARGET_SCHEMA = "aims_dbo3"  # e.g., 'bronze', 'staging'
TABLE_SAVE_MODE = "overwrite"       # or "append"
# Assuming 'output_path_' is defined elsewhere in your notebook environment
# Placeholder, replace with actual output_path_
output_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/v2_processed_raw_source_data_aims2025/"

# STEP 1: Load all CSVs
for path in csv_files:
    file_name = os.path.basename(path)
    
    # Get the predefined schema for the current file, if it exists
    schema_to_use = FILE_SCHEMAS.get(file_name)

    try:
        print(f"\n🔍 Inspecting: {path}")
        df, stats = detect_header_and_read_csv(path, predefined_schema=schema_to_use)
        
        parsed_dfs[path] = df
        successfully_loaded.append(path)
        
        print(f"📊 Rows in CSV after header (before loading into DataFrame): {stats['rows_after_header']}")
        print(f"✅ Valid rows loaded into DataFrame: {stats['valid_rows']}")
        print(f"❌ Invalid rows detected: {stats['invalid_rows']}")
        
        # Check for discrepancy based on valid rows vs. rows after header
        if stats['rows_after_header'] != stats['valid_rows']:
            print(f"⚠️ Discrepancy: Valid row count ({stats['valid_rows']}) differs from rows after header ({stats['rows_after_header']}).")
            print("This indicates malformed rows were dropped or set to null during parsing.")
        else:
            print("✅ No significant discrepancy detected between rows after header and valid rows.")

        df.printSchema()
        df.show(2)
        
        # Add status to stats for summary
        stats["status"] = "SUCCESS"
        processing_summary.append(stats)

    except Exception as e:
        print(f"❌ Error loading {path}: {e}")
        failed_to_load.append((path, str(e)))
        # Add failed status to stats for summary, with default counts
        processing_summary.append({
            "raw_rows": 0,
            "rows_after_header": 0,
            "valid_rows": 0,
            "invalid_rows": 0,
            "filename": file_name,
            "status": "FAILED"
        })


# STEP 2: Write to tables
for path, df in parsed_dfs.items():
    table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    full_table_name = f"{TARGET_SCHEMA}.{table_name}"

    try:
        print(f"\n💾 Writing to table: {full_table_name}")
        df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
        print(f"✅ Successfully written: {full_table_name}")
        successfully_written.append(full_table_name)
    except Exception as e:
        print(f"❌ Failed to write {full_table_name}: {e}")
        failed_to_write.append((full_table_name, str(e)))

# STEP 3: Summary Report
print("\n📊 LOAD SUMMARY")
print(f"✅ Loaded: {len(successfully_loaded)} files")
print(f"❌ Failed to load: {len(failed_to_load)} files")
if failed_to_load:
    for path, err in failed_to_load:
        print(f"   - {path} → {err}")

print("\n📊 WRITE SUMMARY")
print(f"✅ Written to tables: {len(successfully_written)}")
print(f"❌ Failed to write: {len(failed_to_write)} files")
if failed_to_write:
    for tbl, err in failed_to_write:
        print(f"   - {tbl} → {err}")

# STEP 4: Processing Summary Table
print("\n--- Processing Summary ---")
header = [
    "File Name",
    "Raw Rows",
    "Rows After Header",
    "Valid Rows",
    "Invalid Rows",
    "Delta (%)",
    "Status"
]
# Determine maximum column widths for formatting
col_widths = {
    "File Name": max(len(s["filename"]) for s in processing_summary) if processing_summary else len("File Name"),
    "Raw Rows": max(len(str(s["raw_rows"])) for s in processing_summary) if processing_summary else len("Raw Rows"),
    "Rows After Header": max(len(str(s["rows_after_header"])) for s in processing_summary) if processing_summary else len("Rows After Header"),
    "Valid Rows": max(len(str(s["valid_rows"])) for s in processing_summary) if processing_summary else len("Valid Rows"),
    "Invalid Rows": max(len(str(s["invalid_rows"])) for s in processing_summary) if processing_summary else len("Invalid Rows"),
    "Delta (%)": len("Delta (%)"), # Fixed width for percentage, or dynamically calculate based on max float string length
    "Status": max(len(s["status"]) for s in processing_summary) if processing_summary else len("Status")
}

# Adjust header widths if they are longer than any data
for h in header:
    col_widths[h] = max(col_widths[h], len(h))

# Print header
header_str = " | ".join([f"{h:<{col_widths[h]}}" for h in header])
print(header_str)
print("-" * len(header_str))

# Print data rows
for stats in processing_summary:
    delta_percent = 0.0
    if stats["rows_after_header"] > 0:
        delta_percent = ((stats["rows_after_header"] - stats["valid_rows"]) / stats["rows_after_header"]) * 100
    
    row_data = [
        f"{stats['filename']:<{col_widths['File Name']}}",
        f"{stats['raw_rows']:>{col_widths['Raw Rows']}}",
        f"{stats['rows_after_header']:>{col_widths['Rows After Header']}}",
        f"{stats['valid_rows']:>{col_widths['Valid Rows']}}",
        f"{stats['invalid_rows']:>{col_widths['Invalid Rows']}}",
        f"{delta_percent:>{col_widths['Delta (%)']}.2f}%",
        f"{stats['status']:<{col_widths['Status']}}"
    ]
    print(" | ".join(row_data))

print("\n--- End of Summary ---")


StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 23, Finished, Available, Finished)

📁 Found 28 CSV files

🔍 Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz
📌 Reading AssetAttributes.csv.gz with predefined schema.
📊 Rows in CSV after header (before loading into DataFrame): 58234447
✅ Valid rows loaded into DataFrame: 58092173
❌ Invalid rows detected: 136785
⚠️ Discrepancy: Valid row count (58092173) differs from rows after header (58234447).
This indicates malformed rows were dropped or set to null during parsing.
root
 |-- ID: string (nullable = true)
 |-- ASSET: string (nullable = true)
 |-- ATTRIBUTE: string (nullable = true)
 |-- TEXTVALUE: string (nullable = true)
 |-- NUMERICVALUE: string (nullable = true)
 |-- DATEVALUE: string (nullable = true)
 |-- DECIMALVALUE2DP: string (nullable = true)
 |-- SELECTIONVALUE: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- VALIDFROM: string (nullable = true)
 |-- VALIDTO: string (nullable = true)
 

In [22]:
# Define the path to your CSV files
csv_files_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/*.csv"

# Read the CSV files
df = spark.read.text(csv_files_path)

# Check for the presence of pipes
pipe_presence_df = df.filter(df.value.contains("|"))

# Show the results
pipe_presence_df.show()

# Count the number of rows with pipes
pipe_count = pipe_presence_df.count()

# Print the result
print(f"Number of rows with pipes: {pipe_count}")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 24, Finished, Available, Finished)

+-----+
|value|
+-----+
+-----+

Number of rows with pipes: 0


In [23]:
from pyspark.sql.functions import input_file_name, col
import os

# Define the path to your CSV files
csv_files_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/*.csv.gz"

# Read all files with filename included
df = spark.read.text(csv_files_path).withColumn("file_name", input_file_name())

# Count the number of files
file_names = df.select("file_name").distinct()
file_count = file_names.count()
print(f"Number of files inspected: {file_count}")

# Total number of rows per file
rows_per_file = df.groupBy("file_name").count().withColumnRenamed("count", "total_rows")

# Rows with pipes
pipe_rows_df = df.filter(col("value").contains("|"))
pipe_rows_per_file = pipe_rows_df.groupBy("file_name").count().withColumnRenamed("count", "pipe_rows")

# Join the two to show both counts per file
result_df = rows_per_file.join(pipe_rows_per_file, on="file_name", how="left").fillna(0)

# Show the results
result_df.show(truncate=False)

# Total pipe-containing rows
total_pipe_rows = pipe_rows_df.count()
print(f"Total number of rows with pipes: {total_pipe_rows}")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 25, Finished, Available, Finished)

Number of files inspected: 5
+-------------------------------------------------------------------------------------------------------------------------------------------------+----------+---------+
|file_name                                                                                                                                        |total_rows|pipe_rows|
+-------------------------------------------------------------------------------------------------------------------------------------------------+----------+---------+
|abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz  |58234448  |2376     |
|abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetLocations.csv.gz   |2187069   |0        |
|abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/Asse

# Exporting malformed rows version

In [24]:
from notebookutils import mssparkutils
import os
from urllib.parse import urlparse
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType

# Define schemas for specific files
FILE_SCHEMAS = {
    # ... (rest of the code remains the same)
}

def list_csv_files_recursive(base_path):
    csv_files = []

    def walk(path):
        for item in mssparkutils.fs.ls(path):
            if item.isDir:
                walk(item.path)
            elif item.path.endswith(".csv") or item.path.endswith(".csv.gz"):
                csv_files.append(item.path)

    walk(base_path)
    return csv_files

# Get all CSV files
csv_files = list_csv_files_recursive("Files/")
print(f"Found {len(csv_files)} CSV files")

def detect_header_and_read_csv(path, predefined_schema=None):
    # ... (rest of the code remains the same)
    return df, df_with_corrupt_record, {
        "raw_rows": total_raw_rows,
        "rows_after_header": rows_after_header,
        "valid_rows": valid_rows,
        "invalid_rows": invalid_rows,
        "filename": file_name
    }

# Setup
parsed_dfs = {}
corrupt_dfs = {}
successfully_loaded = []
failed_to_load = []

successfully_written = []
failed_to_write = []

processing_summary = [] # New list to store summary data

TARGET_SCHEMA = "aims_dbo3"  # e.g., 'bronze', 'staging'
TABLE_SAVE_MODE = "overwrite"       # or "append"
output_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/v2_processed_raw_source_data_aims2025/"

# STEP 1: Load all CSVs
for path in csv_files:
    file_name = os.path.basename(path)
    
    # Get the predefined schema for the current file, if it exists
    schema_to_use = FILE_SCHEMAS.get(file_name)

    try:
        print(f"\nInspecting: {path}")
        df, df_with_corrupt_record, stats = detect_header_and_read_csv(path, predefined_schema=schema_to_use)
        
        parsed_dfs[path] = df
        corrupt_dfs[path] = df_with_corrupt_record
        successfully_loaded.append(path)
        
        print(f"Rows in CSV after header (before loading into DataFrame): {stats['rows_after_header']}")
        print(f"Valid rows loaded into DataFrame: {stats['valid_rows']}")
        print(f"Invalid rows detected: {stats['invalid_rows']}")
        
        # Check for discrepancy based on valid rows vs. rows after header
        if stats['rows_after_header'] != stats['valid_rows']:
            print(f"Discrepancy: Valid row count ({stats['valid_rows']}) differs from rows after header ({stats['rows_after_header']}).")
            print("This indicates malformed rows were dropped or set to null during parsing.")
        else:
            print("No significant discrepancy detected between rows after header and valid rows.")

        df.printSchema()
        df.show(2)
        
        # Add status to stats for summary
        stats["status"] = "SUCCESS"
        processing_summary.append(stats)

    except Exception as e:
        print(f"Error loading {path}: {e}")
        failed_to_load.append((path, str(e)))
        # Add failed status to stats for summary, with default counts
        processing_summary.append({
            "raw_rows": 0,
            "rows_after_header": 0,
            "valid_rows": 0,
            "invalid_rows": 0,
            "filename": file_name,
            "status": "FAILED"
        })

# STEP 2: Write to tables
for path, df in parsed_dfs.items():
    table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    full_table_name = f"{TARGET_SCHEMA}.{table_name}"

    try:
        print(f"\nWriting to table: {full_table_name}")
        df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
        print(f"Successfully written: {full_table_name}")
        successfully_written.append(full_table_name)
    except Exception as e:
        print(f"Failed to write {full_table_name}: {e}")
        failed_to_write.append((full_table_name, str(e)))

# STEP 3: Summary Report
print("\nLOAD SUMMARY")
print(f"Loaded: {len(successfully_loaded)} files")
print(f"Failed to load: {len(failed_to_load)} files")
if failed_to_load:
    for path, err in failed_to_load:
        print(f"   - {path} → {err}")

print("\nWRITE SUMMARY")
print(f"Written to tables: {len(successfully_written)}")
print(f"Failed to write: {len(failed_to_write)} files")
if failed_to_write:
    for tbl, err in failed_to_write:
        print(f"   - {tbl} → {err}")

# STEP 4: Processing Summary Table
print("\n--- Processing Summary ---")
header = [
    "File Name",
    "Raw Rows",
    "Rows After Header",
    "Valid Rows",
    "Invalid Rows",
    "Delta (%)",
    "Status"
]
col_widths = {
    "File Name": max(len(s["filename"]) for s in processing_summary) if processing_summary else len("File Name"),
    "Raw Rows": max(len(str(s["raw_rows"])) for s in processing_summary) if processing_summary else len("Raw Rows"),
    "Rows After Header": max(len(str(s["rows_after_header"])) for s in processing_summary) if processing_summary else len("Rows After Header"),
    "Valid Rows": max(len(str(s["valid_rows"])) for s in processing_summary) if processing_summary else len("Valid Rows"),
    "Invalid Rows": max(len(str(s["invalid_rows"])) for s in processing_summary) if processing_summary else len("Invalid Rows"),
    "Delta (%)": len("Delta (%)"), 
    "Status": max(len(s["status"]) for s in processing_summary) if processing_summary else len("Status")
}

for h in header:
    col_widths[h] = max(col_widths[h], len(h))

header_str = " | ".join([f"{h:<{col_widths[h]}}" for h in header])
print(header_str)
print("-" * len(header_str))

for stats in processing_summary:
    delta_percent = 0.0
    if stats["rows_after_header"] > 0:
        delta_percent = ((stats["rows_after_header"] - stats["valid_rows"]) / stats["rows_after_header"]) * 100
    
    row_data = [
        f"{stats['filename']:<{col_widths['File Name']}}",
        f"{stats['raw_rows']:>{col_widths['Raw Rows']}}",
        f"{stats['rows_after_header']:>{col_widths['Rows After Header']}}",
        f"{stats['valid_rows']:>{col_widths['Valid Rows']}}",
        f"{stats['invalid_rows']:>{col_widths['Invalid Rows']}}",
        f"{delta_percent:>{col_widths['Delta (%)']}.2f}%",
        f"{stats['status']:<{col_widths['Status']}}"
    ]
    print(" | ".join(row_data))

print("\n--- Malformed Rows ---")
for path, df_with_corrupt_record in corrupt_dfs.items():
    file_name = os.path.basename(path)
    malformed_rows_df = df_with_corrupt_record.filter(df_with_corrupt_record["_corrupt_record"].isNotNull())
    print(f"Malformed rows in {file_name}:")
    malformed_rows_df.show(truncate=False)

print("\n--- End of Summary ---")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 26, Finished, Available, Finished)

Found 28 CSV files

Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz
Error loading abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetAttributes.csv.gz: name 'df_with_corrupt_record' is not defined

Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetClassAttributes.csv
Error loading abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetClassAttributes.csv: name 'df_with_corrupt_record' is not defined

Inspecting: abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/AssetClassChangeLogs.csv
Error loading abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b5

# !! new approach 2nd July

In [25]:
%%sql
-- CREATE DATABASE aims_dbo4;

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, 27, Finished, Available, Finished)

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near end of input.(line 1, pos 29)

== SQL ==
-- CREATE DATABASE aims_dbo4;
-----------------------------^^^


In [ ]:
from notebookutils import mssparkutils
import os
from urllib.parse import urlparse
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
from pyspark.sql.functions import col

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, -1, Cancelled, , Cancelled)

In [ ]:
FILE_SCHEMAS = {
    "AssetAttributes.csv.gz": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSET", StringType(), True),
        StructField("ATTRIBUTE", StringType(), True),
        StructField("TEXTVALUE", StringType(), True),
        StructField("NUMERICVALUE", StringType(), True),
        StructField("DATEVALUE", StringType(), True),
        StructField("DECIMALVALUE2DP", StringType(), True),
        StructField("SELECTIONVALUE", StringType(), True),
        StructField("LOCATION", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("HS2_KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("BASEDON", StringType(), True),
        StructField("OWNER", StringType(), True),
        StructField("PHASE", StringType(), True),
        StructField("PERSON", StringType(), True),
        StructField("ORGANISATION", StringType(), True),
        StructField("TEXTSTOREFORTYPETRANSITION", StringType(), True),
        StructField("DECIMALSTOREFORTYPETRANSITION", StringType(), True),
        StructField("NUMERICSTOREFORTYPETRANSITION", StringType(), True),
        StructField("STARTCHAINAGE", IntegerType(), True), # Updated to IntegerType
        StructField("ENDCHAINAGE", StringType(), True),
        StructField("NUMERICENDVALUE", StringType(), True),
        StructField("DECIMALENDVALUE2DP", StringType(), True),
        StructField("ISBEINGSPLIT", StringType(), True),
        StructField("SPLITATCHAINAGE", StringType(), True),
        StructField("AFTERNUMERICSTARTVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("AFTERNUMERICENDVALUE", StringType(), True),
        StructField("AFTERDECIMALSTARTVALUE2DP", StringType(), True),
        StructField("AFTERDECIMALENDVALUE2DP", StringType(), True),
        StructField("AFTERTEXTVALUE", StringType(), True),
        StructField("AFTERDATEVALUE", StringType(), True),
        StructField("AFTERSELECTIONVALUE", StringType(), True),
        StructField("ROUTE", StringType(), True),
        StructField("DECIMALVALUE3DP", StringType(), True),
        StructField("DECIMALENDVALUE3DP", StringType(), True),
        StructField("AFTERDECIMALSTARTVALUE3DP", StringType(), True),
        StructField("AFTERDECIMALENDVALUE3DP", StringType(), True),
        StructField("COPIEDFROM", StringType(), True),
        StructField("ASSETCLASS", StringType(), True),
        StructField("DECIMALENDSTOREFORTYPETRANS", StringType(), True),
        StructField("NUMERICENDSTOREFORTYPETRANS", IntegerType(), True), # Updated to IntegerType
        StructField("DECIMALVALUE4DP", DoubleType(), True), # Updated to DoubleType
        StructField("CONSTRAINTOGROUP", IntegerType(), True), # Updated to IntegerType
        StructField("ADDRESSLINE1", StringType(), True),
        StructField("ADDRESSLINE2", StringType(), True),
        StructField("ADDRESSLINE3", StringType(), True),
        StructField("TOWNCITY", StringType(), True),
        StructField("POSTCODE", StringType(), True),
        StructField("STAGE", IntegerType(), True), # Updated to IntegerType
        StructField("STATUS", IntegerType(), True), # Updated to IntegerType
        StructField("CHAINAGEBASELINE", IntegerType(), True), # Updated to IntegerType
        StructField("BOOLEANVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("TIMEVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("ACCEPTEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("NEWCONTRACTSTAGECODE", IntegerType(), True), # Updated to IntegerType
        StructField("MEMOVALUE", StringType(), True),
        StructField("MEMOSTOREFORTYPETRANSITION", StringType(), True),
        StructField("MATRIXUPDATEWIZARD", StringType(), True),
        StructField("REQUIRESATTRIBUTEDERIVATION", StringType(), True),
        StructField("TEXTDOMAINVALUE", StringType(), True),
        StructField("AFTERTEXTDOMAINVALUE", StringType(), True),
        StructField("VALUESTATUS", StringType(), True),
        StructField("DEFAULTEDFROM", StringType(), True),
        StructField("ACCEPTEDOUTSIDERANGE", StringType(), True),
        StructField("INFORMATIONPACKAGE", StringType(), True),
        StructField("PUBLISHEDATSTAGE", StringType(), True),
        StructField("ISLATESTPUBLISHED", IntegerType(), True), # Updated to IntegerType
        StructField("WORKORDER", IntegerType(), True), # Updated to IntegerType
        StructField("TRACK", IntegerType(), True), # Updated to IntegerType
        StructField("TYPECONVERSIONFAILED", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True) # Column to capture malformed rows
    ]),
    "AssetClassAttributes.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSETCLASS", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("REQUIREDATSTAGEOLD", StringType(), True),
        StructField("AVAILABLEFROMSTAGEOLD", IntegerType(), True), # Updated to IntegerType
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("EXPECTED", IntegerType(), True), # Updated to IntegerType
        StructField("DESCRIPTION", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("AVAILABLEFROMSTAGE", IntegerType(), True), # Updated to IntegerType
        StructField("REQUIREDATSTAGE", IntegerType(), True), # Updated to IntegerType
        StructField("CREATEDBYUPDATE", IntegerType(), True), # Updated to IntegerType
        StructField("OWNER", IntegerType(), True), # Updated to IntegerType
        StructField("MODELLINGPROPERTY", IntegerType(), True), # Updated to IntegerType
        StructField("STATUS", StringType(), True),
        StructField("LOCATIONSSETDENORM", IntegerType(), True), # Updated to IntegerType
        StructField("LOCATIONSUNSETDENORM", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTEDOMAIN", IntegerType(), True), # Updated to IntegerType
        StructField("PHASEAPPLICABILITY", IntegerType(), True), # Updated to IntegerType
        StructField("PHASEAPPLICABILITYDENORM", StringType(), True),
        StructField("SETFROMEXEMPLAR", IntegerType(), True), # Updated to IntegerType
        StructField("IGNOREINTRANSITIONCHECK", IntegerType(), True), # Updated to IntegerType
        StructField("INHERIT", IntegerType(), True), # Updated to IntegerType
        StructField("INHERITEDFROM", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEBY", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEDATE", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClassChangeLogs.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSETCLASS", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("CHAINAGE", StringType(), True),
        StructField("POSTALADDRESS", StringType(), True),
        StructField("GRIDREFERENCE", StringType(), True),
        StructField("CHANGEDATE", StringType(), True),
        StructField("CHANGES", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("PHASE", StringType(), True),
        StructField("IDENTIFICATIONPLATEINFO", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClassRelationships.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("SOURCEASSETCLASS", IntegerType(), True), # Updated to IntegerType
        StructField("TARGETASSETCLASS", IntegerType(), True), # Updated to IntegerType
        StructField("RELATIONSHIPTYPE", IntegerType(), True), # Updated to IntegerType
        StructField("REQUIREDATSTAGEOLD", StringType(), True),
        StructField("EXPECTED", IntegerType(), True), # Updated to IntegerType
        StructField("ADDITIONALDESCRIPTION", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("REQUIREDATSTAGE", IntegerType(), True), # Updated to IntegerType
        StructField("CREATEDBYUPDATE", IntegerType(), True), # Updated to IntegerType
        StructField("OWNER", IntegerType(), True), # Updated to IntegerType
        StructField("UUID", StringType(), True),
        StructField("STATUS", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEBY", IntegerType(), True), # Updated to IntegerType
        StructField("MOVETOLIVEDATE", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClasses.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("PARENT", StringType(), True),
        StructField("LEVELNUMBER", StringType(), True),
        StructField("GROUPINGCLASSFORHIERARCHYREPOR", StringType(), True),
        StructField("OLDHIERARCHYENTITY", StringType(), True),
        StructField("DISPLAYCOLOUR", StringType(), True),
        StructField("CONTAINER", StringType(), True),
        StructField("BACKGROUNDCOLOUR", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("PARENTINHERITEDKEY", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("DOCUMENTNUMBER", StringType(), True),
        StructField("CURRENTVERSION", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("CHAINAGEAPPLICABILITY", StringType(), True),
        StructField("POSTALADDRESSAPPLICABILITY", StringType(), True),
        StructField("GRIDCOORDINATES", StringType(), True),
        StructField("ASSETLEVEL", IntegerType(), True), # Updated to IntegerType
        StructField("INCLUDEIDENTIFCATIONPLATES", StringType(), True),
        StructField("IDENTIFICATIONPLATEDESCRIPTION", StringType(), True),
        StructField("DEFAULTVISUALISATION", IntegerType(), True), # Updated to IntegerType
        StructField("INTANGIBLEGROUPINGCODECHARACTE", StringType(), True),
        StructField("INTANGIBLENLR", IntegerType(), True), # Updated to IntegerType
        StructField("SHOWGISVIEWERPANEL", IntegerType(), True), # Updated to IntegerType
        StructField("OWNER", IntegerType(), True), # Updated to IntegerType
        StructField("ASSETFORMTITLE", StringType(), True),
        StructField("UUID", IntegerType(), True), # Updated to IntegerType
        StructField("ALLOWATTRIBUTEGRAPHING", IntegerType(), True), # Updated to IntegerType
        StructField("NEARESTAUTHORITATIVECLASS", IntegerType(), True), # Updated to IntegerType
        StructField("SCOPEBOOKDEFINITION", IntegerType(), True), # Updated to IntegerType
        StructField("SENSITIVITYLEVEL", IntegerType(), True), # Updated to IntegerType
        StructField("RELSHIPSECTIONHEADEROVERRIDE", StringType(), True),
        StructField("PRIMARYEXEMPLAR", IntegerType(), True), # Updated to IntegerType
        StructField("PHASEAPPLICABILITY", StringType(), True),
        StructField("LATESTLOG", StringType(), True),
        StructField("HIDEINHIERARCHYREPORT", IntegerType(), True), # Updated to IntegerType
        StructField("DISALLOWSELECTIONFORASSETS", IntegerType(), True), # Updated to IntegerType
        StructField("INSTRUCTEDDATE", IntegerType(), True), # Updated to IntegerType
        StructField("STATUS", IntegerType(), True), # Updated to IntegerType
        StructField("HASDRAFTITEM", IntegerType(), True), # Updated to IntegerType
        StructField("CANHAVEASSETPLANS", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEBY", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEDATE", StringType(), True),
        StructField("UAIDOVERRIDECHARACTER", IntegerType(), True), # Updated to IntegerType
        StructField("ALLOWASSIGNMENTOFPRODUCT", StringType(), True),
        StructField("DISPLAYALERTONASSETFORM", IntegerType(), True), # Updated to IntegerType
        StructField("ALERTTEXT", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVALREASON", StringType(), True),
        StructField("INTANGIBLE", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetLocations.csv.gz": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSET", IntegerType(), True), # Updated to IntegerType
        StructField("ROUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ASSETCLASS", IntegerType(), True), # Updated to IntegerType
        StructField("INSTANCEHIDDEN", IntegerType(), True), # Updated to IntegerType
        StructField("STARTCHAINAGE", DoubleType(), True), # Updated to DoubleType
        StructField("ENDCHAINAGE", DoubleType(), True), # Updated to DoubleType
        StructField("TRACKIDOLD", DoubleType(), True), # Updated to DoubleType
        StructField("KINO_CSID", DoubleType(), True), # Updated to DoubleType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("ASSIGNEDTOORGANISATION", IntegerType(), True), # Updated to IntegerType
        StructField("ROUTEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("STARTCHAINAGEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ENDCHAINAGEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("TRACKIDASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("INSTANCEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("COPIEDFROM", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("LOCATIONID", IntegerType(), True), # Updated to IntegerType
        StructField("TRACKAPPLICABILITY", IntegerType(), True), # Updated to IntegerType
        StructField("TRACKAPPLICABILITYASSETATTRIBU", IntegerType(), True), # Updated to IntegerType
        StructField("ASSETCLASSASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVEDREASON", StringType(), True),
        StructField("CHAINAGEUNITS", IntegerType(), True), # Updated to IntegerType
        StructField("STARTCHAINAGEMILES", DoubleType(), True), # Updated to DoubleType
        StructField("ENDCHAINAGEMILES", DoubleType(), True), # Updated to DoubleType
        StructField("OSGBEASTINGASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBNORTHINGASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ADDRESSATTRIBUTEVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("OSGBNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("CHAINAGEBASELINE", IntegerType(), True), # Updated to IntegerType
        StructField("BASELINEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("UNICLASS", StringType(), True),
        StructField("OFFSETFIELD", DoubleType(), True), # Updated to DoubleType
        StructField("OFFSETASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("SUPPRESSPRIMARYEXEMPLARCOPY", IntegerType(), True), # Updated to IntegerType
        StructField("STARTOSGBEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("STARTOSGBNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("ENDOSGBEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("ENDOSGBNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("OSGBEASTINGASSETATTRIBUTE1", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBNORTHINGASSETATTRIBUTE1", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBEASTINGASSETATTRIBUTE2", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBNORTHINGASSETATTRIBUTE2", IntegerType(), True), # Updated to IntegerType
        StructField("ENDOFFSET", DoubleType(), True), # Updated to DoubleType
        StructField("ENDOFFSETASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("SNAKEGRIDSTARTEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("SNAKEGRIDSTARTNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("SNAKEGRIDENDEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("SNAKEGRIDENDNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("TRACK", IntegerType(), True), # Updated to IntegerType
        StructField("TRACKASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("STARTOFFSET", DoubleType(), True), # Updated to DoubleType
        StructField("STARTOFFSETASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("SNAKEGRIDEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("SNAKEGRIDNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("H2SGEASTINGASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGEASTING1ASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGEASTING2ASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGNORTHINGASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGNORTHING1ASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGNORTHING2ASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ADDEDTOARCQUEUE", StringType(), True),
        StructField("STARTCHAINAGEEXPLICITZERO", IntegerType(), True), # Updated to IntegerType
        StructField("ENDCHAINAGEEXPLICITZERO", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetStatuses.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("PUBLISHED", IntegerType(), True), # Updated to IntegerType
        StructField("CODE", StringType(), True),
        StructField("CLIENTSHARED", IntegerType(), True), # Updated to IntegerType
        StructField("IGNOREINDAILYCOUNTS", IntegerType(), True), # Updated to IntegerType
        StructField("ASSETSTATUSFORAUTHOST", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYORDER", IntegerType(), True), # Updated to IntegerType
        StructField("STAGESPECIFIC", IntegerType(), True), # Updated to IntegerType
        StructField("STAGE", StringType(), True),
        StructField("ALLOWSENDINGTOAUTHHOST", IntegerType(), True), # Updated to IntegerType
        StructField("EXCLUDEFROMKPIS", IntegerType(), True), # Updated to IntegerType
        StructField("TARGETSTATUSWHENPUBLISHED", StringType(), True),
        StructField("PRESUBMISSIONAUTOTRANSITION", StringType(), True),
        StructField("SCHEDULESYNCAFTERASSETUPDATE", StringType(), True),
        StructField("IGNOREINSTATUSCHECK", IntegerType(), True), # Updated to IntegerType
        StructField("IGNOREWHENCASCADINGTRANSITIONS", IntegerType(), True), # Updated to IntegerType
        StructField("ALLOWFORGENERICASSETLOADER", IntegerType(), True), # Updated to IntegerType
        StructField("COORDINATESREQUIRED", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Assets.csv.gz": StructType([
        StructField("ID", StringType(), True),
        StructField("OWNER", StringType(), True),
        StructField("PHASE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("NOTES", StringType(), True),
        StructField("HYBRIDBILLCODE", StringType(), True),
        StructField("ACCESSDBID", StringType(), True),
        StructField("OWNERASSETATTRIBUTE", StringType(), True),
        StructField("PHASEASSETATTRIBUTE", StringType(), True),
        StructField("HYBRIDBILLCODEASSETATTRIBUTE", StringType(), True),
        StructField("NAMEASSETATTRIBUTE", StringType(), True),
        StructField("REFERENCECHAINAGE", StringType(), True),
        StructField("LENGTH", IntegerType(), True), # Updated to IntegerType
        StructField("LEGACYID", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("DELETED", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("XMLDEBUG", StringType(), True),
        StructField("UNIQUEASSETID", StringType(), True),
        StructField("LASTUPDATEDATE", StringType(), True),
        StructField("REFCHAINAGEASSETATTRIBUTE", StringType(), True),
        StructField("LENGTHASSETATTRIBUTE", StringType(), True),
        StructField("NOTESASSETATTRIBUTE", StringType(), True),
        StructField("COUNTLOCATIONSDENORM", StringType(), True),
        StructField("HASMULTIPLELOCATIONSDENORM", StringType(), True),
        StructField("COPIEDFROM", IntegerType(), True), # Updated to IntegerType
        StructField("LEGACYIDASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVEDREASON", StringType(), True),
        StructField("STAGE", StringType(), True),
        StructField("STAGEASSETATTRIBUTE", StringType(), True),
        StructField("STATUS", StringType(), True),
        StructField("STATUSASSETATTRIBUTE", StringType(), True),
        StructField("WBSCODE", StringType(), True),
        StructField("PROCESSEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("DELIVERYAREACODE", IntegerType(), True), # Updated to IntegerType
        StructField("REMOTEIDOUTSTANDING", IntegerType(), True), # Updated to IntegerType
        StructField("ISPRIMARYASSET", IntegerType(), True), # Updated to IntegerType
        StructField("ISPRIMARYASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("UUID", IntegerType(), True), # Updated to IntegerType
        StructField("DATALOADED", IntegerType(), True), # Updated to IntegerType
        StructField("RELATIONSHIPSNOTFULLYRESOLVED", IntegerType(), True), # Updated to IntegerType
        StructField("SUPPLYCHAINID", IntegerType(), True), # Updated to IntegerType
        StructField("SENIORPROJECTMANAGER", IntegerType(), True), # Updated to IntegerType
        StructField("SENIORCONSTRUCTIONMANAGER", IntegerType(), True), # Updated to IntegerType
        StructField("EVENTUALOWNER", IntegerType(), True), # Updated to IntegerType
        StructField("ACCESSCONTROLOBJECT", IntegerType(), True), # Updated to IntegerType
        StructField("AUTHORITATIVESTATUS", IntegerType(), True), # Updated to IntegerType
        StructField("AUTHORITATIVESTAGE", IntegerType(), True), # Updated to IntegerType
        StructField("OVERRIDEASSETLEVEL", IntegerType(), True), # Updated to IntegerType
        StructField("LATESTINBOUNDDL_REQUEST", StringType(), True),
        StructField("REQUIRESAUTHHOSTSYNC", IntegerType(), True), # Updated to IntegerType
        StructField("VIEWERTYPES", IntegerType(), True), # Updated to IntegerType
        StructField("METADATANAME", IntegerType(), True), # Updated to IntegerType
        StructField("PRODUCT", IntegerType(), True), # Updated to IntegerType
        StructField("STAGECOMPLIANTDENORM", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AttributeDomainValues.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("ATTRIBUTENLR", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("LOWERBOUND", IntegerType(), True), # Updated to IntegerType
        StructField("UPPERBOUND", DoubleType(), True), # Updated to DoubleType
        StructField("OUPUTVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("FORMAT", IntegerType(), True), # Updated to IntegerType
        StructField("VALUECONTAINSCOMMA", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTEDOMAIN", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AttributeDomains.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("DOMAINTYPE", IntegerType(), True), # Updated to IntegerType
        StructField("LOWERLIMIT", IntegerType(), True), # Updated to IntegerType
        StructField("UPPERLIMIT", IntegerType(), True), # Updated to IntegerType
        StructField("CODE", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("LOWESTEXPECTED", IntegerType(), True), # Updated to IntegerType
        StructField("HIGHESTEXPECTED", IntegerType(), True), # Updated to IntegerType
        StructField("MAXIMUMCHARACTERS", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AttributeGroups.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("PREFIX", StringType(), True),
        StructField("SUFFIX", StringType(), True),
        StructField("DISPLAYORDER", IntegerType(), True), # Updated to IntegerType
        StructField("VISIBILITY", IntegerType(), True), # Updated to IntegerType
        StructField("DEFAULTVIEW", IntegerType(), True), # Updated to IntegerType
        StructField("DEFINESMATRIX", IntegerType(), True), # Updated to IntegerType
        StructField("ROWCOLUMNDELIMITER", StringType(), True),
        StructField("SECURITY", IntegerType(), True), # Updated to IntegerType
        StructField("REALM", StringType(), True),
        StructField("REALMLEVEL", StringType(), True),
        StructField("DEFINESLOCATIONREFERENCE", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Attributes.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("TYPE", StringType(), True),
        StructField("CLASS", StringType(), True),
        StructField("UNITOFMEASURE", StringType(), True),
        StructField("DEFINITION", StringType(), True),
        StructField("ATTRIBUTELEVEL", StringType(), True),
        StructField("GRAPHCOLOUR", StringType(), True),
        StructField("LINEARATTRIBUTE", StringType(), True),
        StructField("INTERPOLATION", StringType(), True),
        StructField("DISPLAYORDER", StringType(), True),
        StructField("HIGHVALUEFORRECIPROCALS", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("TRACKATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTEGROUP", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYCODE", StringType(), True),
        StructField("AD4SECTION", StringType(), True),
        StructField("AD4DISPLAYORDER", StringType(), True),
        StructField("AD4MANDATORY", IntegerType(), True), # Updated to IntegerType
        StructField("AD4TYPE", StringType(), True),
        StructField("SOURCE", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTE", StringType(), True),
        StructField("DERIVATIONTYPE", IntegerType(), True), # Updated to IntegerType
        StructField("STATICVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("FORMATTING", IntegerType(), True), # Updated to IntegerType
        StructField("MATRIXROW", IntegerType(), True), # Updated to IntegerType
        StructField("MATRIXCOLUMN", IntegerType(), True), # Updated to IntegerType
        StructField("CONTRACTORTOCOMPLETE", IntegerType(), True), # Updated to IntegerType
        StructField("OWNER", StringType(), True),
        StructField("AGGREGATABLE", IntegerType(), True), # Updated to IntegerType
        StructField("LMJVPROPERTY", IntegerType(), True), # Updated to IntegerType
        StructField("RELATIONSHIPTYPE", StringType(), True),
        StructField("AGGREGATEATTRIBUTE", StringType(), True),
        StructField("AGGREGATEFUNCTION", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTEDOMAIN", IntegerType(), True), # Updated to IntegerType
        StructField("CONTRACTOROVERRIDE", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("REMOVALREASON", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "ChainageBaselines.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "LinkTypes.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("SYSTEMFIELD", StringType(), True),
        StructField("DESLINK", IntegerType(), True), # Updated to IntegerType
        StructField("URLROOT", StringType(), True),
        StructField("URLTAIL", StringType(), True),
        StructField("SOURCE", IntegerType(), True), # Updated to IntegerType
        StructField("INCLUDEINWEBSERVICES", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYORDER", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYASPRIMARYASSETIMAGE", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Links.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSET", IntegerType(), True), # Updated to IntegerType
        StructField("LINKTYPE", IntegerType(), True), # Updated to IntegerType
        StructField("ADDRESS", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("HS2_KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("URL", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("BASEDON", IntegerType(), True), # Updated to IntegerType
        StructField("COPIEDFROM", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("FILETYPE", IntegerType(), True), # Updated to IntegerType
        StructField("FILEFIELD", StringType(), True),
        StructField("FILENAMEFIELD", StringType(), True),
        StructField("IMAGEWIDTH", IntegerType(), True), # Updated to IntegerType
        StructField("FILESIZE", IntegerType(), True), # Updated to IntegerType
        StructField("PROCESSEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Organisations.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("PARENT", IntegerType(), True), # Updated to IntegerType
        StructField("PARENTKEYHIERARCHY", StringType(), True),
        StructField("OWNER", IntegerType(), True), # Updated to IntegerType
        StructField("ISCONTRACTOR", IntegerType(), True), # Updated to IntegerType
        StructField("DEFAULTASSETSTATUS", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYORDER", IntegerType(), True), # Updated to IntegerType
        StructField("ISPRINCIPALCONTRACTOR", IntegerType(), True), # Updated to IntegerType
        StructField("ISUAALLOWABLE", IntegerType(), True), # Updated to IntegerType
        StructField("DEFAULTEMPLOYER", StringType(), True),
        StructField("WITHINLMJVSCOPE", IntegerType(), True), # Updated to IntegerType
        StructField("CONTACTNAME", StringType(), True),
        StructField("CONTACTNUMBER", StringType(), True),
        StructField("CONTACTEMAIL", StringType(), True),
        StructField("SUPPRESSCONTRACTORTOCOMPLETE", IntegerType(), True), # Updated to IntegerType
        StructField("ALLOWCHILDRENTOBESELECTEDASTHI", IntegerType(), True), # Updated to IntegerType
        StructField("ECOLOGYORIGINATORCODE", StringType(), True),
        StructField("ATTRIBUTESTOUPDATEINDATALOADS", IntegerType(), True), # Updated to IntegerType
        StructField("PREFILTERFORASSETRESULTS", IntegerType(), True), # Updated to IntegerType
        StructField("FILTERCRITERIA", StringType(), True),
        StructField("APPLICABLEASSETCLASSES", IntegerType(), True), # Updated to IntegerType
        StructField("LEADER", StringType(), True),
        StructField("ALLOWINFODELIVERYPACKAGING", IntegerType(), True), # Updated to IntegerType
        StructField("DEFAULTASSIGNEEFORWORKORDERS", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Owners.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("UAIDPREFIXOVERRIDE", StringType(), True),
        StructField("ALLOWATTRIBUTES", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "People.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("FIRSTNAME", StringType(), True),
        StructField("LASTNAME", StringType(), True),
        StructField("USERID", IntegerType(), True), # Updated to IntegerType
        StructField("ORGANISATION", IntegerType(), True), # Updated to IntegerType
        StructField("PREDECESSOR", StringType(), True),
        StructField("SUCCEEDEDON", StringType(), True),
        StructField("EMAILADDRESS", StringType(), True),
        StructField("CREATEUSERPROCESS", IntegerType(), True), # Updated to IntegerType
        StructField("INITIALPASSWORD", StringType(), True),
        StructField("USERIDFORNEWACCOUNT", StringType(), True),
        StructField("CURRENTBOOKMARK", IntegerType(), True), # Updated to IntegerType
        StructField("HOLDDATALOADS", IntegerType(), True), # Updated to IntegerType
        StructField("SHOWGVIEWERFRAMEONASSETFORM", IntegerType(), True), # Updated to IntegerType
        StructField("TEMPLATE", StringType(), True),
        StructField("EMAILNEWUSER", IntegerType(), True), # Updated to IntegerType
        StructField("INCLUDEINKPIFILTER", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ])
    # Add other schemas here as needed for other CSV files.
    # For any file not explicitly listed here, the script will fall back to
    # header detection and inferring a schema based on the first few lines.
}


StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, -1, Cancelled, , Cancelled)

In [ ]:
from notebookutils import mssparkutils
import os
from urllib.parse import urlparse
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
from pyspark.sql.functions import col

# --- 1. Define Schemas for Specific Files ---
# These schemas are based on the 'root' schema output you provided.
# Data types have been refined (e.g., to IntegerType, DoubleType) where specified.
# A '_corrupt_record' column is added to each schema to capture malformed rows.

FILE_SCHEMAS = {
    "AssetAttributes.csv.gz": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSET", StringType(), True),
        StructField("ATTRIBUTE", StringType(), True),
        StructField("TEXTVALUE", StringType(), True),
        StructField("NUMERICVALUE", StringType(), True),
        StructField("DATEVALUE", StringType(), True),
        StructField("DECIMALVALUE2DP", StringType(), True),
        StructField("SELECTIONVALUE", StringType(), True),
        StructField("LOCATION", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("HS2_KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("BASEDON", StringType(), True),
        StructField("OWNER", StringType(), True),
        StructField("PHASE", StringType(), True),
        StructField("PERSON", StringType(), True),
        StructField("ORGANISATION", StringType(), True),
        StructField("TEXTSTOREFORTYPETRANSITION", StringType(), True),
        StructField("DECIMALSTOREFORTYPETRANSITION", StringType(), True),
        StructField("NUMERICSTOREFORTYPETRANSITION", StringType(), True),
        StructField("STARTCHAINAGE", IntegerType(), True), # Updated to IntegerType
        StructField("ENDCHAINAGE", StringType(), True),
        StructField("NUMERICENDVALUE", StringType(), True),
        StructField("DECIMALENDVALUE2DP", StringType(), True),
        StructField("ISBEINGSPLIT", StringType(), True),
        StructField("SPLITATCHAINAGE", StringType(), True),
        StructField("AFTERNUMERICSTARTVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("AFTERNUMERICENDVALUE", StringType(), True),
        StructField("AFTERDECIMALSTARTVALUE2DP", StringType(), True),
        StructField("AFTERDECIMALENDVALUE2DP", StringType(), True),
        StructField("AFTERTEXTVALUE", StringType(), True),
        StructField("AFTERDATEVALUE", StringType(), True),
        StructField("AFTERSELECTIONVALUE", StringType(), True),
        StructField("ROUTE", StringType(), True),
        StructField("DECIMALVALUE3DP", StringType(), True),
        StructField("DECIMALENDVALUE3DP", StringType(), True),
        StructField("AFTERDECIMALSTARTVALUE3DP", StringType(), True),
        StructField("AFTERDECIMALENDVALUE3DP", StringType(), True),
        StructField("COPIEDFROM", StringType(), True),
        StructField("ASSETCLASS", StringType(), True),
        StructField("DECIMALENDSTOREFORTYPETRANS", StringType(), True),
        StructField("NUMERICENDSTOREFORTYPETRANS", IntegerType(), True), # Updated to IntegerType
        StructField("DECIMALVALUE4DP", DoubleType(), True), # Updated to DoubleType
        StructField("CONSTRAINTOGROUP", IntegerType(), True), # Updated to IntegerType
        StructField("ADDRESSLINE1", StringType(), True),
        StructField("ADDRESSLINE2", StringType(), True),
        StructField("ADDRESSLINE3", StringType(), True),
        StructField("TOWNCITY", StringType(), True),
        StructField("POSTCODE", StringType(), True),
        StructField("STAGE", IntegerType(), True), # Updated to IntegerType
        StructField("STATUS", IntegerType(), True), # Updated to IntegerType
        StructField("CHAINAGEBASELINE", IntegerType(), True), # Updated to IntegerType
        StructField("BOOLEANVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("TIMEVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("ACCEPTEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("NEWCONTRACTSTAGECODE", IntegerType(), True), # Updated to IntegerType
        StructField("MEMOVALUE", StringType(), True),
        StructField("MEMOSTOREFORTYPETRANSITION", StringType(), True),
        StructField("MATRIXUPDATEWIZARD", StringType(), True),
        StructField("REQUIRESATTRIBUTEDERIVATION", StringType(), True),
        StructField("TEXTDOMAINVALUE", StringType(), True),
        StructField("AFTERTEXTDOMAINVALUE", StringType(), True),
        StructField("VALUESTATUS", StringType(), True),
        StructField("DEFAULTEDFROM", StringType(), True),
        StructField("ACCEPTEDOUTSIDERANGE", StringType(), True),
        StructField("INFORMATIONPACKAGE", StringType(), True),
        StructField("PUBLISHEDATSTAGE", StringType(), True),
        StructField("ISLATESTPUBLISHED", IntegerType(), True), # Updated to IntegerType
        StructField("WORKORDER", IntegerType(), True), # Updated to IntegerType
        StructField("TRACK", IntegerType(), True), # Updated to IntegerType
        StructField("TYPECONVERSIONFAILED", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True) # Column to capture malformed rows
    ]),
    "AssetClassAttributes.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSETCLASS", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("REQUIREDATSTAGEOLD", StringType(), True),
        StructField("AVAILABLEFROMSTAGEOLD", IntegerType(), True), # Updated to IntegerType
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("EXPECTED", IntegerType(), True), # Updated to IntegerType
        StructField("DESCRIPTION", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("AVAILABLEFROMSTAGE", IntegerType(), True), # Updated to IntegerType
        StructField("REQUIREDATSTAGE", IntegerType(), True), # Updated to IntegerType
        StructField("CREATEDBYUPDATE", IntegerType(), True), # Updated to IntegerType
        StructField("OWNER", IntegerType(), True), # Updated to IntegerType
        StructField("MODELLINGPROPERTY", IntegerType(), True), # Updated to IntegerType
        StructField("STATUS", StringType(), True),
        StructField("LOCATIONSSETDENORM", IntegerType(), True), # Updated to IntegerType
        StructField("LOCATIONSUNSETDENORM", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTEDOMAIN", IntegerType(), True), # Updated to IntegerType
        StructField("PHASEAPPLICABILITY", IntegerType(), True), # Updated to IntegerType
        StructField("PHASEAPPLICABILITYDENORM", StringType(), True),
        StructField("SETFROMEXEMPLAR", IntegerType(), True), # Updated to IntegerType
        StructField("IGNOREINTRANSITIONCHECK", IntegerType(), True), # Updated to IntegerType
        StructField("INHERIT", IntegerType(), True), # Updated to IntegerType
        StructField("INHERITEDFROM", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEBY", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEDATE", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClassChangeLogs.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSETCLASS", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("CHAINAGE", StringType(), True),
        StructField("POSTALADDRESS", StringType(), True),
        StructField("GRIDREFERENCE", StringType(), True),
        StructField("CHANGEDATE", StringType(), True),
        StructField("CHANGES", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("PHASE", StringType(), True),
        StructField("IDENTIFICATIONPLATEINFO", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClassRelationships.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("SOURCEASSETCLASS", IntegerType(), True), # Updated to IntegerType
        StructField("TARGETASSETCLASS", IntegerType(), True), # Updated to IntegerType
        StructField("RELATIONSHIPTYPE", IntegerType(), True), # Updated to IntegerType
        StructField("REQUIREDATSTAGEOLD", StringType(), True),
        StructField("EXPECTED", IntegerType(), True), # Updated to IntegerType
        StructField("ADDITIONALDESCRIPTION", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("REQUIREDATSTAGE", IntegerType(), True), # Updated to IntegerType
        StructField("CREATEDBYUPDATE", IntegerType(), True), # Updated to IntegerType
        StructField("OWNER", IntegerType(), True), # Updated to IntegerType
        StructField("UUID", StringType(), True),
        StructField("STATUS", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEBY", IntegerType(), True), # Updated to IntegerType
        StructField("MOVETOLIVEDATE", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetClasses.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("PARENT", StringType(), True),
        StructField("LEVELNUMBER", StringType(), True),
        StructField("GROUPINGCLASSFORHIERARCHYREPOR", StringType(), True),
        StructField("OLDHIERARCHYENTITY", StringType(), True),
        StructField("DISPLAYCOLOUR", StringType(), True),
        StructField("CONTAINER", StringType(), True),
        StructField("BACKGROUNDCOLOUR", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("PARENTINHERITEDKEY", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("DOCUMENTNUMBER", StringType(), True),
        StructField("CURRENTVERSION", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("CHAINAGEAPPLICABILITY", StringType(), True),
        StructField("POSTALADDRESSAPPLICABILITY", StringType(), True),
        StructField("GRIDCOORDINATES", StringType(), True),
        StructField("ASSETLEVEL", IntegerType(), True), # Updated to IntegerType
        StructField("INCLUDEIDENTIFCATIONPLATES", StringType(), True),
        StructField("IDENTIFICATIONPLATEDESCRIPTION", StringType(), True),
        StructField("DEFAULTVISUALISATION", IntegerType(), True), # Updated to IntegerType
        StructField("INTANGIBLEGROUPINGCODECHARACTE", StringType(), True),
        StructField("INTANGIBLENLR", IntegerType(), True), # Updated to IntegerType
        StructField("SHOWGISVIEWERPANEL", IntegerType(), True), # Updated to IntegerType
        StructField("OWNER", IntegerType(), True), # Updated to IntegerType
        StructField("ASSETFORMTITLE", StringType(), True),
        StructField("UUID", IntegerType(), True), # Updated to IntegerType
        StructField("ALLOWATTRIBUTEGRAPHING", IntegerType(), True), # Updated to IntegerType
        StructField("NEARESTAUTHORITATIVECLASS", IntegerType(), True), # Updated to IntegerType
        StructField("SCOPEBOOKDEFINITION", IntegerType(), True), # Updated to IntegerType
        StructField("SENSITIVITYLEVEL", IntegerType(), True), # Updated to IntegerType
        StructField("RELSHIPSECTIONHEADEROVERRIDE", StringType(), True),
        StructField("PRIMARYEXEMPLAR", IntegerType(), True), # Updated to IntegerType
        StructField("PHASEAPPLICABILITY", StringType(), True),
        StructField("LATESTLOG", StringType(), True),
        StructField("HIDEINHIERARCHYREPORT", IntegerType(), True), # Updated to IntegerType
        StructField("DISALLOWSELECTIONFORASSETS", IntegerType(), True), # Updated to IntegerType
        StructField("INSTRUCTEDDATE", IntegerType(), True), # Updated to IntegerType
        StructField("STATUS", IntegerType(), True), # Updated to IntegerType
        StructField("HASDRAFTITEM", IntegerType(), True), # Updated to IntegerType
        StructField("CANHAVEASSETPLANS", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEBY", IntegerType(), True), # Updated to IntegerType
        StructField("MOVEDTOLIVEDATE", StringType(), True),
        StructField("UAIDOVERRIDECHARACTER", IntegerType(), True), # Updated to IntegerType
        StructField("ALLOWASSIGNMENTOFPRODUCT", StringType(), True),
        StructField("DISPLAYALERTONASSETFORM", IntegerType(), True), # Updated to IntegerType
        StructField("ALERTTEXT", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVALREASON", StringType(), True),
        StructField("INTANGIBLE", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetLocations.csv.gz": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSET", IntegerType(), True), # Updated to IntegerType
        StructField("ROUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ASSETCLASS", IntegerType(), True), # Updated to IntegerType
        StructField("INSTANCEHIDDEN", IntegerType(), True), # Updated to IntegerType
        StructField("STARTCHAINAGE", DoubleType(), True), # Updated to DoubleType
        StructField("ENDCHAINAGE", DoubleType(), True), # Updated to DoubleType
        StructField("TRACKIDOLD", DoubleType(), True), # Updated to DoubleType
        StructField("KINO_CSID", DoubleType(), True), # Updated to DoubleType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("ASSIGNEDTOORGANISATION", IntegerType(), True), # Updated to IntegerType
        StructField("ROUTEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("STARTCHAINAGEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ENDCHAINAGEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("TRACKIDASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("INSTANCEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("COPIEDFROM", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("LOCATIONID", IntegerType(), True), # Updated to IntegerType
        StructField("TRACKAPPLICABILITY", IntegerType(), True), # Updated to IntegerType
        StructField("TRACKAPPLICABILITYASSETATTRIBU", IntegerType(), True), # Updated to IntegerType
        StructField("ASSETCLASSASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVEDREASON", StringType(), True),
        StructField("CHAINAGEUNITS", IntegerType(), True), # Updated to IntegerType
        StructField("STARTCHAINAGEMILES", DoubleType(), True), # Updated to DoubleType
        StructField("ENDCHAINAGEMILES", DoubleType(), True), # Updated to DoubleType
        StructField("OSGBEASTINGASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBNORTHINGASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ADDRESSATTRIBUTEVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("OSGBNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("CHAINAGEBASELINE", IntegerType(), True), # Updated to IntegerType
        StructField("BASELINEASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("UNICLASS", StringType(), True),
        StructField("OFFSETFIELD", DoubleType(), True), # Updated to DoubleType
        StructField("OFFSETASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("SUPPRESSPRIMARYEXEMPLARCOPY", IntegerType(), True), # Updated to IntegerType
        StructField("STARTOSGBEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("STARTOSGBNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("ENDOSGBEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("ENDOSGBNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("OSGBEASTINGASSETATTRIBUTE1", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBNORTHINGASSETATTRIBUTE1", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBEASTINGASSETATTRIBUTE2", IntegerType(), True), # Updated to IntegerType
        StructField("OSGBNORTHINGASSETATTRIBUTE2", IntegerType(), True), # Updated to IntegerType
        StructField("ENDOFFSET", DoubleType(), True), # Updated to DoubleType
        StructField("ENDOFFSETASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("SNAKEGRIDSTARTEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("SNAKEGRIDSTARTNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("SNAKEGRIDENDEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("SNAKEGRIDENDNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("TRACK", IntegerType(), True), # Updated to IntegerType
        StructField("TRACKASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("STARTOFFSET", DoubleType(), True), # Updated to DoubleType
        StructField("STARTOFFSETASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("SNAKEGRIDEASTING", DoubleType(), True), # Updated to DoubleType
        StructField("SNAKEGRIDNORTHING", DoubleType(), True), # Updated to DoubleType
        StructField("H2SGEASTINGASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGEASTING1ASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGEASTING2ASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGNORTHINGASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGNORTHING1ASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("H2SGNORTHING2ASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ADDEDTOARCQUEUE", StringType(), True),
        StructField("STARTCHAINAGEEXPLICITZERO", IntegerType(), True), # Updated to IntegerType
        StructField("ENDCHAINAGEEXPLICITZERO", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AssetStatuses.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("PUBLISHED", IntegerType(), True), # Updated to IntegerType
        StructField("CODE", StringType(), True),
        StructField("CLIENTSHARED", IntegerType(), True), # Updated to IntegerType
        StructField("IGNOREINDAILYCOUNTS", IntegerType(), True), # Updated to IntegerType
        StructField("ASSETSTATUSFORAUTHOST", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYORDER", IntegerType(), True), # Updated to IntegerType
        StructField("STAGESPECIFIC", IntegerType(), True), # Updated to IntegerType
        StructField("STAGE", StringType(), True),
        StructField("ALLOWSENDINGTOAUTHHOST", IntegerType(), True), # Updated to IntegerType
        StructField("EXCLUDEFROMKPIS", IntegerType(), True), # Updated to IntegerType
        StructField("TARGETSTATUSWHENPUBLISHED", StringType(), True),
        StructField("PRESUBMISSIONAUTOTRANSITION", StringType(), True),
        StructField("SCHEDULESYNCAFTERASSETUPDATE", StringType(), True),
        StructField("IGNOREINSTATUSCHECK", IntegerType(), True), # Updated to IntegerType
        StructField("IGNOREWHENCASCADINGTRANSITIONS", IntegerType(), True), # Updated to IntegerType
        StructField("ALLOWFORGENERICASSETLOADER", IntegerType(), True), # Updated to IntegerType
        StructField("COORDINATESREQUIRED", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Assets.csv.gz": StructType([
        StructField("ID", StringType(), True),
        StructField("OWNER", StringType(), True),
        StructField("PHASE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("NOTES", StringType(), True),
        StructField("HYBRIDBILLCODE", StringType(), True),
        StructField("ACCESSDBID", StringType(), True),
        StructField("OWNERASSETATTRIBUTE", StringType(), True),
        StructField("PHASEASSETATTRIBUTE", StringType(), True),
        StructField("HYBRIDBILLCODEASSETATTRIBUTE", StringType(), True),
        StructField("NAMEASSETATTRIBUTE", StringType(), True),
        StructField("REFERENCECHAINAGE", StringType(), True),
        StructField("LENGTH", IntegerType(), True), # Updated to IntegerType
        StructField("LEGACYID", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("DELETED", StringType(), True),
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("XMLDEBUG", StringType(), True),
        StructField("UNIQUEASSETID", StringType(), True),
        StructField("LASTUPDATEDATE", StringType(), True),
        StructField("REFCHAINAGEASSETATTRIBUTE", StringType(), True),
        StructField("LENGTHASSETATTRIBUTE", StringType(), True),
        StructField("NOTESASSETATTRIBUTE", StringType(), True),
        StructField("COUNTLOCATIONSDENORM", StringType(), True),
        StructField("HASMULTIPLELOCATIONSDENORM", StringType(), True),
        StructField("COPIEDFROM", IntegerType(), True), # Updated to IntegerType
        StructField("LEGACYIDASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("REMOVEDREASON", StringType(), True),
        StructField("STAGE", StringType(), True),
        StructField("STAGEASSETATTRIBUTE", StringType(), True),
        StructField("STATUS", StringType(), True),
        StructField("STATUSASSETATTRIBUTE", StringType(), True),
        StructField("WBSCODE", StringType(), True),
        StructField("PROCESSEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("DELIVERYAREACODE", IntegerType(), True), # Updated to IntegerType
        StructField("REMOTEIDOUTSTANDING", IntegerType(), True), # Updated to IntegerType
        StructField("ISPRIMARYASSET", IntegerType(), True), # Updated to IntegerType
        StructField("ISPRIMARYASSETATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("UUID", IntegerType(), True), # Updated to IntegerType
        StructField("DATALOADED", IntegerType(), True), # Updated to IntegerType
        StructField("RELATIONSHIPSNOTFULLYRESOLVED", IntegerType(), True), # Updated to IntegerType
        StructField("SUPPLYCHAINID", IntegerType(), True), # Updated to IntegerType
        StructField("SENIORPROJECTMANAGER", IntegerType(), True), # Updated to IntegerType
        StructField("SENIORCONSTRUCTIONMANAGER", IntegerType(), True), # Updated to IntegerType
        StructField("EVENTUALOWNER", IntegerType(), True), # Updated to IntegerType
        StructField("ACCESSCONTROLOBJECT", IntegerType(), True), # Updated to IntegerType
        StructField("AUTHORITATIVESTATUS", IntegerType(), True), # Updated to IntegerType
        StructField("AUTHORITATIVESTAGE", IntegerType(), True), # Updated to IntegerType
        StructField("OVERRIDEASSETLEVEL", IntegerType(), True), # Updated to IntegerType
        StructField("LATESTINBOUNDDL_REQUEST", StringType(), True),
        StructField("REQUIRESAUTHHOSTSYNC", IntegerType(), True), # Updated to IntegerType
        StructField("VIEWERTYPES", IntegerType(), True), # Updated to IntegerType
        StructField("METADATANAME", IntegerType(), True), # Updated to IntegerType
        StructField("PRODUCT", IntegerType(), True), # Updated to IntegerType
        StructField("STAGECOMPLIANTDENORM", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AttributeDomainValues.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("ATTRIBUTENLR", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("LOWERBOUND", IntegerType(), True), # Updated to IntegerType
        StructField("UPPERBOUND", DoubleType(), True), # Updated to DoubleType
        StructField("OUPUTVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("FORMAT", IntegerType(), True), # Updated to IntegerType
        StructField("VALUECONTAINSCOMMA", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTEDOMAIN", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AttributeDomains.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("DOMAINTYPE", IntegerType(), True), # Updated to IntegerType
        StructField("LOWERLIMIT", IntegerType(), True), # Updated to IntegerType
        StructField("UPPERLIMIT", IntegerType(), True), # Updated to IntegerType
        StructField("CODE", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("LOWESTEXPECTED", IntegerType(), True), # Updated to IntegerType
        StructField("HIGHESTEXPECTED", IntegerType(), True), # Updated to IntegerType
        StructField("MAXIMUMCHARACTERS", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "AttributeGroups.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("PREFIX", StringType(), True),
        StructField("SUFFIX", StringType(), True),
        StructField("DISPLAYORDER", IntegerType(), True), # Updated to IntegerType
        StructField("VISIBILITY", IntegerType(), True), # Updated to IntegerType
        StructField("DEFAULTVIEW", IntegerType(), True), # Updated to IntegerType
        StructField("DEFINESMATRIX", IntegerType(), True), # Updated to IntegerType
        StructField("ROWCOLUMNDELIMITER", StringType(), True),
        StructField("SECURITY", IntegerType(), True), # Updated to IntegerType
        StructField("REALM", StringType(), True),
        StructField("REALMLEVEL", StringType(), True),
        StructField("DEFINESLOCATIONREFERENCE", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Attributes.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("TYPE", StringType(), True),
        StructField("CLASS", StringType(), True),
        StructField("UNITOFMEASURE", StringType(), True),
        StructField("DEFINITION", StringType(), True),
        StructField("ATTRIBUTELEVEL", StringType(), True),
        StructField("GRAPHCOLOUR", StringType(), True),
        StructField("LINEARATTRIBUTE", StringType(), True),
        StructField("INTERPOLATION", StringType(), True),
        StructField("DISPLAYORDER", StringType(), True),
        StructField("HIGHVALUEFORRECIPROCALS", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("TRACKATTRIBUTE", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTEGROUP", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYCODE", StringType(), True),
        StructField("AD4SECTION", StringType(), True),
        StructField("AD4DISPLAYORDER", StringType(), True),
        StructField("AD4MANDATORY", IntegerType(), True), # Updated to IntegerType
        StructField("AD4TYPE", StringType(), True),
        StructField("SOURCE", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTE", StringType(), True),
        StructField("DERIVATIONTYPE", IntegerType(), True), # Updated to IntegerType
        StructField("STATICVALUE", IntegerType(), True), # Updated to IntegerType
        StructField("FORMATTING", IntegerType(), True), # Updated to IntegerType
        StructField("MATRIXROW", IntegerType(), True), # Updated to IntegerType
        StructField("MATRIXCOLUMN", IntegerType(), True), # Updated to IntegerType
        StructField("CONTRACTORTOCOMPLETE", IntegerType(), True), # Updated to IntegerType
        StructField("OWNER", StringType(), True),
        StructField("AGGREGATABLE", IntegerType(), True), # Updated to IntegerType
        StructField("LMJVPROPERTY", IntegerType(), True), # Updated to IntegerType
        StructField("RELATIONSHIPTYPE", StringType(), True),
        StructField("AGGREGATEATTRIBUTE", StringType(), True),
        StructField("AGGREGATEFUNCTION", IntegerType(), True), # Updated to IntegerType
        StructField("ATTRIBUTEDOMAIN", IntegerType(), True), # Updated to IntegerType
        StructField("CONTRACTOROVERRIDE", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("REMOVALREASON", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "ChainageBaselines.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "LinkTypes.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("SYSTEMFIELD", StringType(), True),
        StructField("DESLINK", IntegerType(), True), # Updated to IntegerType
        StructField("URLROOT", StringType(), True),
        StructField("URLTAIL", StringType(), True),
        StructField("SOURCE", IntegerType(), True), # Updated to IntegerType
        StructField("INCLUDEINWEBSERVICES", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYORDER", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYASPRIMARYASSETIMAGE", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CSID", StringType(), True),
        StructField("KINO_CUID", StringType(), True),
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Links.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("ASSET", IntegerType(), True), # Updated to IntegerType
        StructField("LINKTYPE", IntegerType(), True), # Updated to IntegerType
        StructField("ADDRESS", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("HS2_KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("URL", StringType(), True),
        StructField("VALIDFROM", StringType(), True),
        StructField("VALIDTO", StringType(), True),
        StructField("BASEDON", IntegerType(), True), # Updated to IntegerType
        StructField("COPIEDFROM", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("FILETYPE", IntegerType(), True), # Updated to IntegerType
        StructField("FILEFIELD", StringType(), True),
        StructField("FILENAMEFIELD", StringType(), True),
        StructField("IMAGEWIDTH", IntegerType(), True), # Updated to IntegerType
        StructField("FILESIZE", IntegerType(), True), # Updated to IntegerType
        StructField("PROCESSEDBY", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Organisations.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_UWHEN", StringType(), True),
        StructField("PARENT", IntegerType(), True), # Updated to IntegerType
        StructField("PARENTKEYHIERARCHY", StringType(), True),
        StructField("OWNER", IntegerType(), True), # Updated to IntegerType
        StructField("ISCONTRACTOR", IntegerType(), True), # Updated to IntegerType
        StructField("DEFAULTASSETSTATUS", IntegerType(), True), # Updated to IntegerType
        StructField("DISPLAYORDER", IntegerType(), True), # Updated to IntegerType
        StructField("ISPRINCIPALCONTRACTOR", IntegerType(), True), # Updated to IntegerType
        StructField("ISUAALLOWABLE", IntegerType(), True), # Updated to IntegerType
        StructField("DEFAULTEMPLOYER", StringType(), True),
        StructField("WITHINLMJVSCOPE", IntegerType(), True), # Updated to IntegerType
        StructField("CONTACTNAME", StringType(), True),
        StructField("CONTACTNUMBER", StringType(), True),
        StructField("CONTACTEMAIL", StringType(), True),
        StructField("SUPPRESSCONTRACTORTOCOMPLETE", IntegerType(), True), # Updated to IntegerType
        StructField("ALLOWCHILDRENTOBESELECTEDASTHI", IntegerType(), True), # Updated to IntegerType
        StructField("ECOLOGYORIGINATORCODE", StringType(), True),
        StructField("ATTRIBUTESTOUPDATEINDATALOADS", IntegerType(), True), # Updated to IntegerType
        StructField("PREFILTERFORASSETRESULTS", IntegerType(), True), # Updated to IntegerType
        StructField("FILTERCRITERIA", StringType(), True),
        StructField("APPLICABLEASSETCLASSES", IntegerType(), True), # Updated to IntegerType
        StructField("LEADER", StringType(), True),
        StructField("ALLOWINFODELIVERYPACKAGING", IntegerType(), True), # Updated to IntegerType
        StructField("DEFAULTASSIGNEEFORWORKORDERS", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "Owners.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("NAME", StringType(), True),
        StructField("CODE", StringType(), True),
        StructField("UAIDPREFIXOVERRIDE", StringType(), True),
        StructField("ALLOWATTRIBUTES", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CSID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CUID", IntegerType(), True), # Updated to IntegerType
        StructField("KINO_CWHEN", StringType(), True),
        StructField("KINO_USID", StringType(), True),
        StructField("KINO_UUID", StringType(), True),
        StructField("KINO_UWHEN", StringType(), True),
        StructField("_corrupt_record", StringType(), True)
    ]),
    "People.csv": StructType([
        StructField("ID", StringType(), True),
        StructField("FIRSTNAME", StringType(), True),
        StructField("LASTNAME", StringType(), True),
        StructField("USERID", IntegerType(), True), # Updated to IntegerType
        StructField("ORGANISATION", IntegerType(), True), # Updated to IntegerType
        StructField("PREDECESSOR", StringType(), True),
        StructField("SUCCEEDEDON", StringType(), True),
        StructField("EMAILADDRESS", StringType(), True),
        StructField("CREATEUSERPROCESS", IntegerType(), True), # Updated to IntegerType
        StructField("INITIALPASSWORD", StringType(), True),
        StructField("USERIDFORNEWACCOUNT", StringType(), True),
        StructField("CURRENTBOOKMARK", IntegerType(), True), # Updated to IntegerType
        StructField("HOLDDATALOADS", IntegerType(), True), # Updated to IntegerType
        StructField("SHOWGVIEWERFRAMEONASSETFORM", IntegerType(), True), # Updated to IntegerType
        StructField("TEMPLATE", StringType(), True),
        StructField("EMAILNEWUSER", IntegerType(), True), # Updated to IntegerType
        StructField("INCLUDEINKPIFILTER", IntegerType(), True), # Updated to IntegerType
        StructField("_corrupt_record", StringType(), True)
    ])
    # Add other schemas here as needed for other CSV files.
    # For any file not explicitly listed here, the script will fall back to
    # header detection and inferring a schema based on the first few lines.
}

# --- 2. Helper Function to List CSV Files Recursively ---
def list_csv_files_recursive(base_path):
    """
    Recursively lists all .csv and .csv.gz files within a given base path
    in the OneLake file system.
    """
    csv_files = []
    def walk(path):
        for item in mssparkutils.fs.ls(path):
            if item.isDir:
                walk(item.path)
            elif item.path.endswith(".csv") or item.path.endswith(".csv.gz"):
                csv_files.append(item.path)
    walk(base_path)
    return csv_files

# --- 3. Function to Detect Header, Read CSV with Schema, and Get Stats ---
def detect_header_and_read_csv(path, predefined_schema=None):
    """
    Reads a CSV file, applying a predefined schema if available, and calculates
    various row counts including raw, after header, valid, and invalid rows.
    Handles malformed records using Spark's PERMISSIVE mode and _corrupt_record column.

    Args:
        path (str): The full path to the CSV file.
        predefined_schema (StructType, optional): A Spark StructType schema to apply.
                                                  If None, header will be detected and
                                                  schema inferred. Defaults to None.

    Returns:
        tuple: A tuple containing:
            - pyspark.sql.DataFrame: The processed DataFrame (without _corrupt_record column).
            - dict: A dictionary of processing statistics.
    """
    file_name = os.path.basename(path)

    # Step 1: Count total raw lines in the file
    # This is done before reading with schema to get the absolute raw count
    total_raw_rows = spark.read.text(path).count()

    df = None
    header_index = 0 # Default to 0, assuming header is first row or no header for raw count
    
    # Step 2: Read file with schema (if provided) or infer schema if not
    if predefined_schema:
        print(f"📌 Reading {file_name} with predefined schema.")
        # Use header=True because the schema assumes the first row is the header
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("mode", "PERMISSIVE") \
            .option("delimiter", ",") \
            .option("quote", "\"") \
            .option("escape", "\"") \
            .schema(predefined_schema) \
            .load(path)
        
        # For accurate 'rows_after_header', we assume 1 header row if header=True
        rows_after_header = total_raw_rows - 1 if total_raw_rows > 0 else 0

    else:
        print(f"📌 No predefined schema for {file_name}. Inferring header and schema.")
        # Fallback to original header detection if no predefined schema
        # This part is less robust for malformed rows but handles unknown files
        raw_lines_rdd = spark.read.text(path).rdd.map(lambda r: r[0])
        raw_lines_with_index = raw_lines_rdd.zipWithIndex()
        preview = raw_lines_with_index.take(20) # Check first 20 lines for header

        def score(line_tuple):
            line = line_tuple[0]
            return line.count(",") + line.count('"')

        # Find the best candidate for header
        if preview: # Ensure preview is not empty
            header_line = max(preview, key=score)
            header_index = header_line[1]
            header_raw = header_line[0]
            header_fields = [col.strip().strip('"').replace('\ufeff', '') for col in header_raw.split(",")]
            num_fields = len(header_fields)
            print(f"📌 Header detected in {file_name} at line {header_index}: {header_fields}")
        else:
            header_fields = []
            num_fields = 0
            print(f"⚠️ Could not detect header for {file_name}. Assuming no header.")

        # Dynamically create a schema for the inferred case, including _corrupt_record
        dynamic_schema_fields = [StructField(f"col_{i}", StringType(), True) for i in range(num_fields)]
        if header_fields: # If header was detected, use those names
             dynamic_schema_fields = [StructField(name, StringType(), True) for name in header_fields]
        
        # Always add _corrupt_record for robust error capture
        dynamic_schema_fields.append(StructField("_corrupt_record", StringType(), True))
        dynamic_schema = StructType(dynamic_schema_fields)

        # Read the data, skipping the header line if detected
        df = spark.read.format("csv") \
            .option("header", "true" if header_index == 0 and header_fields else "false") \
            .option("mode", "PERMISSIVE") \
            .option("delimiter", ",") \
            .schema(dynamic_schema) \
            .load(path)
        
        # Adjust rows_after_header based on detected header index
        rows_after_header = total_raw_rows - (header_index + 1) if header_index is not None else total_raw_rows

    # Ensure df is not None before caching and counting
    if df is None:
        raise ValueError(f"DataFrame could not be loaded for {file_name}.")

    # Cache the DataFrame to allow querying _corrupt_record without re-reading raw files
    df.cache()

    # Calculate valid and invalid rows using the _corrupt_record column
    valid_rows = df.filter(col("_corrupt_record").isNull()).count()
    invalid_rows = df.filter(col("_corrupt_record").isNotNull()).count()

    # Unpersist the DataFrame to release cached data
    df.unpersist()

    # Drop the _corrupt_record column before returning the DataFrame
    # This ensures the final DataFrame schema matches the original desired output
    if "_corrupt_record" in df.columns:
        df = df.drop("_corrupt_record")

    # If a header was detected and read as part of the data (because header=False was used
    # or if the header was explicitly skipped via temp file), remove the first row
    # if it matches the expected header values.
    # This is particularly relevant for files where header_index > 0.
    if header_index > 0 and df.count() > 0:
        # Get the header fields that were actually used for the DataFrame columns
        current_df_columns = df.columns
        # Read the raw header line again to compare
        raw_header_line = spark.read.text(path).rdd.map(lambda r: r[0]).zipWithIndex().filter(lambda x: x[1] == header_index).map(lambda x: x[0]).collect()
        if raw_header_line:
            header_values = [col_val.strip().strip('"').replace('\ufeff', '') for col_val in raw_header_line[0].split(',')]
            # Check if the first row of the DataFrame matches the header values
            # This is a heuristic and might need refinement for edge cases
            first_row_values = [str(c) for c in df.head()]
            if len(first_row_values) == len(header_values) and all(fr == hv for fr, hv in zip(first_row_values, header_values)):
                print(f"ℹ️ Removing detected header row from DataFrame for {file_name}.")
                df = df.filter(col(current_df_columns[0]) != header_values[0]) # Filter out based on first column value

    return df, {
        "raw_rows": total_raw_rows,
        "rows_after_header": rows_after_header,
        "valid_rows": valid_rows,
        "invalid_rows": invalid_rows,
        "filename": file_name
    }

# --- 4. Main Script Execution ---

# Setup
parsed_dfs = {}
successfully_loaded = []
failed_to_load = []

successfully_written = []
failed_to_write = []

processing_summary = [] # List to store summary data for the final report

TARGET_SCHEMA = "aims_dbo4"  # Define your target schema name
TABLE_SAVE_MODE = "overwrite"       # or "append"

# Define your base path for input CSV files
# This is a placeholder, replace with your actual OneLake path if different
input_base_path = "Files/" 

# Define your output path for temporary files and Delta tables
# This is a placeholder, replace with your actual OneLake path if different
output_path = "abfss://abc64232-25a2-499d-90ae-9fe5939ae437@onelake.dfs.fabric.microsoft.com/61b591cb-dc34-45e9-bc7f-b0e8c426e39a/Files/v2_processed_raw_source_data_aims2025/"


# Get all CSV files from the input base path
csv_files = list_csv_files_recursive(input_base_path)
print(f"📁 Found {len(csv_files)} CSV files to process.")

# STEP 1: Load and Process all CSVs
print("\n--- Starting CSV File Processing ---")
for path in csv_files:
    file_name = os.path.basename(path)
    
    # Get the predefined schema for the current file, if it exists
    schema_to_use = FILE_SCHEMAS.get(file_name)

    try:
        print(f"\n🔍 Inspecting: {path}")
        df, stats = detect_header_and_read_csv(path, predefined_schema=schema_to_use)
        
        parsed_dfs[path] = df
        successfully_loaded.append(path)
        
        print(f"📊 Rows in CSV after header (before loading into DataFrame): {stats['rows_after_header']}")
        print(f"✅ Valid rows loaded into DataFrame: {stats['valid_rows']}")
        print(f"❌ Invalid rows detected: {stats['invalid_rows']}")
        
        # Check for discrepancy based on valid rows vs. rows after header
        if stats['rows_after_header'] != stats['valid_rows']:
            print(f"⚠️ Discrepancy: Valid row count ({stats['valid_rows']}) differs from rows after header ({stats['rows_after_header']}).")
            print("This indicates malformed rows were dropped or set to null during parsing.")
        else:
            print("✅ No significant discrepancy detected between rows after header and valid rows.")

        print("--- Inferred/Applied Schema ---")
        df.printSchema()
        print("--- Sample Data (Top 2 Rows) ---")
        df.show(2)
        
        # Add status to stats for summary
        stats["status"] = "SUCCESS"
        processing_summary.append(stats)

    except Exception as e:
        print(f"❌ Error loading {path}: {e}")
        failed_to_load.append((path, str(e)))
        # Add failed status to stats for summary, with default counts
        processing_summary.append({
            "raw_rows": 0,
            "rows_after_header": 0,
            "valid_rows": 0,
            "invalid_rows": 0,
            "filename": file_name,
            "status": "FAILED"
        })

print("\n--- Finished CSV File Processing ---")

# STEP 2: Write to Delta tables
print("\n--- Starting Data Write to Delta Tables ---")
for path, df in parsed_dfs.items():
    table_name = os.path.splitext(os.path.basename(path))[0].replace(".", "_").lower()
    full_table_name = f"{TARGET_SCHEMA}.{table_name}"

    try:
        print(f"\n💾 Writing to table: {full_table_name}")
        df.write.format("delta").mode(TABLE_SAVE_MODE).saveAsTable(full_table_name)
        print(f"✅ Successfully written: {full_table_name}")
        successfully_written.append(full_table_name)
    except Exception as e:
        print(f"❌ Failed to write {full_table_name}: {e}")
        failed_to_write.append((full_table_name, str(e)))

print("\n--- Finished Data Write to Delta Tables ---")

# STEP 3: Summary Reports
print("\n📊 LOAD SUMMARY")
print(f"✅ Loaded: {len(successfully_loaded)} files")
print(f"❌ Failed to load: {len(failed_to_load)} files")
if failed_to_load:
    for path, err in failed_to_load:
        print(f"   - {path} → {err}")

print("\n📊 WRITE SUMMARY")
print(f"✅ Written to tables: {len(successfully_written)}")
print(f"❌ Failed to write: {len(failed_to_write)} files")
if failed_to_write:
    for tbl, err in failed_to_write:
        print(f"   - {tbl} → {err}")

# STEP 4: Detailed Processing Summary Table
print("\n--- Detailed Processing Summary ---")
header = [
    "File Name",
    "Raw Rows",
    "Rows After Header",
    "Valid Rows",
    "Invalid Rows",
    "Delta (%)",
    "Status"
]
# Determine maximum column widths for formatting
col_widths = {
    "File Name": max(len(s["filename"]) for s in processing_summary) if processing_summary else len("File Name"),
    "Raw Rows": max(len(str(s["raw_rows"])) for s in processing_summary) if processing_summary else len("Raw Rows"),
    "Rows After Header": max(len(str(s["rows_after_header"])) for s in processing_summary) if processing_summary else len("Rows After Header"),
    "Valid Rows": max(len(str(s["valid_rows"])) for s in processing_summary) if processing_summary else len("Valid Rows"),
    "Invalid Rows": max(len(str(s["invalid_rows"])) for s in processing_summary) if processing_summary else len("Invalid Rows"),
    "Delta (%)": len("Delta (%)"), # Fixed width for percentage, or dynamically calculate based on max float string length
    "Status": max(len(s["status"]) for s in processing_summary) if processing_summary else len("Status")
}

# Adjust header widths if they are longer than any data
for h in header:
    col_widths[h] = max(col_widths[h], len(h))

# Print header
header_str = " | ".join([f"{h:<{col_widths[h]}}" for h in header])
print(header_str)
print("-" * len(header_str))

# Print data rows
for stats in processing_summary:
    delta_percent = 0.0
    if stats["rows_after_header"] > 0:
        delta_percent = ((stats["rows_after_header"] - stats["valid_rows"]) / stats["rows_after_header"]) * 100
    
    row_data = [
        f"{stats['filename']:<{col_widths['File Name']}}",
        f"{stats['raw_rows']:>{col_widths['Raw Rows']}}",
        f"{stats['rows_after_header']:>{col_widths['Rows After Header']}}",
        f"{stats['valid_rows']:>{col_widths['Valid Rows']}}",
        f"{stats['invalid_rows']:>{col_widths['Invalid Rows']}}",
        f"{delta_percent:>{col_widths['Delta (%)']}.2f}%",
        f"{stats['status']:<{col_widths['Status']}}"
    ]
    print(" | ".join(row_data))

print("\n--- End of Detailed Summary ---")

# # --- 5. Accessing Processed DataFrames ---
# print("\n--- Accessing Processed DataFrames ---")
# if parsed_dfs:
#     for file_path, df_processed in parsed_dfs.items():
#         file_name = os.path.basename(file_path)
#         print(f"\nAccessing DataFrame for: {file_name}")
#         print(f"DataFrame for {file_name} has {df_processed.count()} rows.")
#         print(f"Sample data for {file_name}:")
#         display(df_processed) # Changed from .show(5, truncate=False) to display()
# else:
#     print("No DataFrames were successfully processed to access.")

# print("\n--- End of DataFrame Access Demonstration ---")


StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, -1, Cancelled, , Cancelled)

In [ ]:
# --- 5. Accessing Processed DataFrames ---
print("\n--- Accessing Processed DataFrames ---")
if parsed_dfs:
    for file_path, df_processed in parsed_dfs.items():
        file_name = os.path.basename(file_path)
        print(f"\nAccessing DataFrame for: {file_name}")
        print(f"DataFrame for {file_name} has {df_processed.count()} rows.")
        print(f"Sample data for {file_name}:")
        display(df_processed) # Changed from .show(5, truncate=False) to display()
else:
    print("No DataFrames were successfully processed to access.")

print("\n--- End of DataFrame Access Demonstration ---")

StatementMeta(, 9ec8bdf6-6596-47e1-a204-99c040cbe50e, -1, Cancelled, , Cancelled)